In [4]:
import pandas as pd
import numpy as np
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers.openai_tools import PydanticToolsParser
from typing import List, Optional
from langchain.prompts import PromptTemplate
import string
from dotenv import load_dotenv
import os
import httpx
import json
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
key=

In [30]:
numbered_markdown = """
[1] 
[2] <PageNum=1 />
[3] <figure>
[4] 
[5] ![](figures/0)<!-- FigureContent="The image displays the word "Agenda" in large, bold white letters on a solid blue background." --></figure>
[6] 
[7] 
[8] Ordinary Meeting Wednesday, 24 July 2024
[9] 
[10] commencing at 9:00am
[11] 
[12] Sunshine Coast City Hall Chamber, 54 First Avenue, Maroochydore
[13] 
[14] <figure>
[15] 
[16] ![](figures/1)<!-- FigureContent="The image features the logo of the Sunshine Coast Council. The logo consists of stylized blue and teal waves or grass-like strokes to the left, representing a dynamic and natural theme. Next to the graphic element, the text "Sunshine Coast" is written in bold, dark gray letters. Below "Sunshine Coast," the word "COUNCIL" is written in a lighter gray and smaller font." --></figure>
[17] 
[18] 
[19] 
[20] <PageNum=2 />
[21] ORDINARY MEETING AGENDA
[22] 
[23] 24 JULY 2024
[24] 
[25] TABLE OF CONTENTS
[26] ===
[27] 
[28] | ITEM | SUBJECT || PAGE NO |
[29] | - | - | - | - |
[30] | 1 | DECLARATION OF OPENING || 7 |
[31] | 2 | WELCOME AND OPENING || 7 |
[32] | 3 | RECORD OF ATTENDANCE AND LEAVE OF ABSENCE || 7 |
[33] | 4 | RECEIPT AND CONFIRMATION OF MINUTES || 7 |
[34] | 5 | MAYORAL MINUTE || 7 |
[35] | 6 | INFORMING OF CONFLICTS OF INTEREST || 7 |
[36] | | 6.1 PRESCRIBED CONFLICTS OF INTEREST ...... || 7 |
[37] | | 6.2 DECLARABLE CONFLICTS OF INTEREST || .7 |
[38] | 7 | PRESENTATIONS / COUNCILLOR REPORTS || 8 |
[39] | 8 | REPORTS DIRECT TO COUNCIL || 9 |
[40] | | 8.1 JUNE 2024 FINANCIAL PERFORMANCE REPORT. || 9 |
[41] | | 8.2 2024 LOCAL GOVERNMENT ASSOCIATION OF QUEENSLAND ANNUAL CONFERENCE MOTIONS || 27 |
[42] | | 8.3 | COUNCIL OF MAYORS SOUTH EAST QUEENSLAND - PROPOSED INTERNATIONAL DELEGATION | 45 |
[43] | | 8.4 | 2024 SPORTS FIELD MAINTENANCE FUNDING PROGRAM | 53 |
[44] | | 8.5 EXCEPTION UNDER LOCAL GOVERNMENT REGULATION 2012 FOR THE DISPOSAL OF AN INTEREST IN LAND, BELLS CREEK || 61 |
[45] | | 8.6 SUNSHINE COAST LIGHTNING SPONSORSHIP AGREEMENT || 73 |
[46] | | 8.7 VISIT SUNSHINE COAST FUNDING AND PERFORMANCE DEED || 79 |
[47] | 9 | NOTIFIED MOTIONS || 85 |
[48] 
[49] <!-- PageFooter="Sunshine Coast Regional Council" -->
[50] 
[51] <!-- PageNumber="OM Agenda Page 2 of 88" -->
[52] 
[53] 
[54] <PageNum=3 />
[55] <!-- PageHeader="ORDINARY MEETING AGENDA 24 JULY 2024" -->
[56] 
[57] 10 TABLING OF PETITIONS 85
[58] 
[59] 11
[60] 
[61] CONFIDENTIAL SESSION
[62] 
[63] 87
[64] 
[65] 11.1
[66] 
[67] CONFIDENTIAL - NOT FOR PUBLIC RELEASE -
[68] 
[69] STRATEGIC LAND ACQUISITION - MOUNT MELLUM
[70] 
[71] 87
[72] 
[73] 12
[74] 
[75] NEXT MEETING
[76] 
[77] 88
[78] 
[79] 13
[80] 
[81] MEETING CLOSURE
[82] 
[83] 88
[84] 
[85] <!-- PageFooter="Sunshine Coast Regional Council" -->
[86] 
[87] <!-- PageFooter="OM Agenda Page 3 of 88" -->
[88] 
[89] 
[90] <PageNum=4 />
[91] 
[92] 
[93] <PageNum=5 />
[94] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[95] 
[96] 24 JULY 2024
[97] 
[98] <figure>
[99] 
[100] ![](figures/2)<!-- FigureContent="The image is the logo of the Sunshine Coast Council. It features a stylized design on the left, which consists of blue and green abstract shapes that look like waves or tropical leaves. To the right of this design, the text "Sunshine Coast" is written in bold, black font. Underneath this, the word "COUNCIL" is displayed in a smaller, gray font. The logo appears to represent a coastal community, focusing on the theme of natural beauty and municipal governance." --></figure>
[101] 
[102] 
[103] ORDINARY MEETING NOTICE
[104] 
[105] 12 July 2024
[106] 
[107] Dear Councillors, Group Executives and relevant staff, In accordance with Section 254C(2) of the Local Government Regulation 2012, I wish to advise that an Ordinary Meeting has been convened for
[108] 
[109] 24 July 2024 commencing at 9.00am.
[110] 
[111] <figure>
[112] 
[113] ![](figures/3)<!-- FigureContent="The image shows part of a handwritten signature. The writing is done in blue ink. The signature features a curling loop on the left side, followed by what appears to be the initials "A" and "M." The strokes are smooth and flowing, suggesting the handwriting may be from a cursive or stylized script." --></figure>
[114] 
[115] 
[116] Debra Robinson | Acting Chief Executive Officer
[117] 
[118] Sunshine Coast Regional Council 54 First Avenue, Maroochydore QLD 4558 P 07 5441 8240 | F 07 5441 8275 E debra.robinson@sunshinecoast.gld.gov.au W www.sunshinecoast.qld.gov.au
[119] 
[120] <!-- PageFooter="Sunshine Coast Regional Council" -->
[121] 
[122] <!-- PageFooter="OM Agenda Page 5 of 88" -->
[123] 
[124] 
[125] <PageNum=6 />
[126] 
[127] 
[128] <PageNum=7 />
[129] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[130] 
[131] <!-- PageHeader="24 JULY 2024" -->
[132] 
[133] 1 DECLARATION OF OPENING
[134] 
[135] On establishing there is a quorum, the Chair will declare the meeting open.
[136] 
[137] 2 WELCOME AND OPENING
[138] 
[139] 3 RECORD OF ATTENDANCE AND LEAVE OF ABSENCE
[140] 
[141] 4 RECEIPT AND CONFIRMATION OF MINUTES
[142] 
[143] That the Minutes of the Ordinary Meeting held on 20 June 2024 and the Special Meeting held on 20 June 2024 be received and confirmed.
[144] 
[145] 5 MAYORAL MINUTE
[146] 
[147] 6 INFORMING OF CONFLICTS OF INTEREST
[148] 
[149] 
[150] ## 6.1 PRESCRIBED CONFLICTS OF INTEREST
[151] 
[152] Pursuant to section 150EL of the Local Government Act 2009 (the Act), a Councillor who has a prescribed conflict of interest in an issue to be considered at a meeting of the local government must -
[153] 
[154] (a) immediately inform the meeting of the prescribed conflict of interest including the particulars stated in section 150EL(4) of the Act and
[155] 
[156] (b) pursuant to section 150EM(2) of the Act must leave the place at which the meeting is being held, including any area set aside for the public, and stay away from the place while the matter is being discussed and voted on.
[157] 
[158] 
[159] ## 6.2 DECLARABLE CONFLICTS OF INTEREST
[160] 
[161] Pursuant to section 150EQ of the Local Government Act 2009, a Councillor who has a declarable conflict of interest in a matter to be considered at a meeting of the local government, must stop participating in the meeting and immediately inform the meeting of the declarable conflict of interest including the particulars stated in section 150EQ(4) of the Act.
[162] 
[163] If the Councillor with a declarable conflict of interest does not voluntarily decide not to participate in the decision, pursuant to section 150ES(3)(a) of the Act the eligible Councillors must, by resolution, decide
[164] 
[165] (a) whether the Councillor may participate in the decision despite the Councillors conflict of interest or
[166] 
[167] (b) that the Councillor must not participate in the decision and must leave the place at which the meeting is being held, including any area set aside for the public and stay away while the eligible Councillors discuss and vote on the matter.
[168] 
[169] The Councillor with the declarable conflict of interest must comply with any conditions the eligible Councillors impose per section 150ES(4) and (5) of the Act.
[170] 
[171] <!-- PageFooter="Sunshine Coast Regional Council" -->
[172] 
[173] <!-- PageNumber="OM Agenda Page 7 of 88" -->
[174] 
[175] 
[176] <PageNum=8 />
[177] ORDINARY MEETING AGENDA
[178] 
[179] 24 JULY 2024
[180] 
[181] 7 PRESENTATIONS / COUNCILLOR REPORTS
[182] 
[183] <!-- PageFooter="Sunshine Coast Regional Council" -->
[184] 
[185] <!-- PageNumber="OM Agenda Page 8 of 88" -->
[186] 
[187] 
[188] <PageNum=9 />
[189] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[190] 
[191] <!-- PageHeader="24 JULY 2024" -->
[192] 
[193] 
[194] ## 8 REPORTS DIRECT TO COUNCIL
[195] 
[196] 
[197] ### 8.1 JUNE 2024 FINANCIAL PERFORMANCE REPORT
[198] 
[199] |||
[200] | - | - |
[201] | File No: | Council Meetings |
[202] | Author: | Acting Coordinator Financial Services Business Performance Group |
[203] | Attachments: | Att 1 - June 2024 Financial Performance Report . 15 ↓ Adebt |
[204] || Att 2 - Capital Grant Funded Project Report June 2024 25 l |
[205] 
[206] 
[207] ## PURPOSE
[208] 
[209] To meet Council's legislative obligations, a monthly report must be presented to Council on its financial performance and investments.
[210] 
[211] 
[212] ## EXECUTIVE SUMMARY
[213] 
[214] This monthly financial performance report provides Council with a summary of performance against budget as at 30 June 2024, in terms of the operating result and delivery of the capital program.
[215] 
[216] 
[217] ## Operating Performance
[218] 
[219] Table 1: Operating Budget as at 30 June 2024
[220] 
[221] | | Original Budget $000 | Current Budget $000 |
[222] | - | - | - |
[223] | Total Operating Revenue | 590,615 | 595,258 |
[224] | Total Operating Expenses | 566,003 | 573,241 |
[225] | Operating Result | 24,612 | 22,017 |
[226] 
[227] Details of the monthly financial report are contained in Attachment 1.
[228] 
[229] 
[230] ## OFFICER RECOMMENDATION
[231] 
[232] That Council receive and note the report titled "June 2024 Financial Performance Report"
[233] 
[234] 
[235] ## FINANCE AND RESOURCING
[236] 
[237] This report sets out the details of Council's financial performance and investments for the month ending 30 June 2024, and meets Council's legislative reporting requirements.
[238] 
[239] <!-- PageFooter="Sunshine Coast Regional Council" -->
[240] 
[241] <!-- PageFooter="OM Agenda Page 9 of 88" -->
[242] 
[243] 
[244] <PageNum=10 />
[245] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[246] 
[247] <!-- PageHeader="24 JULY 2024" -->
[248] 
[249] 
[250] ## CORPORATE PLAN
[251] 
[252] Corporate Plan Goal: Outcome: Operational Activity:
[253] 
[254] Our outstanding organisation
[255] 
[256] We serve our community by providing this great service S28 - Financial and procurement services - financial and procurement management and governance, ensuring effective business management and legislative compliance, coordination and development of Council's budget process, administration of financial systems, sundry debtors, accounts payable, financial and asset accounting, treasury, procurement, contract and supply functions.
[257] 
[258] 
[259] ## CONSULTATION
[260] 
[261] Councillor Consultation
[262] 
[263] Consultation has been undertaken with the Portfolio Councillors, E Hungerford and J Broderick.
[264] 
[265] Internal Consultation
[266] 
[267] This report has been written in conjunction with advice from:
[268] 
[269] • Acting Group Executive Business Performance
[270] 
[271] • Acting Chief Financial Officer
[272] 
[273] External Consultation
[274] 
[275] No external consultation is required for this report.
[276] 
[277] Community Engagement
[278] 
[279] No community engagement is required for this report.
[280] 
[281] Legal
[282] 
[283] This report ensures that Council complies with its legislative obligations with respect to financial reporting in accordance with Section 204 of the Local Government Regulation 2012.
[284] 
[285] Investment of funds is in accordance with the provisions of the Statutory Bodies Financial Arrangements Act 1982 and the associated Regulations and the Local Government Act 2009.
[286] 
[287] Policy
[288] 
[289] Sunshine Coast Council's 2023-24 Investment Policy, and Sunshine Coast Council's 2023-24 Debt Policy.
[290] 
[291] Risk
[292] 
[293] Failure to achieve the budgeted operating result will negatively impact Council's capacity to complete its capital expenditure program.
[294] 
[295] <!-- PageFooter="Sunshine Coast Regional Council" -->
[296] 
[297] <!-- PageFooter="OM Agenda Page 10 of 88" -->
[298] 
[299] 
[300] <PageNum=11 />
[301] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[302] 
[303] <!-- PageHeader="24 JULY 2024" -->
[304] 
[305] Previous Council Resolution
[306] 
[307] Ordinary Meeting 14 December 2023 (OM23/149)
[308] 
[309] That Council:
[310] 
[311] (a) receive and note the report titled "Budget Review 2 2023/2024"
[312] 
[313] (b) adopt Appendix A as tabled, pursuant to sections 169 and 170 of the Local Government Regulation 2012, Council's amended budget for 2023/24 financial year incorporating:
[314] 
[315] (i) the statement of income and expenditure
[316] 
[317] (ii) the statement of financial position
[318] 
[319] (iii) the statements of changes in equity
[320] 
[321] (iv) the statement of cash flow
[322] 
[323] (v) the relevant measures of financial sustainability
[324] 
[325] (vi) the long-term financial forecast
[326] 
[327] (vii) Council's 2023/24 Capital Works Program, endorse the indicative four-year program for the period 2025 to 2028, and note the five-year program for the period 2029 to 2033
[328] 
[329] (c) note the following documentation applies as adopted 22 June 2023
[330] 
[331] (i) the Debt Policy
[332] 
[333] (ii) the Revenue Policy
[334] 
[335] (iii) the total value of the change, expressed as a percentage, in the rates and utility charges levied for the financial year compared with the rates and utility charges levied in the previous budget, and
[336] 
[337] (iv) the Revenue Statement
[338] 
[339] (v) the rates and charges to be levied for the 2023/24 financial year and other matters as adopted 22 June 2023
[340] 
[341] (vi) the Strategic Environment Levy Policy
[342] 
[343] (vii) the Strategic Arts and Heritage Levy Policy
[344] 
[345] (viii) the Strategic Transport Levy Policy
[346] 
[347] (ix) the Derivatives Policy and
[348] 
[349] (d) endorse the Minor Capital Works Program (Appendix B).
[350] 
[351] Ordinary Meeting 21 September 2023 (OM23/104) That Council:
[352] 
[353] (a) receive and note the report titled "Budget Review 1 - 2023/24" and
[354] 
[355] (b) adopt Appendix A as tabled, pursuant to sections 169 and 170 of the Local Government Regulation 2012, Council's amended budget for 2023/24 financial year incorporating:
[356] 
[357] i. the statement of income and expenditure
[358] 
[359] <!-- PageFooter="Sunshine Coast Regional Council" -->
[360] 
[361] <!-- PageNumber="OM Agenda Page 11 of 88" -->
[362] :selected:
[363] 
[364] <PageNum=12 />
[365] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[366] 
[367] 24 JULY 2024
[368] 
[369] ii. the statement of financial position
[370] 
[371] iii. the statement of changes in equity
[372] 
[373] iv. the statement of cash flow
[374] 
[375] V. the relevant measurers of financial sustainability
[376] 
[377] vi. the long-term financial forecast
[378] 
[379] vii. Council's 2023/24 Capital Works Program, endorse the indicative four-year program for a period 2025 to 2028, and note the five-year program for the period 2029 to 2033
[380] 
[381] (c) note the following documentation applies as adopted 22 June 2023
[382] 
[383] i. the Debt policy
[384] 
[385] ii. the Revenue policy
[386] 
[387] iii. the total value of change, expressed as a percentage, in the rates and utility charges levied for the financial year compared with the rates and utility charges levied in the previous budget
[388] 
[389] iv. the Revenue statement
[390] 
[391] V. the rates and charges to be levied for the 2023/24 financial year and other matters as adopted 22 June 2023
[392] 
[393] vi. the Strategic Environment Levy Policy
[394] 
[395] vii. the Strategic Arts and Heritage Levy Policy
[396] 
[397] viii. the Strategic Transport Levy Policy
[398] 
[399] ix. the Derivates Policy and
[400] 
[401] (d) endorse the Minor Capital Works Program (Appendix B)
[402] 
[403] 
[404] ## Special Meeting 22 June 2023 (SM23/2)
[405] 
[406] That Council:
[407] 
[408] 1\. STATEMENT OF ESTIMATED FINANCIAL POSITION
[409] 
[410] receive and note Appendix A, pursuant to section 205 of the Local Government Regulation 2012, the statement of the financial operations and financial position of the Council in respect to the 2022/23 financial year
[411] 
[412] 
[413] ## 2\. ADOPTION OF BUDGET
[414] 
[415] adopt Appendix A as tabled, pursuant to sections 169 and 170 of the Local Government Regulation 2012, Council's budget for 2023/24 financial year incorporating:
[416] 
[417] i. the statement of income and expenditure
[418] 
[419] ii. the statement of financial position
[420] 
[421] iii. the statement of changes in equity
[422] 
[423] iv. the statement of cash flow
[424] 
[425] <!-- PageFooter="Sunshine Coast Regional Council" -->
[426] 
[427] <!-- PageNumber="OM Agenda Page 12 of 88" -->
[428] 
[429] 
[430] <PageNum=13 />
[431] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[432] 
[433] 24 JULY 2024
[434] 
[435] V. the relevant measures of financial sustainability
[436] 
[437] vi. the long-term financial forecast
[438] 
[439] vii. the Debt Policy (adopted by Council resolution on 25 May 2023)
[440] 
[441] viii. the Revenue Policy (adopted by Council resolution on 25 May 2023)
[442] 
[443] ix. the total value of the change, expressed as a percentage, in the rates and utility charges levied for the financial year compared with the rates and utility charges levied in the previous budget
[444] 
[445] X. the Revenue Statement
[446] 
[447] xi. Council's 2023/24 Capital Works Program, endorse the indicative four-year program for the period 2025 to 2028, and note the five-year program for the period 2029 to 2033
[448] 
[449] xii. the rates and charges to be levied for the 2023/24 financial year and other matters as detailed below in clauses 3 to 10
[450] 
[451] xiii. the 2023/24 Minor Capital Works Program
[452] 
[453] xiv. the Strategic Environment Levy Policy
[454] 
[455] XV.
[456] 
[457] the Strategic Arts and Heritage Levy Policy
[458] 
[459] xvi.
[460] 
[461] the Strategic Transport Levy Policy and
[462] 
[463] xvii. the Derivatives Policy
[464] 
[465] Related Documentation
[466] 
[467] 2023-24 Adopted Budget
[468] 
[469] Critical Dates
[470] 
[471] There are no critical dates for this report.
[472] 
[473] Implementation
[474] 
[475] There are no implementation details to include in this report.
[476] 
[477] <!-- PageFooter="Sunshine Coast Regional Council" -->
[478] 
[479] <!-- PageNumber="OM Agenda Page 13 of 88" -->
[480] :selected:
[481] 
[482] <PageNum=14 />
[483] 
[484] 
[485] <PageNum=15 />
[486] 24 JULY 2024
[487] 
[488] Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 1 June 2024 Financial Performance Report
[489] 
[490] <figure>
[491] 
[492] ![](figures/4)<!-- FigureContent="The image is a cover page for the "2023-24 Budget" of the Sunshine Coast Council. It features the title "Financial Performance Report – Interim Report" dated June 2024. The background is dark teal, with the word "BUDGET" prominently displayed in large white letters. To the left of the word "BUDGET" is a small green leaf icon.
[493] 
[494] The lower left corner of the image has the logo and text for the Sunshine Coast Council. The logo consists of stylized curved lines forming a wave-like design, accompanied by the words "Sunshine Coast COUNCIL."
[495] 
[496] On the right side of the image, there is a series of colorful, curved lines creating a rainbow-like arc, framing a small section of a photograph. The photograph shows a scenic view of mountains against a sunset or sunrise backdrop, with clouds and a warm sky." --></figure>
[497] 
[498] 
[499] <!-- PageFooter="Sunshine Coast Regional Council" -->
[500] 
[501] <!-- PageNumber="OM Agenda Page 15 of 88" -->
[502] 
[503] 
[504] <PageNum=16 />
[505] <!-- PageHeader="24 JULY 2024" -->
[506] 
[507] <!-- PageHeader="Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 1 June 2024 Financial Performance Report" -->
[508] 
[509] Statement of Income and Expenses
[510] ===
[511] 
[512] <figure>
[513] 
[514] <figcaption>
[515] 
[516] 2023/24 Operating Result
[517] 
[518] </figcaption>
[519] 
[520] ![](figures/5)<!-- FigureContent="This image is a bar chart titled "2023/24 Operating Result." The chart shows a comparison between Year-To-Date (YTD) Budget and YTD Actuals in millions of dollars for each month in the fiscal year, from July to June. The YTD Budget is represented by gray bars, and the YTD Actuals are represented by blue bars.
[521] 
[522] Here is a summary of the data:
[523] 
[524] - **July**: The YTD Budget and YTD Actuals are both around $180 million, with the YTD Actuals slightly higher.
[525] - **August**: The YTD Budget is slightly below $140 million, while the YTD Actuals are slightly above $140 million.
[526] - **September**: The YTD Budget is around $100 million, whereas the YTD Actuals are approximately $120 million.
[527] - **October**: The YTD Budget and YTD Actuals are both around $90 million, with the YTD Actuals slightly higher.
[528] - **November**: The YTD Budget is around $80 million, and the YTD Actuals are slightly above $80 million.
[529] - **December**: The YTD Budget is around $20 million, while the YTD Actuals are around $10 million.
[530] - **January**: Both the YTD Budget and YTD Actuals are slightly below $180 million, with the YTD Actuals being marginally higher.
[531] - **February**: The YTD Budget is slightly below $140 million, and the YTD Actuals are slightly above $140 million.
[532] - **March**: The YTD Budget and YTD Actuals are both around $100 million, with the YTD Actuals slightly higher.
[533] - **April**: The YTD Budget is around $90 million, and the YTD Actuals are slightly above $80 million.
[534] - **May**: The YTD Budget is slightly above $60 million, and the YTD Actuals are slightly below $50 million.
[535] - **June**: The YTD Budget is around $20 million, while the YTD Actuals are nearly $0.
[536] 
[537] The chart demonstrates various differences between the budgeted and actual operating results across the 2023/24 fiscal year." --></figure>
[538] 
[539] 
[540] At the end of June 2024, Council has an interim operating result of $500,000, revenue is below budget by $22.4 million, and expenses are below budget by $909,000 resulting in a $21.5 million unfavourable variance.
[541] 
[542] There are further end of financial year processes still to complete, which will result in changes to the Operating Result and Capital Works Program.
[543] 
[544] | Statement of Income & Expenses ||||||||
[545] | | Annual || YTD |||| Annual |
[546] | | Original Budget | Current Budget | Current Budget | Actuals | Variance | Variance | Forecast Budget $000 |
[547] || $000 | $000 | $000 | $000 | $000 | % ||
[548] | - | - | - | - | - | - | - | - |
[549] | Operating Revenue | | | | | | | |
[550] | General Rates | 298,265 | 298,265 | 298,265 | 298,367 | 102 | 0.0% | 319,728 |
[551] | Cleansing Charges | 78,328 | 78,328 | 78,328 | 78,328 | (0) | (0.0%) | 78,328 |
[552] | Levies | 28,756 | 30,352 | 30,352 | 29,804 | (548) | (1.8%) | 29,552 |
[553] | Fees and Charges | 82,335 | 79,254 | 79,254 | 73,403 | (5,851) | (7.4%) | 77,152 |
[554] | Interest Received from Investments | 13,850 | 15,350 | 15,350 | 13,994 | (1,356) | (8.8%) | 14,883 |
[555] | Operating Grants and Subsidies | 14,977 | 18,537 | 18,537 | 4,688 | (13,849) | (74.7%) | 16,129 |
[556] | Operating Contributions | 320 | 320 | 320 | 268 | (52) | (16.3%) | 320 |
[557] | Unitywater Participation | 52,438 | 52,438 | 52,438 | 52,476 | 38 | 0.1% | 52,500 |
[558] | Other Revenue | 17,787 | 18,436 | 18,436 | 20,306 | 1,870 | 10.1% | 18,775 |
[559] | Internal Sales/Recoveries | 3,560 | 3,977 | 3,977 | 1,197 | (2,780) | (69.9%) | 2,781 |
[560] | Total Operating Revenue | 590,615 | 595,258 | 595,258 | 572,831 | (22,427) | (3.8%) | 610,147 |
[561] | Operating Expenses | | | | | | | |
[562] | Employee Costs | 176,203 | 176,994 | 176,994 | 179,490 | 2,496 | 1.4% | 186,126 |
[563] | Materials and Services | 235,600 | 242,231 | 242,231 | 232,373 | (9,859) | (4.1%) | 234,321 |
[564] | Finance Costs | 13,137 | 12,809 | 12,809 | 12,410 | (399) | (3.1%) | 13,284 |
[565] | Company Contributions | 4,657 | 4,657 | 4,657 | 4,656 | (0) | (0.0%) | 4,149 |
[566] | Depreciation Expense | 105,674 | 105,674 | 105,674 | 105,747 | 73 | 0.1% | 115,664 |
[567] | Other Expenses | 26,732 | 26,874 | 26,874 | 27,642 | 767 | 2.9% | 29,587 |
[568] | Recurrent Capital Expenses | 4,000 | 4,000 | 4,000 | 10,013 | 6,013 | 150.3% | 4,000 |
[569] | Total Operating Expenses | 566,003 | 573,240 | 573,240 | 572,332 | (909) | (0.2%) | 587,130 |
[570] | Operating Result | 24,612 | 22,017 | 22,017 | 499 | (21,518) | (97.7%) | 23,017 |
[571] | Capital Revenue | | | | | | | |
[572] | Capital Grants and Subsidies | 70,836 | 99,758 | 99,758 | 63,384 | (36,373) | (36.5%) | - |
[573] | Capital Contributions - Cash | 32,384 | 32,481 | 32,481 | 28,341 | (4,141) | (12.7%) | - |
[574] | Capital Contributions - Fixed Assets | 82,820 | 82,820 | 82,820 | 18,439 | (64,381) | (77.7%) | - |
[575] | Total Capital Revenue | 186,040 | 215,059 | 215,059 | 110,163 | (104,896) | (48.8%) | - |
[576] | Non-recurrent Expenses | | | | | | | |
[577] | Profit/Loss on disposal, revaluation & | | | | | | | |
[578] | impairment | - | - | - | 4,286 | 4,286 | - | - |
[579] | Movements in landfill and quarry provisions | 2,982 | 2,982 | 2,982 | 2,982 | - | - | 2,982 |
[580] | Recurrent Capital Expenses - Prior Year | - | - | - | 6,523 | 6,523 | - | - |
[581] | Assets Transferred to Third Parties | - | - | - | - | - | - | - |
[582] | Total Non-recurrent Expenses | 2,982 | 2,982 | 2,982 | 13,790 | 10,808 | 362.5% | 2,982 |
[583] | Net Result | 207,671 | 234,095 | 234,095 | 96,873 | (137,222) | (58.6%) | 20,035 |
[584] 
[585] <!-- PageFooter="Sunshine Coast Regional Council" -->
[586] 
[587] <!-- PageFooter="OM Agenda Page 16 of 88" -->
[588] 
[589] 
[590] <PageNum=17 />
[591] <!-- PageHeader="24 JULY 2024" -->
[592] 
[593] Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 1 June 2024 Financial Performance Report
[594] 
[595] Operating Result - June 2024
[596] ===
[597] 
[598] <figure>
[599] 
[600] ![](figures/6)<!-- FigureContent="This is a horizontal bar chart titled "Operating Revenue," showing various sources of revenue in millions of dollars. 
[601] 
[602] The revenue sources listed on the vertical axis, from top to bottom, are:
[603] 1. Fees & Charges
[604] 2. Interest Received from Investments
[605] 3. Grants and Subsidies - Recurrent
[606] 4. Interest Received from Unitywater
[607] 5. Other Revenue
[608] 6. Internal Revenue
[609] 
[610] Each revenue source is represented by two bars: one in grey and one in blue. 
[611] 
[612] Here's a summary of the values for each revenue source:
[613] - **Fees & Charges**: Approximately 80 million dollars.
[614] - **Interest Received from Investments**: About 18 million dollars.
[615] - **Grants and Subsidies - Recurrent**: Approximately 13 million dollars.
[616] - **Interest Received from Unitywater**: Roughly 40 million dollars.
[617] - **Other Revenue**: About 25 million dollars.
[618] - **Internal Revenue**: Approximately 4 million dollars." --></figure>
[619] 
[620] 
[621] YTD Budget
[622]  :selected:
[623] YTD Actuals
[624]  :selected:
[625] | Substantial Revenue Variance for the Period Ending June 2024 |||||||
[626] | | Annual || YTD ||||
[627] | | Original Budget | Current Budget | Current Budget | Actuals | Variance | Variance |
[628] || $000 | $000 | $000 | $000 | $000 | % |
[629] | - | - | - | - | - | - | - |
[630] | Operating Revenue | | | | | | |
[631] | Fees and Charges | 82,335 | 79,254 | 79,254 | 73,403 | (5,851) | (7.4%) |
[632] | Operating Grants and Subsidies | 14,977 | 18,537 | 18,537 | 4,688 | (13,849) | (74.7%) |
[633] 
[634] | Operating Summary |||||||
[635] | | Annual || YTD ||||
[636] || Original Budget | Current Budget | Current Budget | Actuals | Variance | Variance |
[637] || $000s | $000s | $000s | $000s | $000s | % |
[638] | - | - | - | - | - | - | - |
[639] | Operating Revenue | 590,615 | 595,258 | 595,258 | 572,831 | (22,427) | (3.8%) |
[640] | Operating Expenses | 562,003 | 569,240 | 569,240 | 562,319 | (6,921) | (1.2%) |
[641] | Recurrent Capital Expenses | 4,000 | 4,000 | 4,000 | 10,013 | 6,013 | 150.3% |
[642] | Operating Result | 24,612 | 22,017 | 22,017 | 499 | (21,518) | (97.7%) |
[643] | Capital Revenue | 186,040 | 215,059 | 215,059 | 110,163 | (104,896) | (48.8%) |
[644] | Non-recurrent Expenses | 2,982 | 2,982 | 2,982 | 13,790 | 10,808 | 362.5% |
[645] | Net Result | 207,671 | 234,095 | 234,095 | 96,873 | (137,222) | (58.6%) |
[646] 
[647] As at 30 June 2024, $572.8 million operating revenue has been achieved resulting in a variance of $22.4 million (3.8%) below budget.
[648] 
[649] Significant revenue variances include:
[650] 
[651] . Operating Grants and Subsidies - The early payment of the 2024-25 Financial Assistance Grant was received 2 July, and therefore cannot be recognised in the 2023-24 financial year, resulting in $10.5m shortfall in the budget. For the past 8 financial years the early payment for the new financial year grant allocation has consistently been received in June. DRFA Street Trees Grant not received 2023-24 as expected $3.2m below budget.
[652] 
[653] . Fees and Charges - Plumbing applications and inspections $3.0 million below budget, Quarry Services sales $2.5 million below budget, Caloundra Music Festival $1.6 million below budget. Holiday Parks above budget $1.7m.
[654] 
[655] . Other Revenue - Additional revenue received for environmental offsets with commercial properties lease revenue higher than budget plus Sale of electricity easements to Energex providing a one-time boost to the bottom line.
[656] 
[657] · Capital Revenue - capital contributed assets below budget as end of year adjustments still to be completed. Capital grants $36.4 million below budget, DRFA funding revenue not received as forecast.
[658] 
[659] <!-- PageFooter="Sunshine Coast Regional Council" -->
[660] 
[661] <!-- PageFooter="OM Agenda Page 17 of 88" -->
[662] 
[663] 
[664] <PageNum=18 />
[665] <!-- PageHeader="24 JULY 2024" -->
[666] 
[667] Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 1 June 2024 Financial Performance Report
[668] 
[669] Operating Result - June 2024 (continued)
[670] ===
[671] 
[672] <figure>
[673] 
[674] <figcaption>
[675] 
[676] Operating Expenses
[677] 
[678] </figcaption>
[679] 
[680] ![](figures/7)<!-- FigureContent="The image is a bar chart displaying various categories of operating expenses, with a caption titled "Operating Expenses." The categories listed on the y-axis include:
[681] 
[682] 1. Employee Costs
[683] 2. Materials & Services
[684] 3. Finance Costs
[685] 4. Company Contributions
[686] 5. Depreciation
[687] 6. Other Expenses
[688] 7. Recurrent Expenditure
[689] 
[690] The x-axis measures expenses in millions of dollars, ranging from 0 to 250. For each category, there are two bars: one for "YTD Budget" (shown in gray) and one for "YTD Actuals" (shown in blue).
[691] 
[692] Observations:
[693] - **Employee Costs** and **Materials & Services** are the highest expenses, both having budgeted and actual values nearing the 200 million dollar mark.
[694] - **Depreciation** also has significant expenses, with the actual value slightly below the budgeted value around the 125 million dollar mark.
[695] - **Other Expenses** and **Finance Costs** are considerably lower, with Finance Costs being the lowest.
[696] - **Recurrent Expenditure** shows minimal values, while **Company Contributions** have slightly more, though still relatively low compared to other categories.
[697] 
[698] Overall, the chart provides a clear comparison of budgeted versus actual expenses for different operating cost categories." --></figure>
[699] 
[700]  :selected:
[701] 3
[702] 
[703] | Operating Summary |||||||
[704] | | Annual || YTD ||||
[705] || Original Budget | Current Budget | Current Budget | Actuals | Variance | Variance |
[706] || $000s | $000s | $000s | $000s | $000s | % |
[707] | - | - | - | - | - | - | - |
[708] | Operating Revenue | 590,615 | 595,258 | 595,258 | 572,831 | (22,427) | (3.8%) |
[709] | Operating Expenses | 562,003 | 569,240 | 569,240 | 562,319 | (6,921) | (1.2%) |
[710] | Recurrent Capital Expenses | 4,000 | 4,000 | 4,000 | 10,013 | 6,013 | 150.3% |
[711] | Operating Result | 24,612 | 22,017 | 22,017 | 499 | (21,518) | (97.7%) |
[712] | Capital Revenue | 186,040 | 215,059 | 215,059 | 110,163 | (104,896) | (48.8%) |
[713] | Non-recurrent Expenses | 2,982 | 2,982 | 2,982 | 13,790 | 10,808 | 362.5% |
[714] | Net Result | 207,671 | 234,095 | 234,095 | 96,873 | (137,222) | (58.6%) |
[715] 
[716] | Substantial Expenditure Variance for the Period Ending June 2024 |||||||
[717] | | Annual || YTD ||||
[718] | | Original Budget | Current Budget | Current Budget | Actuals | Variance | Variance |
[719] || $000 | $000 | $000 | $000 | $000 | % |
[720] | - | - | - | - | - | - | - |
[721] | Operating Expenses | | | | | | |
[722] | Employee Costs | 176,203 | 176,994 | 176,994 | 179,490 | 2,496 | 1.4% |
[723] | Materials and Services | 235,600 | 242,231 | 242,231 | 232,373 | (9,859) | (4.1%) |
[724] 
[725] As at 30 June 2024, $572 million had been expended resulting in a variance of $909,000 (0.2%) below budget.
[726] 
[727] Materials and Services - Lower than expected contractor expenses have been realised throughout various areas of Council resulting in lower than budget expenditure in Waste $2.4 million, Blue Carbon Grant expenditure $1.5 million, Solar Farm energy charges $1.9 million, Quarry Services $2.5 million and Caloundra Music Festival $649,000.
[728] 
[729] . Employee Costs - Overtime $1.6 million above budget and level of allocation to capital works $950,000 less than expected. Further end of year adjustments for employee provisions still to be completed.
[730] 
[731] . Other expenses - Reactive maintenance expenses for council fleet vehicles is $291,000 higher than budget.
[732] 
[733] <!-- PageFooter="Sunshine Coast Regional Council" -->
[734] 
[735] <!-- PageFooter="OM Agenda Page 18 of 88" -->
[736] 
[737] 
[738] <PageNum=19 />
[739] <!-- PageHeader="24 JULY 2024" -->
[740] 
[741] <!-- PageHeader="Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 1 June 2024 Financial Performance Report" -->
[742] 
[743] Capital Expenditure - June 2024
[744] ===
[745] 
[746] SCC Core Capital Expenditure
[747] 
[748] <figure>
[749] 
[750] ![](figures/8)<!-- FigureContent="The image is a bar and line graph displaying "SCC Core Capital Expenditure." The y-axis represents the expenditure in millions of dollars, ranging from 0 to 160, while the x-axis lists the months from July to June.
[751] 
[752] - **Bar Graph (light blue bars):** Represents the "Monthly Actuals" of capital expenditure for each month.
[753]   - The bars seem to show a varying pattern with specific months like October, November, and June showing higher expenditures compared to months like January and February.
[754] 
[755] - **Line Graph (dotted grey line):** Represents the "Forecast" of capital expenditure for each month.
[756]   - The forecast line appears to demonstrate a steady upward trend over the year.
[757] 
[758] - **Line Graph (solid blue line with circles):** Represents the "Actuals" of capital expenditure for each month.
[759]   - The actual expenditure generally shows an increasing trend over the months, starting at below $20 million in July and reaching above $140 million by June.
[760] 
[761] The data illustrates that the actual expenditures have been gradually increasing and often correspond closely with the forecasted values, especially towards the end of the period." --></figure>
[762] 
[763] 
[764] . On 30 June 2024, $303.7 million (91.3%) of Council's $332.6 million Capital Works Program had been financially expended.
[765] 
[766] · The Core Capital Program has progressed 79% of budget, an actual spend of $134 million.
[767] 
[768] · Corporate Major Projects has progressed 94%
[769] 
[770] • A range of significant projects continue to progress with construction during 2024 including
[771] 
[772] •
[773] 
[774] Caloundra Centre Activation - Library
[775] 
[776] • Honey Farm Road Sports and Recreation Precinct works
[777] 
[778] • Stringybark Road Pedestrian and Cycle Bridge
[779] 
[780] • Charles Clarke Park Revetment Wall
[781] 
[782] • Outrigger Park Boat Ramp and Carpark
[783] 
[784] | Capital Expenditure |||||||
[785] | | Annual || YTD || Year End ||
[786] || Original Budget $000s | Current Budget $000s | Actuals $000s | % of FY Budget Spent | Forecast Year End Actual | Forecast Year End Variance to Budget |
[787] | - | - | - | - | - | - | - |
[788] | Core Capital Works Program | | | | | | |
[789] | Aerodromes | 845 | 995 | 1,151 | 115.7% | 1,151 | 156 |
[790] | Buildings and Facilities | 31,331 | 20,905 | 17,025 | 81.4% | 17,025 | (3,880) |
[791] | Coast and Canals | 2,087 | 6,548 | 3,273 | 50.0% | 3,273 | (3,275) |
[792] | Environmental Assets | 3,497 | 3,857 | 2,892 | 75.0% | 2,892 | (965) |
[793] | Minor Works | 5,020 | 5,726 | 3,459 | 60.4% | 3,459 | (2,267) |
[794] | Holiday Parks | 2,556 | 1,643 | 1,271 | 77.4% | 1,271 | (372) |
[795] | Parks and Gardens | 12,176 | 12,024 | 12,613 | 104.9% | 12,613 | 589 |
[796] | Sports Facilities | 19,438 | 10,621 | 9,884 | 93.1% | 9,884 | (737) |
[797] | Quarries | - | - | 62 | - | 62 | 62 |
[798] | Stormwater | 15,515 | 11,230 | 10,327 | 92.0% | 10,327 | (903) |
[799] | Transportation | 87,862 | 96,062 | 71,710 | 74.6% | 71,710 | (24,352) |
[800] | Total SCC Core Capital Program | 180,328 | 169,612 | 133,668 | 78.8% | 133,668 | (35,943) |
[801] | Disaster Recovery Funding Arrangements | 40,089 | 41,553 | 36,527 | 87.9% | 36,527 | (5,026) |
[802] | Fleet | 5,242 | 4,000 | 3,326 | 83.2% | 3,326 | (674) |
[803] | Information Technology | 10,000 | 9,631 | 7,278 | 75.6% | 7,278 | (2,353) |
[804] | Waste | 25,203 | 49,503 | 47,734 | 96.4% | 47,734 | (1,768) |
[805] | Corporate Major Projects | 32,975 | 28,007 | 26,297 | 93.9% | 26,297 | (1,710) |
[806] | Strategic Land and Commercial Properties | 36,880 | 30,075 | 47,345 | 157.4% | 47,345 | 17,270 |
[807] | Maroochydore City Centre | - | 275 | 120 | 43.5% | 120 | (155) |
[808] | Sunshine Coast Airport Runway | - | - | 1,398 | - | 1,398 | 1,398 |
[809] | Total Other Capital Program | 150,389 | 163,044 | 170,026 | 104.3% | 170,026 | 6,982 |
[810] | TOTAL | 330,716 | 332,656 | 303,694 | 91.3% | 303,694 | (28,962) |
[811] 
[812] The above program of works includes recurrent and non-recurrent expenditure, as reported in the operating statement
[813] 
[814] <!-- PageFooter="Sunshine Coast Regional Council" -->
[815] 
[816] <!-- PageFooter="OM Agenda Page 19 of 88" -->
[817] 
[818] 20
[819] 
[820] 15
[821] 
[822] 10
[823] 
[824] 5
[825] 
[826] $ Monthly Millions
[827] 
[828] 
[829] <PageNum=20 />
[830] <!-- PageHeader="24 JULY 2024" -->
[831] 
[832] <!-- PageHeader="Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 1 June 2024 Financial Performance Report" -->
[833] 
[834] Capital Expenditure - June 2024 (continued)
[835] ===
[836] 
[837] <figure>
[838] 
[839] <figcaption>
[840] 
[841] % YTD spent compared to profiled budget by Program
[842] 
[843] </figcaption>
[844] 
[845] ![](figures/9)<!-- FigureContent="The image is a bar chart titled “% YTD spent compared to profiled budget by Program.” The chart compares the Year-to-Date (YTD) spending actuals against budget forecasts for various programs. 
[846] 
[847] **Legend:**
[848] - The light blue bars represent "YTD Actual" spend.
[849] - The grey bar lines represent "Year-End Forecast."
[850] - The green line represents the "Portion of Year Passed."
[851] - The red line represents the “Year-End Target.”
[852] 
[853] **Programs:**
[854] The programs listed along the x-axis include:
[855] 1. Buildings & Facilities
[856] 2. Coast & Canals
[857] 3. Environmental Assets
[858] 4. Holiday Parks
[859] 5. Parks and Gardens
[860] 6. Sports Facilities
[861] 7. Stormwater
[862] 8. Transportation
[863] 9. Fleet
[864] 10. Information Technology
[865] 11. Waste
[866] 12. Strategic Land and Comm. Prop.
[867] 
[868] **Observations:**
[869] - Most programs have YTD actual spending below the 100% mark, indicating they are under-budget year-to-date.
[870] - The "Sports Facilities" program has a YTD spending close to the 100% mark.
[871] - The "Waste" program significantly exceeds the 100% mark in YTD spending, approaching or exceeding 160%.
[872] - The green and red lines are aligned near the 100% mark, indicating the proportional year passed and year-end target, respectively.
[873] " --></figure>
[874] 
[875] 
[876] 
[877] ## Coast and Canals
[878] 
[879] • $2.3 million budget for Charles Clarke Park, works started February 2024, 81% expensed.
[880] 
[881] 
[882] ## Building and Facilities
[883] 
[884] $20.9 million budget including community and corporate facilities has progressed 81%, with a spend of $17 million
[885] 
[886] Fleet
[887] 
[888] • $4 million budget, with $3.3 million of fleet received so far
[889] 
[890] 
[891] ## Parks and Gardens
[892] 
[893] • $3.6 million Coastal Pathways has progressed 141% with a spend of $5.1 million with higher than budgeted costs for pathway between beach access 233 to 229 in Warana.
[894] 
[895] $7.2 million Recreational Parks program has progressed 80% with a spend of $5.8 million
[896] 
[897] 
[898] ### Sports Facilities
[899] 
[900] • $5.8 million Honey Farm Road Project has progressed 84% with a spend of $4.9 million
[901] 
[902] 
[903] ## Strategic Land and Commercial Properties
[904] 
[905] • Acquisitions and negotiations for land associated with Council's trunk road infrastructure for Sugar Road and Maud Street Upgrade and Caloundra Transport Corridor Upgrade.
[906] 
[907] 
[908] ## Transportation
[909] 
[910] • Council's $25 million road resurfacing and rehabilitation program has progressed 98% with a spend of $24.7 million
[911] 
[912] • Council's $9.8 million Unsealed Road Upgrades program has progressed 102% with a spend of $10 million
[913] 
[914] Council's $1.2 million bus stop program has progressed 89% with a spend of $1.1 million
[915] 
[916] Pedestrian and Cycling Enabling Facilities has progressed 94% with a spend of $3.8 million •
[917] 
[918] • Disaster Recovery Funded Projects have progressed 84%, $5.4 million under budget
[919] 
[920] • Streetscape Program 60% progressed, $7.7 millon under budget - First Avenue $4.1 million
[921] 
[922] • WOR LED Streetlighting Upgrade $3.6 million under budget - delivery by Energex in 2024-25
[923] 
[924] 
[925] ## Waste
[926] 
[927] · Waste Program is 96% spent with construction complete on the new Material Resource Facility (MRF) and the facility is now operational at the Nambour Waste Precinct
[928] 
[929] The Landfill Expansion at Nambour has expensed $13.9 million this financial year
[930] 
[931] <!-- PageFooter="Sunshine Coast Regional Council" -->
[932] 
[933] <!-- PageFooter="OM Agenda Page 20 of 88" -->
[934] :unselected: :unselected: :unselected: :unselected:
[935] 
[936] <PageNum=21 />
[937] <!-- PageHeader="24 JULY 2024" -->
[938] 
[939] <!-- PageHeader="Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 1 June 2024 Financial Performance Report" -->
[940] 
[941] Cash Flows and Balance Sheet
[942] ===
[943] 
[944] <figure>
[945] 
[946] ![](figures/10)<!-- FigureContent="The image consists of two line charts:
[947] 
[948] 1. **Cash Balance Chart (Top)**
[949]    - **Y-Axis:** Represents cash balance in millions of dollars, ranging from $100 million to $400 million.
[950]    - **X-Axis:** Represents time, from June 2018 to June 2024.
[951]    - **Description:** The line graph shows fluctuations in cash balance over time. It starts around $300 million and exhibits a cyclical pattern with notable peaks and troughs. The balance seems to hover mostly between $200 million and $300 million.
[952] 
[953] 2. **Net Cash Chart (Bottom)**
[954]    - **Y-Axis:** Represents net cash in millions of dollars, ranging from -$600 million to $100 million.
[955]    - **X-Axis:** Represents time, from June 2018 to June 2024.
[956]    - **Description:** The line graph shows a downward trend in net cash starting at around $80 million in June 2018 and bottoming out at around -$500 million. There is some fluctuation within that range, with peaks and troughs similar to the cash balance chart.
[957] 
[958] The data in both charts exhibit volatility and periodic fluctuation trends over the observed time period." --></figure>
[959] 
[960] 
[961] | Cash and Balance Sheet 2023-24 ||
[962] | | Current Budget $000s |
[963] | - | - |
[964] | CASH FLOWS | |
[965] | Opening Cash | 260,120 |
[966] | Net Cash Inflow/(Outflows) from: | |
[967] | Operating Activities | 85,682 |
[968] | Investing Activities | (144,892) |
[969] | Financing Activities | 4,457 |
[970] | Net Increase/(decrease) in Cash Held | (54,753) |
[971] | Cash at year end | 205,367 |
[972] | BALANCE SHEET | |
[973] | Current Assets | 279,734 |
[974] | Non Current Assets | 7,332,142 |
[975] | Total Assets | 7,611,876 |
[976] | Current Liabilities | 178,915 |
[977] | Non Current Liabilities | 509,749 |
[978] | Total Liabilities | 688,664 |
[979] | Net Community Assets/Total Community Equity | 6,923,212 |
[980] 
[981] • Council's cash at 30 June 2024 is $194 million
[982] 
[983] · Council's debt at 30 June 2024 is $434 million
[984] 
[985] <!-- PageFooter="Sunshine Coast Regional Council" -->
[986] 
[987] <!-- PageFooter="OM Agenda Page 21 of 88" -->
[988] 
[989] 
[990] <PageNum=22 />
[991] Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 1 June 2024 Financial Performance Report
[992] 
[993] Debt
[994] ===
[995] 
[996] Sunshine Coast Council's debt program is governed by the 2023/24 Debt Policy, which was adopted with the Original Budget adoption on 22 June 2023.
[997] 
[998] New borrowings are undertaken in accordance with the Queensland Treasury Corporation Guidelines, the Statutory Bodies Financial Arrangements Act 1982 and Section 192 of the Local Government Regulation 2012.
[999] 
[1000] Council's borrowings for the 2023-24 financial year are $28.5 million and relate to:
[1001] 
[1002] . $26.6 million for Waste
[1003] 
[1004] . $1.9 million for Holiday Parks
[1005] 
[1006] These borrowings were drawn down during the month of June.
[1007] 
[1008] Council's current debt as at 30 June 2024 is $434 million. Council repays debt twice yearly in August and February.
[1009] 
[1010] | Debt - 2023/24 |||||
[1011] | | Opening Balance $000 | Debt| Redemption Borrowings $000  New $000 | Closing Balance $000 |
[1012] | - | - | - | - | - |
[1013] | Sunshine Coast Council Core | 335,286 | 18,103 | 28,542 | 345,724 |
[1014] | Maroochydore City Centre | 106,481 | 6,042 | - | 100,439 |
[1015] | Total | 441,767 | 24,145 | 28,542 | 446,164 |
[1016] 
[1017] Table 1: 2023/24 Adopted Debt Balance
[1018] 
[1019] 
[1020] ## Debt Balance
[1021] 
[1022] <figure>
[1023] 
[1024] ![](figures/11)<!-- FigureContent="The image is a line graph depicting data over time, extending from June 2018 to June 2024. The y-axis represents dollar amounts in millions, ranging from $100 million to $800 million. Key observations from the graph:
[1025] 
[1026] 1. From June 2018 to around June 2019, the value fluctuates around $300 million.
[1027] 2. A gradual increase is observed from approximately June 2019, reaching around $500 million by June 2020.
[1028] 3. The value experiences further growth, peaking at around $700 million by June 2021.
[1029] 4. A steep decline is seen around June 2022, dropping to about $500 million, and the value remains relatively steady with slight declines till June 2024, where it shows a minor uptick.
[1030] 
[1031] Overall, the graph exhibits an initial period of stability, growth peaking in 2021, followed by a decline and stabilization in the subsequent years." --></figure>
[1032] 
[1033] 
[1034] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1035] 
[1036] <!-- PageFooter="OM Agenda Page 22 of 88" -->
[1037] 
[1038] <!-- PageHeader="24 JULY 2024" -->
[1039] 
[1040] 
[1041] <PageNum=23 />
[1042] <!-- PageHeader="24 JULY 2024" -->
[1043] 
[1044] <!-- PageHeader="Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 1 June 2024 Financial Performance Report" -->
[1045] 
[1046] 
[1047] ## Investment Performance
[1048] 
[1049] Investment per financial institution (%)
[1050] 
[1051] BOQ
[1052] 
[1053] <figure>
[1054] 
[1055] ![](figures/12)<!-- FigureContent="The image is a pie chart representing the market shares of different financial institutions. The market shares are divided as follows:
[1056] 
[1057] - **QTC (light blue)** holds the largest share at 54%.
[1058] - **Westpac/St George (light green)** has 18%.
[1059] - **CBA (teal)** has 14%.
[1060] - **NAB (brown)** holds 9%.
[1061] - There is an additional segment labeled as **3%**, which is not associated with any particular institution in the provided description.
[1062] 
[1063] Each sector is visually distinguishable by different colors to represent the respective percentages of each institution." --></figure>
[1064] 
[1065] 
[1066] Investment by Standard & Poor's (Short Term Credit Rating)
[1067] 
[1068] <figure>
[1069] 
[1070] ![](figures/13)<!-- FigureContent="This is a pie chart with five sections, each labeled with a category and its corresponding percentage. The chart is divided as follows:
[1071] 
[1072] - "A1" represents 0% and is not visually represented, suggesting insignificance.
[1073] - "A2" represents 4%, depicted as a small light teal segment.
[1074] - "A3" represents 0% and is not visually represented, suggesting insignificance.
[1075] - "A1+ (Other)" represents 42%, depicted as a large maroon segment.
[1076] - "A1+(QTC)" represents 54%, depicted as the largest light blue segment.
[1077] 
[1078] The segments are differentiated by color and labeled accordingly to show their proportionate shares." --></figure>
[1079] 
[1080] 
[1081] Investment Performance - June 2024
[1082] 
[1083] | Liquidity as at: | | 30/06/2024 |
[1084] | | $'000's | |
[1085] | - | - | - |
[1086] | At-call accounts | | |
[1087] | QTC + CBA (excl. trust) | 124,164 | 56.88% |
[1088] | Maturities within 7 days | - | 0.00% |
[1089] | Total at-call | 124,164 | 56.88% |
[1090] | Investment Policy Target | | 10.00% |
[1091] 
[1092] | Term deposits maturing: || |
[1093] | | $'000's | Count |
[1094] | - | - | - |
[1095] | within 30 days | 30,000 | 3 |
[1096] | 30-59 days | - | - |
[1097] | 60-89 days | 30,000 | 3 |
[1098] | 90-179 days | 10,000 | 1 |
[1099] | 180-364 days | - | - |
[1100] | 1 year - 3 years | - | - |
[1101] | Total | 70,000 | 7 |
[1102] 
[1103] | INVESTMENT SUMMARY (including Trust) as at: ||||||| Investment Policy ||
[1104] | | 30/06/2024 || 31/03/2024 || 30/06/2023 || Individual Limit | Group Limits |
[1105] | - | - | - | - | - | - | - | - | - |
[1106] | A1+ (QTC) | 117,489 | 54% | 92,953 | 30.7% | 75,591 | 36.8% | 100% | 100% |
[1107] | A1+ (Other) | 90,812 | 42% | 190,045 | 62.7% | 157,707 | 38.9% | 100% | 100% |
[1108] | A1 | - | 0% | - | 0.0% | 30,000 | 14.6% | 60% | 100% |
[1109] | A2 | 10,000 | 5% | 20,000 | 6.6% | 20,000 | 9.7% | 60% | 90% |
[1110] | A3 | - | 0% | - | 0.0% | - | 0.0% | 10% | 30% |
[1111] | Total Funds | 218,300 | | 302,998 | | 283,298 | | | |
[1112] | FUND SUMMARY | | | | | | | | |
[1113] | General Funds | 194,164 | | 279,522 | | 271,971 | | | |
[1114] | Trust Funds | 24,136 | | 23,476 | | 11,327 | | | |
[1115] | Total Funds | 218,300 | | 302,998 | | 283,298 | | | |
[1116] 
[1117] . All investment parameters remain within the guidelines established by the Investment Policy.
[1118] 
[1119] . For the month ending 30 June 2024 Council had $194 million cash (excluding Trust Fund), with an average interest rate of 5.01%, being 0.81% above benchmark. This is compared to the same period last year with $272 million cash (excluding Trust Fund) with an average interest rate of 4.67%, being 1.12% above benchmark.
[1120] 
[1121] . The benchmark used to measure performance of cash funds is Bloomberg AusBond Bank Bill index (BAUBIL).
[1122] 
[1123] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1124] 
[1125] <!-- PageFooter="OM Agenda Page 23 of 88" -->
[1126] 
[1127] 
[1128] <PageNum=24 />
[1129] Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 1 June 2024 Financial Performance Report
[1130] 
[1131] <figure>
[1132] 
[1133] ![](figures/14)<!-- FigureContent="The image contains a predominantly dark blue background. There is a series of curved lines on the right side of the image, running parallel to each other and moving towards the lower right corner. These lines are colored green, light blue, turquoise, purple, magenta, orange, and yellow.
[1134] 
[1135] In the lower right corner, there is a small, rectangular sub-image with rounded corners. This sub-image features a landscape, showing a range of mountainous silhouettes against a sky highlighted by the warm colors of either a sunrise or sunset with some cloud cover. The overall design gives a modern and dynamic feel." --></figure>
[1136] 
[1137] 
[1138] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1139] 
[1140] <!-- PageNumber="OM Agenda Page 24 of 88" -->
[1141] 
[1142] 24 JULY 2024
[1143] 
[1144] 
[1145] <PageNum=25 />
[1146] 24 JULY 2024
[1147] 
[1148] Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 2 Capital Grant Funded Project Report June 2024
[1149] 
[1150] 2023/24 Financial Year Grant Funding
[1151] 
[1152] | | Description | Division | Suburb | Estimated Construction Start Month | Construction Completed Month | TOTAL Grant Revenue $'000 | Project Expenditure to date 000.5 |
[1153] | - | - | - | - | - | - | - | - |
[1154] | || | | | | | |
[1155] | Federal Government || | | | | ($14,682) | $6,281 |
[1156] | | | | | | | | |
[1157] | 1 | Roads to Recovery Program | | | | | ($3,644) | |
[1158] | | | | | | | | |
[1159] | Local Road and Community Infrastructure Grant Funding - Round 2 - Agreement end date 30 June 2022 |||| | | ($400) | $442 |
[1160] | 2 | H3799 - LRCIP2 - Zgrajewski Road Gravel Road Upgrade | Division 09 | Yandina Creek | January 2022 | November 2022 | ($400) | $442 |
[1161] | | | | | | | | |
[1162] | Local Road and Community Infrastructure Grant Funding - Round 3 - Agreement end date 30 June 2023 |||| | | ($770) | $774 |
[1163] | 3 | H5465 - LRCIP3 Lake Weyba Foreshore Trail Upgrade | Division 09 | Lake Weyba | | | ($370 | $399 |
[1164] | 4 | K2732 - LRCIP3 WOR LED Streetlighting Upgrades | Whole of Council | | | June 2024 | ($400) | $376 |
[1165] | | | | | | | | |
[1166] | | Black Summer Bushfire Recover Grants Program | | | | | ($459) | $804 |
[1167] | 5 | K3402 - Nambour Showgrounds PA System | Division 10 | Nambour | | | ($459) | $804 |
[1168] | | | | | | | | |
[1169] | | Urban Congestion Fund | | | | | ($4,337) | $4,260 |
[1170] | 6 | H3747 - Mooloolaba Transport Corridor - D and C | Division 04 | Mooloolaba | | November 2023 | ($4,337) | $4,260 |
[1171] | | | | | | | | |
[1172] | 7 | Recycling Modernisation Fund | | | | | ($5,072) | |
[1173] | | | | | | | | |
[1174] | State Government || | | | | ($79,580) | $15,134 |
[1175] | | | | | | | | |
[1176] | 8 | Disaster Recovery Funding Arrangements | | | | | ($41,806) | |
[1177] | | | | | | | | |
[1178] | | Blackspot Funding | | | | | ($2,802) | $746 |
[1179] | 9 | K3443 - Blackspot Old Gympie Road Beerwah to Landsborough | Division 01 | Beerwah | | | ($66) | $46 |
[1180] | 10 | K3444 - Blackspot Ilkley Road Ilkley - Hazard Signage and Delineation Upgrade | Division 05 | Ilkley | | | ($258) | $21 |
[1181] | 11 | K3445 - Blackspot Brisbane Road Mooloolaba - Signal Upgrade | Division 04 | Mooloolaba | February 2024 | March 2024 | ($77) | $20 |
[1182] | 12 | K4894 - Blackspot - Beerburrum Street and Cooroy | Division 02 | Dicky Beach | March 2024 | July 2024 | ($1,305) | $221 |
[1183] | 13 | K4895 - BlackSpot - Old Gympie Road | Division 01 | Beerwah | May 2024 | June 2024 | ($1,000) | $59 |
[1184] | 14 | K2085 - BlackSpot Edmund Street and William Street Intersection | Division 02 | Shelly Beach | | | ($97) | $181 |
[1185] | | | | | | | | |
[1186] | | Local Government Alliance | | | | | ($11) | $0 |
[1187] | 15 | B1514 - TMR Local Government Alliance | Whole of Council | | | | ($11) | SO |
[1188] | | | | | | | | |
[1189] | | 2022-24 Local Government Grants and Subsidies Program | | | | | ($686) | $2,697 |
[1190] | 16 | H5637 - Caloundra Headland Coastal Pathway | Division 02 | Kings Beach | May 2023 | July 2024 | ($686) | $2,697 |
[1191] | | | | | | | | |
[1192] | | Queensland Transport MOU | | | | | ($1,620) | $11 |
[1193] | 17 | H8786 - Design and Permits Outrigger Boat Ramp | Division 04 | Minyama | | | ($1,620) | $111 |
[1194] | | | | | | | | |
[1195] | | Queensland Transport Cycle Network Program | | | | | ($2,819) | $2,205 |
[1196] | 18 | H3839 - LGIP Stringybark Rd Footbridge-Pathway | Division 07 | Sippy Downs | September 2023 | December 2023 | ($2,800) | $2,130 |
[1197] | 19 | H4851 - LGIP Ballinger Rd Active Transport Link | Division 07 | Buderim | | | ($19) | $76 |
[1198] | | | | | | | | |
[1199] | | Community Recreational Assets Recovery Resilience Program | | | | | ($750) | $814 |
[1200] | 20 | H1977 - CRARRP - Maroochy Multi-Sports Development | Division 07 | Maroochydore | | | ($297) | $302 |
[1201] | 21 | K4238 - CRARRP - Maroochydore Regional Football | Division 07 | Kunda Park | | | ($40) | $80 |
[1202] | 22 | K4240 - CRARRP - Suncoast Hinterland BMX Club | Division 05 | Landsborough | September 2023 | September 2023 | ($81) | $42 |
[1203] | 23 | K4243 - CRARRP - Sugar Bag Eastern Trail Repairs | Division 02 | Caloundra West | | | ($279) | $363 |
[1204] | 24 | K4244 - CRARRP - Parklands Conservation Park Trails Restoration | Division 09 | Bli Bli | | | ($52 | $28 |
[1205] | | | | | | | | |
[1206] | 25 | Passenger Transport Accessible Infrastructure Program | | | | | ($330) | |
[1207] | | | | | | | | |
[1208] | 26 | Passenger Transport Infrastructure Investment Program | | | | | ($250) | |
[1209] | | | | | | | | |
[1210] | | Minor Infrastructure Program | | | | | ($840) | $1,802 |
[1211] | 27 | K3519 - Nambour Yandina United Football Club change rooms designs | Division 10 | Yandina | | | ($590) | $1,647 |
[1212] | 28 | H3892 - Lions-Norrie Job Park, Coolum Landscape Plan | Division 09 :selected: | Coolum Beach | | | ($250) | $155 |
[1213] | | | | | | | | |
[1214] | | School Transport Infrastructure Program | | | | | ($940) | $1,107 |
[1215] | 29 | K4380 - STIP - Chancellor State College - Columbia Street | Division 06 :selected: :selected: | Sippy Downs | November 2023 | February 2024 | ($300) | $400 |
[1216] | 30 | K4382 - STIP - Currimundi State School Buderim | Division 03 :selected: | Battery Hill | | | ($253 | $282 |
[1217] | 31 | K4383 - STIP - Buderim Mountain State School Lindsay Road | Division 07 :unselected: | Buderim | March 2024 | May 2024 | ($205 | $200 |
[1218] | 32 | K4384 - STIP - Peachester Primary School Peachester | Division 01 :unselected: | Peachester | | | ($182) | $226 |
[1219] | | | | | | | | |
[1220] 
[1221] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1222] 
[1223] <!-- PageNumber="OM Agenda Page 25 of 88" -->
[1224] 
[1225] 
[1226] <PageNum=26 />
[1227] 24 JULY 2024
[1228] 
[1229] Ordinary Meeting Agenda Item 8.1 June 2024 Financial Performance Report Attachment 2 Capital Grant Funded Project Report June 2024
[1230] 
[1231] 2023/24 Financial Year Grant Funding
[1232] 
[1233] | | Description | Division || Suburb | Estimated Construction Start Month | Construction Completed Month | TOTAL Grant Revenue $'000 | Project Expenditure to date $'000 |
[1234] | - | - | - | - | - | - | - | - | - |
[1235] | | | | | | | | | |
[1236] | | Transport Infrastructure Development Scheme | | | | | | ($1,964) | $1,172 |
[1237] | 33 | H4615 - Sippy Downs School Precinct | Division | 06 | Sippy Downs | | | ($690 | $294 |
[1238] | 34 | H4617 - Mooloolaba School Precinct | Division | 04 | Mooloolaba | | | ($10) | $22 |
[1239] | 35 | H8416 - Emu Mountain Road Pathway Construction | Division | 09 | Coolum Beach | | | ($120) | $111 |
[1240] | 36 | K2723 - Camp Flat Road Upgrade Stage 4 | Division | 09 | Bli Bli | September 2023 | | ($1,144) | $745 |
[1241] | | | | | | | | | |
[1242] | South East Queensland Community Stimulus Program - Agreement end date 30 June 2024 |||| | | | ($2,512) | $4,033 |
[1243] | 37 | H1670 - SEQCSP First Avenue Streetscape | Division | 04 | Maroochydore | February 2024 | June 2025 | ($910 | $234 |
[1244] | 38 | K1618 - SEQCSP Caloundra Indoor Sports Stadium - Roof Replacement | Division | 02 | Caloundra | July 2023 | December 2023 | ($357) | $2,552 |
[1245] | 39 | H7141 - SEQCSP Jack Morgan Park Amenities | Division | 04 | Minyama | | | ($178 | $49 |
[1246] | 40 | H7863 - SEQCSP Coolum Sports Complex intersection | Division | 09 | Coolum Beach | September 2023 | November 2023 | ($647) | $667 |
[1247] | 41 | H4602 - SEQCSP Woombye Streetscape Placemaking | Division | 05 | Woombye | October 2022 | April 2024 | ($420) | $85 |
[1248] | | | | | | | | | |
[1249] | | Qcoast 2100 | | | | | | ($250) | $446 |
[1250] | 42 | K4295 - Esplanade Golden Beach Foreshoare Rehabiliation | Division | 02 | Golden Beach | | | ($250 | $446 |
[1251] | | | | | | | | | |
[1252] | 43 | Recycling and Jobs Fund | | | | | | ($22,000) | |
[1253] 
[1254] Project Complete
[1255] 
[1256] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1257] 
[1258] <!-- PageNumber="OM Agenda Page 26 of 88" -->
[1259] 
[1260] 
[1261] <PageNum=27 />
[1262] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1263] 
[1264] <!-- PageHeader="24 JULY 2024" -->
[1265] 
[1266] 
[1267] ### 8.2 2024 LOCAL GOVERNMENT ASSOCIATION OF QUEENSLAND ANNUAL CONFERENCE MOTIONS
[1268] 
[1269] |||
[1270] | - | - |
[1271] | File No: | Council Meetings |
[1272] | Author: | Team Leader Councillor Liaison Civic Governance |
[1273] | Appendices: | App A - Motion - Personal Mobility Devices 35 L |
[1274] | Attachments: | Att 1 - Local Government Association of Queensland Conference Motions Status of Council Resolutions. 39 L |
[1275] | ||
[1276] 
[1277] 
[1278] ## PURPOSE
[1279] 
[1280] This report proposes one motion for consideration at the Local Government Association of Queensland's (LGAQ's) 2024 Annual Conference.
[1281] 
[1282] Draft motions are prepared for Council's review and endorsement prior to the Annual Conference to be held in October 2024.
[1283] 
[1284] 
[1285] ## EXECUTIVE SUMMARY
[1286] 
[1287] The 2024 LGAQ Annual Conference is to be held in Brisbane from 21 to 23 October.
[1288] 
[1289] Each year, the Conference debates motions put forward by both the LGAQ Policy Executive and member Councils. Motions that are supported and resolved at the Annual Conference guide and inform the LGAQ's Annual Advocacy Plan and the LGAQ Policy Statement.
[1290] 
[1291] Under the LGAQ Conference rules, any motion from a local government, must be endorsed by the submitting Council at a Council meeting. To be considered at the 2024 Annual Conference, all motions must be received by the LGAQ by 29 July 2024.
[1292] 
[1293] The Conference agenda will prioritise motions that raise new issues of state-wide relevance or an issue impacting a large portion or segment of the LGAQ membership. Motions should not relate to a matter considered at a previous Annual Conference in the last five years, unless seeking to alter or update the adopted policy position. Motions must also be well formed and presented in a way that is most likely to garner the support of the broader membership of the LGAQ - namely the 77 local governments in Queensland.
[1294] 
[1295] One motion is proposed for consideration by Sunshine Coast Regional Council to submit to the LGAQ for consideration and debate at the 2024 Conference. This motion - should it be endorsed by Council and subsequently resolved at the LGAQ Annual Conference - would provide greater impetus for introducing speed limitations for personal mobility devices.
[1296] 
[1297] The details of the motion, including background information and desired outcomes, are outlined in Appendix A.
[1298] 
[1299] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1300] 
[1301] <!-- PageFooter="OM Agenda Page 27 of 88" -->
[1302] 
[1303] 
[1304] <PageNum=28 />
[1305] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1306] 
[1307] <!-- PageHeader="24 JULY 2024" -->
[1308] 
[1309] OFFICER RECOMMENDATION
[1310] 
[1311] That Council:
[1312] 
[1313] (a) receive and note the report titled "Motions for the 2024 Local Government Association of Queensland Annual Conference" and
[1314] 
[1315] (b) endorse the following motion for forwarding to the Local Government Association of Queensland for inclusion in the 2024 Annual Conference agenda:
[1316] 
[1317] (i) That the Local Government Association of Queensland call on the State Government to introduce device limitations and standards, such as those for e-bikes, to control the speed and capabilities of personal mobility devices (Appendix A).
[1318] 
[1319] 
[1320] ## FINANCE AND RESOURCING
[1321] 
[1322] There are no financial or resourcing impacts for Council arising from submitting motions for consideration at the Local Government Association of Queensland Annual Conference.
[1323] 
[1324] CORPORATE PLAN
[1325] 
[1326] Corporate Plan Goal:
[1327] 
[1328] Our outstanding organisation
[1329] 
[1330] Outcome:
[1331] 
[1332] We serve our community by providing this great service
[1333] 
[1334] Operational Activity:
[1335] 
[1336] S27 - Elected Council - providing community leadership,
[1337] 
[1338] democratic representation, advocacy and decision-making.
[1339] 
[1340] 
[1341] ## CONSULTATION
[1342] 
[1343] Councillor Consultation
[1344] 
[1345] All Councillors were invited to consider subject areas for potential Conference motions.
[1346] 
[1347] As Strong Community portfolio holders, Councillor D Law and Councillor T Bunnag were invited to provide feedback into the motion requesting the consideration of introducing speed limitations for personal mobility devices.
[1348] 
[1349] 
[1350] ### Internal Consultation
[1351] 
[1352] Consultation has been undertaken with the Acting Chief Executive Officer and members of the Executive Leadership Team in framing the proposed Conference Motion which is the subject of this report. Specific engagement has also occurred with:
[1353] 
[1354] • Group Executive Liveability and Natural Assets
[1355] 
[1356] · Acting Group Executive Customer and Planning Services
[1357] 
[1358] • Manager Project Delivery
[1359] 
[1360] • Manager Customer Response
[1361] 
[1362] 
[1363] ### External Consultation
[1364] 
[1365] There has been no external consultation undertaken in relation to this report.
[1366] 
[1367] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1368] 
[1369] <!-- PageNumber="OM Agenda Page 28 of 88" -->
[1370] 
[1371] 
[1372] <PageNum=29 />
[1373] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1374] 
[1375] <!-- PageHeader="24 JULY 2024" -->
[1376] 
[1377] Community Engagement
[1378] 
[1379] Community engagement has not been required to formulate this report.
[1380] 
[1381] 
[1382] ## PROPOSAL
[1383] 
[1384] The next Local Government Association of Queensland Annual Conference is to be held in Brisbane from 21 to 23 October 2024.
[1385] 
[1386] The Local Government Association of Queensland Policy Executive and member Councils submit motions to the conference for debate. Motions supported at the Conference form part of the LGAQ's Annual Advocacy Plan and the LGAQ Policy Statement.
[1387] 
[1388] Local governments must first endorse motions before they are submitted to the LGAQ for inclusion in the Conference Agenda. All motions for this year's conference must be received by the LGAQ by 29 July 2024.
[1389] 
[1390] Issues raised at the conference should be of state-wide relevance and generally will call for action by another tier of government.
[1391] 
[1392] On 22 May 2024, Councillors and the Executive Leadership Team were asked to commence consideration of potential motions. On 3 June 2024, the Local Government Association of Queensland formally called for motions from member Councils.
[1393] 
[1394] One motion is proposed for consideration by Council and, if endorsed, the Local Government Association of Queensland Annual Conference delegates. The motion is of state-wide relevance and requests the Local Government Association of Queensland to advocate for introducing speed limitations for personal mobility devices.
[1395] 
[1396] The proposed motion would, if endorsed, further the policy objectives of Council and is also likely to be relevant to other local governments.
[1397] 
[1398] Details associated with the motion are outlined in Appendix A.
[1399] 
[1400] Status of previous Conference Motions submitted by Council
[1401] 
[1402] Council has regularly submitted motions for debate at the Local Government Association of Queensland Annual Conference. In the last two years, each motion submitted by Council was carried or amended and carried at the Conference. Below is the list of motions put forward by Council in the last two years. See Attachment 1 for an update on the status of each motion.
[1403] 
[1404] 2023 - Motion 40 - Vegetation clearing exemptions for necessary firebreaks or fire management lines
[1405] 
[1406] The LGAQ calls on the State Government to review, in consultation with local governments, s20A of Schedule 6 of the Planning Regulation 2017, with a view to provide a more balanced approach to circumstances where vegetation clearing is allowed for bushfire hazard reduction, to minimise biodiversity impacts associated with such clearing.
[1407] 
[1408] 2023 - Motion 102 - Active transport connectivity
[1409] 
[1410] The LGAQ calls on State Government to ensure that continuous active transport connections are provided to existing local networks, when planning and delivering major state led infrastructure projects.
[1411] 
[1412] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1413] 
[1414] <!-- PageNumber="OM Agenda Page 29 of 88" -->
[1415] 
[1416] 
[1417] <PageNum=30 />
[1418] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1419] 
[1420] <!-- PageHeader="24 JULY 2024" -->
[1421] 
[1422] 2023 - Motion 117 - Market Valuation Option for Rating Purposes for Strata Titled Properties
[1423] 
[1424] The LGAQ calls on the State Government to amend the Land Valuation Act 2010 to include the option for councils to request market valuations for the purpose of rating strata titled / community titled properties.
[1425] 
[1426] 2022 - Motion 24 - Review of Financial Statement Materiality Thresholds and Application of Australian Auditing Standards Relating to Delays in Recording of Contributed Assets
[1427] 
[1428] The Local Government Association of Queensland calls on the State Government to work with the Queensland Audit Office (QAO) to review financial statement materiality levels for contributed assets received by councils experiencing high rates of growth.
[1429] 
[1430] 2022 - Motion 36 - Locally Responsive and Sustainable Design Outcomes
[1431] 
[1432] The LGAQ calls on the State and Federal governments for changes to existing planning and building legislation and plan-making processes to remove barriers to the achievement of locally responsive and sustainable design, including by:
[1433] 
[1434] 1\. Making changes to the Queensland Development Code and National Construction Code to provide for more locally responsive and sustainable design measures to be incorporated in planning schemes or alternatively, amending these Codes to include such measures as mandatory requirements;
[1435] 
[1436] 2\. Amending State plan-making guidelines and the State interest-review process to better support the incorporation of locally responsive and sustainable design outcomes in planning schemes; and
[1437] 
[1438] 3\. Any such other actions considered necessary and appropriate to achieve the objective of the motion.
[1439] 
[1440] 2022 - Motion 70 - Better Regulation of the Breeding and Sale of Dogs and Cats
[1441] 
[1442] The Local Government Association of Queensland calls on the State Government to undertake a full review of the Animal Care and Protection Act 2001 and the Animal Management (Cats and Dogs) Act 2008, together with their interaction with the Planning Act 2016, to achieve a more effective regulatory framework for the breeding and sale of cats and dogs and the assessment of development applications relating to such uses.
[1443] 
[1444] 
[1445] ## Legal
[1446] 
[1447] There are no legal implications associated with submitting the proposed motion to the Local Government Association of Queensland for consideration and debate at the Annual Conference.
[1448] 
[1449] 
[1450] ## Policy
[1451] 
[1452] The proposed motion in this report does not conflict with the adopted policy positions or objectives of Council. Should the proposed motion be approved by Council and resolved at the Local Government Association of Queensland Annual Conference, it will influence and shape future advocacy efforts of the LGAQ, which may result in policy and program decisions by the other tiers of government that will be advantageous to the interests of Council.
[1453] 
[1454] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1455] 
[1456] <!-- PageNumber="OM Agenda Page 30 of 88" -->
[1457] 
[1458] 
[1459] <PageNum=31 />
[1460] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1461] 
[1462] <!-- PageHeader="24 JULY 2024" -->
[1463] 
[1464] Risk
[1465] 
[1466] No discernible risks have been identified from potentially submitting the proposed motion to the Local Government Association of Queensland for consideration and debate at its Annual Conference.
[1467] 
[1468] Previous Council Resolution
[1469] 
[1470] Ordinary Meeting 27 July 2023 (OM23/74)
[1471] 
[1472] That Council:
[1473] 
[1474] (a) receive and note the report titled "Motions for the 2023 Local Government Association of Queensland Annual Conference" and
[1475] 
[1476] (b) endorse the following motions for forwarding to the Local Government Association of Queensland for inclusion in the 2023 Annual Conference agenda:
[1477] 
[1478] (i) That Local Government Association of Queensland advocate to the State Government to ensure that the active transport pathways provided as part of the delivery of major roads projects are connected to nearby local networks and not left stranded at the extent of the road project
[1479] 
[1480] (ii) That the Local Government Association of Queensland calls on the State Government to amend the Land Valuation Act 2010 to include the option for councils to request market valuations for the purpose of rating strata titled / community titled properties
[1481] 
[1482] (iii) That the Local Government Association of Queensland calls on the State Government to review, in consultation with local governments, section 20A of Schedule 6 of the Planning Regulation 2017, with a view to provide a more balanced approach to circumstances where vegetation clearing is allowed for bushfire hazard reduction, to minimise biodiversity impacts associated with such clearing.
[1483] 
[1484] 
[1485] ## Ordinary Meeting 28 July 2022 (OM22/53)
[1486] 
[1487] That Council:
[1488] 
[1489] (a) receive and note the report titled "Motions for the 126th Local Government Association of Queensland Annual Conference" and
[1490] 
[1491] (b) endorse the following motions for forwarding to the Local Government Association of Queensland for inclusion in the 126th Annual Conference agenda -
[1492] 
[1493] 1\. That the Local Government Association of Queensland advocate to the Queensland Government to undertake a full review of the Animal Care and Protection Act 2001 and the Animal Management (Cats and Dogs) Act 2008, together with their interaction with the Planning Act 2016, to achieve a more effective regulatory framework for the breeding and sale of cats and dogs and the assessment of development applications relating to such uses.
[1494] 
[1495] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1496] 
[1497] <!-- PageFooter="OM Agenda Page 31 of 88" -->
[1498] 
[1499] 
[1500] <PageNum=32 />
[1501] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1502] 
[1503] <!-- PageHeader="24 JULY 2024" -->
[1504] 
[1505] 2\. That the Local Government Association of Queensland advocate to the Federal and Queensland Governments for changes to existing planning and building legislation and plan-making processes to remove barriers to the achievement of locally responsive and sustainable design, including by:
[1506] 
[1507] a. making changes to the Queensland Development Code and National Construction Code to provide for more locally responsive and sustainable design measures to be incorporated in planning schemes or alternatively, amending these codes to include such measures as mandatory requirements
[1508] 
[1509] b. amending State plan-making guidelines and the State interest-review process to better support the incorporation of locally responsive and sustainable design outcomes in planning schemes and
[1510] 
[1511] c. any such other actions considered necessary and appropriate to achieve the objective of the motion.
[1512] 
[1513] 3\. That the Local Government Association of Queensland advocate to the Queensland Government to work with the Queensland Audit Office (QAO) to review financial statement materiality levels for contributed assets received by Councils experiencing high rates of growth.
[1514] 
[1515] 
[1516] ## Ordinary Meeting 22 July 2021 (OM21/72)
[1517] 
[1518] 
[1519] ### That Council:
[1520] 
[1521] (a) receive and note the report titled "2024 Local Government Association of Queensland Annual Conference Motions" and
[1522] 
[1523] (b) endorse the following motions for forwarding to the Local Government Association of Queensland for inclusion in the 125th Annual Conference agenda -
[1524] 
[1525] ii. That the Local Government Association of Queensland advocate to the Queensland Government to develop and agree a joint State/Local government package of measures that will enable a more timely response to housing affordability and availability in high growth areas of the State.
[1526] 
[1527] iii. That the Local Government Association of Queensland advocate to the Queensland Government to amend section 236 of the Local Government Regulation 2012 to afford greater flexibility to Councils to approve an exemption from the compulsory tender or auction requirements in relation to the disposal of non-current assets, in circumstances where there is a clear business case that demonstrates the disposal of the asset would provide clear beneficial outcomes for the community.
[1528] 
[1529] 
[1530] ## Ordinary Meeting 25 July 2019 (OM19/105)
[1531] 
[1532] That Council:
[1533] 
[1534] (a) receive and note the report titled "Motion for the 123rd Local Government Association of Queensland Annual Conference" and
[1535] 
[1536] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1537] 
[1538] <!-- PageFooter="OM Agenda Page 32 of 88" -->
[1539] 
[1540] 
[1541] <PageNum=33 />
[1542] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1543] 
[1544] <!-- PageHeader="24 JULY 2024" -->
[1545] 
[1546] (b) endorse the following motion titled "Partnership with Power Utilities in delivering Underground Power" for forwarding to the Local Government Association of Queensland for inclusion in the 123rd Annual Conference agenda -
[1547] 
[1548] That the Local Government Association of Queensland lobbies the peak power utilities' industry body to jointly fund undergrounding power where the change to existing overhead power supplies is required as part of a local government's urban upgrade capital works program and is called up in the local government's planning scheme.
[1549] 
[1550] 
[1551] ## Ordinary Meeting 16 August 2018 (OM18/126)
[1552] 
[1553] 
[1554] ### That Council:
[1555] 
[1556] (a) receive and note the report titled "Motion for the 122nd Local Government Association of Queensland Conference 2018" and
[1557] 
[1558] (b) endorse the following motion titled "Enhancing Queensland's future drought resilience" for forwarding to the Local Government Association of Queensland for inclusion in the 122nd Annual Conference agenda -
[1559] 
[1560] That the Local Government Association of Queensland lobby State and Federal governments to develop options (in partnership with local governments, bulk water providers and water retailers) that improve drought resilience in Queensland - particularly with regard to the agricultural sector - through the more efficient storage, movement and use of all types of water, including recycled water and storm/rain water.
[1561] 
[1562] 
[1563] ### Ordinary Meeting 17 August 2017 (OM17/157)
[1564] 
[1565] That Council authorise the Chief Executive Officer to submit to the Local Government Association of Queensland the following motion for inclusion in its 2017 Annual Conference agenda -
[1566] 
[1567] 'That the Local Government Association of Queensland lobby the Queensland Government to undertake an immediate review of the legislative and regulatory framework for local government procurement so that it is more agile and adaptive to a constantly evolving digital environment, more responsive to innovative proposals and products and can be better utilised by Councils to support start-ups and entrepreneurialism in their regions'.
[1568] 
[1569] 
[1570] ## Ordinary Meeting 18 August 2016 (OM16/138)
[1571] 
[1572] 
[1573] ### That Council:
[1574] 
[1575] (a) receive and note the report titled "Motion for 120th Annual Local Government Association Queensland Conference 2016"
[1576] 
[1577] (b) forward the motion titled 'To develop new guidelines to replace Guidelines On Arrangements for Infrastructure External to State Government Sites and Non-State Schools - December 1997' (Appendix A) to the Local Government Association Queensland for inclusion in the 120th Annual Conference Agenda
[1578] 
[1579] (c) forward the motion titled 'To investigate providing legislative power to local government to apply catchment based developer contributions for delivery of non-trunk
[1580] 
[1581] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1582] 
[1583] <!-- PageFooter="OM Agenda Page 33 of 88" -->
[1584] 
[1585] 
[1586] <PageNum=34 />
[1587] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1588] 
[1589] <!-- PageHeader="24 JULY 2024" -->
[1590] 
[1591] infrastructure' (Appendix B) to the Local Government Association Queensland for inclusion in the 120th Annual Conference Agenda and
[1592] 
[1593] (d) forward the motion titled 'To request that the State Government review Planning Scheme amendment processes with a view to removing or significantly reducing the timeframes associated with First State Interest Check Reviews' (Appendix C) to the Local Government Association Queensland for inclusion in the 120th Annual Conference Agenda.
[1594] 
[1595] 
[1596] ### Ordinary Meeting 20 August 2015 (OM15/132)
[1597] 
[1598] That Council:
[1599] 
[1600] (a) receive and note the report titled "119th Annual Local Government Association Queensland Conference 2015 - Motion - Valuation Methodology for Strata Units" and
[1601] 
[1602] (b) forward the motion titled 'To introduce a different valuation methodology to Strata Units by using market value' as detailed in Appendix A, to the Local Government Association Queensland for inclusion in the 119th Annual Conference Agenda.
[1603] 
[1604] 
[1605] ## Related Documentation
[1606] 
[1607] • Local Government Association of Queensland 2024 Annual Conference Program.
[1608] 
[1609] • Constitution of the Local Government Association of Queensland.
[1610] 
[1611] 
[1612] ## Critical Dates
[1613] 
[1614] This report must be considered by Council at the July 2024 Ordinary Meeting to enable any endorsed motions to be submitted to the Local Government Association of Queensland by 29 July 2024.
[1615] 
[1616] 
[1617] ### Implementation
[1618] 
[1619] Subject to Council endorsing the recommendations in this report, the approved proposed motion will be forwarded to the Local Government Association of Queensland for consideration for inclusion in the Annual Conference Motions Agenda.
[1620] 
[1621] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1622] 
[1623] <!-- PageNumber="OM Agenda Page 34 of 88" -->
[1624] 
[1625] 
[1626] <PageNum=35 />
[1627] 24 JULY 2024
[1628] 
[1629] Ordinary Meeting Agenda Item 8.2 2024 Local Government Association of Queensland Annual Conference Motions Motion - Personal Mobility Devices
[1630] 
[1631] Appendix A
[1632] 
[1633] <figure>
[1634] 
[1635] ![](figures/15)<!-- FigureContent="The image displays the logo of LGAQ. The letters "LGAQ" are presented in a bold, blue font. There is a curved, grey shape that intersects the text, creating a dynamic and modern look. The design suggests a professional or organizational identity." --></figure>
[1636] 
[1637] 
[1638] Every Queensland community deserves to be a liveable one
[1639] 
[1640] 2024 LGAQ Annual Conference - Motions template
[1641] 
[1642] |||
[1643] | - | - |
[1644] | Who is the key contact for this motion? (required) | Shanagh Jacobs |
[1645] | Submitting council (required) | Sunshine Coast Council |
[1646] | Supporting organisation (if applicable) | |
[1647] | Council resolution # (required) | |
[1648] | Date of council resolution (required) | 24/07/2024 |
[1649]  :unselected:
[1650] Does this motion have state-wide relevance? Yes
[1651] 
[1652] |||
[1653] | - | - |
[1654] | Title of motion (required) | Speed limiters for personal mobility devices (e-scooters, e-skateboards and self-balancing single wheeled devices (like e-unicycles and e-boards)) |
[1655] | Motion (required) | The LGAQ calls on the State Government to introduce device limitations and standards, like those for e-bikes, to control the speed and capabilities of personal mobility devices. |
[1656] | What is the desired outcome sought? (required) 200 word limit | Through controlling the capability of personal mobility devices, such as e- scooters, the motion seeks to achieve the following desired outcomes · improved safety for device users, motorists, and pedestrians, · increased use of active and e- transport options and · reduced demand and expenses related to patrol and regulation programs to manage the use of devices. |
[1657] | Background (required) 350 word limit | Local Governments continue to have a focus on encouraging the use of alternate travel modes to extend the sustainability of their road networks, achieve positive environmental outcomes and to improve the connectivity of public transport networks. |
[1658] 
[1659] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1660] 
[1661] <!-- PageNumber="OM Agenda Page 35 of 88" -->
[1662] 
[1663] 
[1664] <PageNum=36 />
[1665] Ordinary Meeting Agenda Item 8.2 2024 Local Government Association of Queensland Annual Conference Motions Motion - Personal Mobility Devices
[1666] 
[1667] <figure>
[1668] 
[1669] ![](figures/16)<!-- FigureContent="The image is a logo for the Local Government Association of Queensland (LGAQ). The logo features the acronym "LGAQ" in bold, blue capital letters. Overlaying the acronym is a stylized design that incorporates a grey, sweeping curved line. Below the acronym, the full name "Local Government Association of Queensland" is written in smaller, black capital letters. The overall design is clean and modern." --></figure>
[1670] 
[1671] 
[1672] |||
[1673] | - | - |
[1674] | | Personal mobility devices, like e-scooters, are expected to play an important role in travel networks and their popularity continues to grow. Safety continues to be a primary consideration for people choosing alternate travel options. The State Government recently introduced a number of legislative controls to regulate the use of these devices. While the legislation may drive behavioural change over time, if enforced, the enforcement programs are resource intensive and expensive for councils and Queensland Police to implement. The State Government currently has standards and controls for e-bikes which prohibit the use of high-powered devices in public spaces. The standards define the capabilities and limitations for devices which impose controls at the device level. These standards compliment the road rules and remove high powered devices from paths and roads reducing the overall risk to the community. |
[1675] | Case study/ Example (optional) 350 word limit | Sunshine Coast Council are currently undertaking an e-bike and e-scooter trial, where an operator has been issued with a local law permit to operate commercially from council-controlled land until 30 September 2024. |
[1676] || The local law permit requires the operator to ensure e-scooter use complies with State Government requirements, whereby devices do not exceed 12km/h on all paths and 25km/h on roads. This is achieved using geofencing technology. While Council has the ability to regulate and impose conditions relating to e- scooter use in public places, the enforcement of private e-scooter users. falls to Queensland Police to implement. According to RACQ (2023), between 1 January 2019 to 30 September 2023, there had been 3,305 presentations to participating Queensland Injury |
[1677] 
[1678] July 24 Wilson Crawley
[1679] 
[1680] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1681] 
[1682] <!-- PageFooter="OM Agenda Page 36 of 88" -->
[1683] 
[1684] 24 JULY 2024
[1685] 
[1686] Appendix A
[1687] 
[1688] 2
[1689] 
[1690] 
[1691] <PageNum=37 />
[1692] Ordinary Meeting Agenda Item 8.2 2024 Local Government Association of Queensland Annual Conference Motions Motion - Personal Mobility Devices
[1693] 
[1694] 24 JULY 2024
[1695] 
[1696] Appendix A
[1697] 
[1698] 3
[1699] 
[1700] <figure>
[1701] 
[1702] ![](figures/17)<!-- FigureContent="The image shows the logo of the Local Government Association of Queensland (LGAQ). The main part of the logo consists of the acronym "LGAQ" in large, bold blue letters. To the right of the acronym, there is a stylized, curved grey shape. Below "LGAQ," the full name "LOCAL GOVERNMENT ASSOCIATION OF QUEENSLAND" is written in smaller, black capital letters." --></figure>
[1703] 
[1704] 
[1705] Surveillance Unit (QISU) emergency departments in Queensland as a result of an e-scooter ride gone wrong.
[1706] 
[1707] The Introduction of device limitations and standards, like those for e-bikes, to control the speed and capabilities of personal mobility devices (e-scooters, e- skateboards and self-balancing single wheeled devices (like e-unicycles and e- boards)), promotes self-regulation and improved safety in public places.
[1708] 
[1709] Source: https://www.racq.com.au/latest- news/news/2023/12/ns191223-data- shows-e-scooter-riders-still-not-taking- safety-seriously
[1710] 
[1711] July 24 Wilson Crawley
[1712] 
[1713] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1714] 
[1715] <!-- PageFooter="OM Agenda Page 37 of 88" -->
[1716] 
[1717] 
[1718] <PageNum=38 />
[1719] 
[1720] 
[1721] <PageNum=39 />
[1722] <!-- PageHeader="24 JULY 2024" -->
[1723] 
[1724] Ordinary Meeting Agenda Item 8.2 2024 Local Government Association of Queensland Annual Conference Motions
[1725] 
[1726] Attachment 1 Local Government Association of Queensland Conference Motions Status of Council Resolutions
[1727] 
[1728] Local Government Association of Queensland - Status of previous Conference Motions submitted by Council
[1729] 
[1730] 2023 - Motion 40 - Vegetation clearing exemptions for necessary firebreaks or fire management lines [carried]
[1731] 
[1732] The LGAQ calls on the State Government to review, in consultation with local governments, s20A of Schedule 6 of the Planning Regulation 2017, with a view to provide a more balanced approach to circumstances where vegetation clearing is allowed for bushfire hazard reduction, to minimise biodiversity impacts associated with such clearing.
[1733] 
[1734] Status: In Progress.
[1735] 
[1736] Latest update from the Local Government Association of Queensland:
[1737] 
[1738] Following the December 2023 Queensland Government Cabinet reshuffle, the LGAQ wrote to the new Minister for Housing, Local Government, Planning and Public Works, the Hon Meaghan Scanlon MP, outlining this resolution. On 23 November 2023, the LGAQ wrote to the Deputy Premier, Minister for State Development, Infrastructure, Local Government and Planning outlining this resolution. On 27 November 2023, the LGAQ wrote to Mike Kaiser, Director-General of the Department of State Development, Infrastructure, Local Government and Planning outlining this resolution.
[1739] 
[1740] 
[1741] #### 2023 - Motion 102 - Active transport connectivity [carried]
[1742] 
[1743] The LGAQ calls on State Government to ensure that continuous active transport connections are provided to existing local networks, when planning and delivering major state led infrastructure projects.
[1744] 
[1745] Status: In Progress.
[1746] 
[1747] Latest update from the Local Government Association of Queensland:
[1748] 
[1749] On 23 November 2023, the LGAQ wrote to the Minister for Transport and Main Roads and Minister for Digital Services, outlining this resolution. On 27 November 2023, the LGAQ wrote to Ms Sally Stannard, Director-General of the Department of Transport and Main Roads outlining this motion. Following the December 2023 Queensland Government Cabinet reshuffle, the LGAQ wrote to the new Minister for Transport and Main Roads, the Hon Bart Mellish MP, outlining this resolution.
[1750] 
[1751] 
[1752] ##### 2023 - Motion 117 - Market Valuation Option for Rating Purposes for Strata Titled Properties [carried]
[1753] 
[1754] The LGAQ calls on the State Government to amend the Land Valuation Act 2010 to include the option for councils to request market valuations for the purpose of rating strata titled / community titled properties.
[1755] 
[1756] Status: In Progress.
[1757] 
[1758] Latest update from the Local Government Association of Queensland:
[1759] 
[1760] On 25 November 2023 The LGAQ wrote to the Hon Scott Stewart seeking consideration of the matter as well as an understanding of how we can work together to address the issues raised.
[1761] 
[1762] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1763] 
[1764] <!-- PageNumber="OM Agenda Page 39 of 88" -->
[1765] 
[1766] 
[1767] <PageNum=40 />
[1768] <!-- PageHeader="24 JULY 2024" -->
[1769] 
[1770] Ordinary Meeting Agenda Item 8.2 2024 Local Government Association of Queensland Annual Conference Motions
[1771] 
[1772] Attachment 1 Local Government Association of Queensland Conference Motions Status of Council Resolutions
[1773] 
[1774] 2022 - Motion 24 - Review of Financial Statement Materiality Thresholds and Application of Australian Auditing Standards Relating to Delays in Recording of Contributed Assets [carried]
[1775] 
[1776] The Local Government Association of Queensland calls on the State Government to work with the Queensland Audit Office (QAO) to review financial statement materiality levels for contributed assets received by councils experiencing high rates of growth.
[1777] 
[1778] Status: In Progress.
[1779] 
[1780] Latest update from the Local Government Association of Queensland:
[1781] 
[1782] In December 2022, the LGAQ wrote to Deputy Premier Steven Miles regarding this request.
[1783] 
[1784] The LGAQ has also received a formal response from the Deputy Premier on 8 May 2023, stating that the Queensland Government is not able to set materiality thresholds for audits conducted by the QAO. The Auditor-General is fully independent, appointed by the Queensland Governor in Council for a seven-year term. The QAO's work is governed by the Auditor-General Act 2009 and guided by auditing and accounting standards (which are not set by the Queensland Government). It is in accordance with this framework that the materiality thresholds are set.
[1785] 
[1786] 2022 - Motion 36 - Locally Responsive and Sustainable Design Outcomes [carried]
[1787] 
[1788] The LGAQ calls on the State and Federal governments for changes to existing planning and building legislation and plan-making processes to remove barriers to the achievement of locally responsive and sustainable design, including by:
[1789] 
[1790] 1\. Making changes to the Queensland Development Code and National Construction Code to provide for more locally responsive and sustainable design measures to be incorporated in planning schemes or alternatively, amending these Codes to include such measures as mandatory requirements;
[1791] 
[1792] 2\. Amending State plan-making guidelines and the State interest-review process to better support the incorporation of locally responsive and sustainable design outcomes in planning schemes; and
[1793] 
[1794] 3\. Any such other actions considered necessary and appropriate to achieve the objective of the motion.
[1795] 
[1796] Status: In Progress.
[1797] 
[1798] Latest update from the Local Government Association of Queensland:
[1799] 
[1800] In December 2022, the LGAQ wrote to the Hon Ed Husic MP, Australian Minister for Industry and Science, the Hon Dr Steven Miles MP, Queensland Deputy Premier, Minister for State Development, Infrastructure, Local Government and Planning Minister Assisting the Premier on Olympics Infrastructure and the Hon Mick de Brenni MP, Queensland Minister for Energy, Renewables and Hydrogen and Minister for Public Works and Procurement, as well as respective Queensland Directors-General, advising of the final outcomes of the 2022 LGAQ Annual Conference, welcoming consideration of this resolution, and seeking a response.
[1801] 
[1802] In early 2023, the LGAQ met with officers from the submitting councils for this motion and a response to LGAQ's original correspondence was received from the Deputy Premier on 8 May 2023, stating the following:
[1803] 
[1804] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1805] 
[1806] <!-- PageNumber="OM Agenda Page 40 of 88" -->
[1807] 
[1808] 
[1809] <PageNum=41 />
[1810] 24 JULY 2024
[1811] 
[1812] Ordinary Meeting Agenda Item 8.2 2024 Local Government Association of Queensland Annual Conference Motions
[1813] 
[1814] Attachment 1 Local Government Association of Queensland Conference Motions Status of Council Resolutions
[1815] 
[1816] The Queensland Government recognises the importance of facilitating locally responsive and sustainable design outcomes through local government planning schemes. In 2018, the Queensland Government released the Design Manual, which sets out nine principles for good urban design outcomes across Queensland. QDesign can be used by all levels of government and built environment stakeholders.
[1817] 
[1818] As LGAQ would be aware, the Queensland Government, through the Department of Energy and Public Works (DEPW), is responsible for the Building Act 1975 and the Queensland Development Code, while the Australian Government is responsible for the National Construction Code.
[1819] 
[1820] The Queensland Government invites the LGAQ to submit further details regarding barriers to achieving locally responsive and sustainable design outcomes particularly through plan making guidelines and State interest review processes. This will require a collaborative approach with LGAQ, multiple state agencies and the Australian Government.
[1821] 
[1822] I recommend LGAQ continues to engage with DEPW on these matters."
[1823] 
[1824] A response from Minister de Brenni to this resolution is still pending. The LGAQ is continuing to engage with the Department of Energy and Public Works and with a number of local governments about this resolution and will be preparing a further response to the State Government to further advocacy on this matter.
[1825] 
[1826] 
[1827] ##### 2022 - Motion 70 - Better Regulation of the Breeding and Sale of Dogs and Cats [carried]
[1828] 
[1829] The Local Government Association of Queensland calls on the State Government to undertake a full review of the Animal Care and Protection Act 2001 and the Animal Management (Cats and Dogs) Act 2008, together with their interaction with the Planning Act 2016, to achieve a more effective regulatory framework for the breeding and sale of cats and dogs and the assessment of development applications relating to such uses.
[1830] 
[1831] Status: In Progress.
[1832] 
[1833] Latest update from the Local Government Association of Queensland:
[1834] 
[1835] The LGAQ succeeded to progress significant changes to the Act. In late 2021, DAF commenced a targeted review of the Animal Management (Cats and Dogs) Act 2008 following the creation of the local government Animal Management Taskforce and technical working group. Following increased incidences of dog attacks in 2023, the Department commenced the process to update the Act, which passed as the Agriculture and Fisheries and Other Legislation Amendment Bill in April 2024.
[1836] 
[1837] The changes include significant requests by the Taskforce including the state-wide ban on restricted dogs, increased pins for attacks and state-wide effective control measures.
[1838] 
[1839] The changes are in direct response to recommendations by the Animal Management Taskforce, with the work of the Taskforce continuing by mid-2024 to implement changes appropriately and progress further legislative changes in line. The call for better regulation for the breeding and sale of cats and dogs is included in the forward work program of the new Taskforce.
[1840] 
[1841] Background:
[1842] 
[1843] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1844] 
[1845] <!-- PageNumber="OM Agenda Page 41 of 88" -->
[1846] 
[1847] 
[1848] <PageNum=42 />
[1849] 24 JULY 2024
[1850] 
[1851] Ordinary Meeting Agenda Item 8.2 2024 Local Government Association of Queensland Annual Conference Motions
[1852] 
[1853] Attachment 1 Local Government Association of Queensland Conference Motions Status of Council Resolutions
[1854] 
[1855] Following the formation of the Animal Management Taskforce and the Animal Management Technical working group in early 2022, the LGAQ provided a submission to the Department of Agriculture and Fisheries on the discussion paper 'Stronger Dog Laws' in August 2023.
[1856] 
[1857] Following overwhelming support by the community for stronger dog laws, the LGAQ provided a submission on the Agriculture and Fisheries Bill 2023 in November 2023 and appeared before the Committee in December 2023 to defend councils positions. The Bill was passed with minor amendments in April 2024.
[1858] 
[1859] Oct 2022 - The Technical Working Group has formed three sub working groups to discuss and identify potential pathways towards solutions regarding domestic dog management. The individual groups focus on:
[1860] 
[1861] 1\. Prohibited breeds, recovery of costs, forfeiture of seized unregistered dogs, non- compliance escalation
[1862] 
[1863] 2\. Amend and speed up dangerous dog declaration, creation of distinction between 'dangerous' and 'menacing'
[1864] 
[1865] 3\. Regulated dog enclosures in apartments, power to issue PINs, power of entry, offspring management of seized dogs, body worn camera support.
[1866] 
[1867] Following the 2022 Annual General Meeting, councils requested further action on Animal Management. The Department of Agriculture and Fisheries has responded to the official letter sent, with response to resolution 70 most relevant for this process:
[1868] 
[1869] "DAF has just completed a comprehensive review of the Animal Care and Protection Act 2001 (ACPA). The review which was conducted over the last two years included a significant stakeholder consultation process which resulted in over 2 500 submissions being received from individuals and organisations. As a result of the review, amendments were made to the ACPA by the Animal Care and Protection Amendment Act 2022 which commenced on 12 December 2022. Overall, the review of the ACPA found that the framework of the Act sets appropriate standards for the care of animals and penalties for breaches of those standards. Section 17 of the ACPA imposes a duty of care on a person that is in charge of an animal to ensure the animal is provided with appropriate food, water, shelter and treatment. The duty of care covers any animal that is in the care of a person including animals kept for breeding. The maximum penalty for a breach of duty of care is 300 penalty units or one year imprisonment. A recent amendment to section 17 means that if the breach causes death, serious deformity, serious disability or prolonged suffering of the animal, the maximum penalty is 2000 penalty units or three years imprisonment. Schedule 3A of the Animal Care and Protection Regulation 2012 contains the Code of Practice for breeding of dogs which imposes specific duty of care requirements on a person in charge of a breeding dog and a relevant puppy and a premises used for breeding a dog and a relevant puppy. These requirements include the provision and accessibility of food, water, enrichment, health inspections and veterinary care. At the request of LGAQ in late 2021, DAF commenced a limited scope, 'targeted' review of the Animal Management (Cats and Dogs) Act 2008. The review is in response to issues raised by local governments and their communities concerning restricted and dangerous dogs, public safety, animal management enforcement and related matters. The review terms of reference, agreed to by LGAQ and nominated Council representatives, doesn't provide for a review of the entire Act. The breeding and sale of cats and dogs is not within the terms of reference; however, feedback generated by
[1870] 
[1871] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1872] 
[1873] <!-- PageNumber="OM Agenda Page 42 of 88" -->
[1874] 
[1875] 
[1876] <PageNum=43 />
[1877] 24 JULY 2024
[1878] 
[1879] Ordinary Meeting Agenda Item 8.2 2024 Local Government Association of Queensland Annual Conference Motions Attachment 1 Local Government Association of Queensland Conference Motions Status of Council Resolutions
[1880] 
[1881] stakeholder local governments may inform further considerations for a future review process. A final report on the targeted review process is scheduled to be provided to the Minister for Agricultural Industry Development and Minister for Rural Communities by mid-2023. DAF is also currently working on enhancements to the Queensland Dog Breeders Register system. The enhancements aim to address fraudulent or incomplete data entry upon application and to raise awareness of the requirements, general knowledge and breeder Standards during the application of renewal process."
[1882] 
[1883] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1884] 
[1885] <!-- PageFooter="OM Agenda Page 43 of 88" -->
[1886] 
[1887] 
[1888] <PageNum=44 />
[1889] 
[1890] 
[1891] <PageNum=45 />
[1892] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1893] 
[1894] <!-- PageHeader="24 JULY 2024" -->
[1895] 
[1896] 
[1897] ### 8.3 COUNCIL OF MAYORS SOUTH EAST QUEENSLAND - PROPOSED INTERNATIONAL DELEGATION
[1898] 
[1899] |||
[1900] | - | - |
[1901] | File No: | Council Meetings |
[1902] | Author: | Coordinator Executive Support Office of the Mayor Civic Governance |
[1903] 
[1904] 
[1905] ## PURPOSE
[1906] 
[1907] This report seeks Council's approval for Mayor Natoli to undertake overseas travel from 4 to 14 February 2025 to participate in the Council of Mayors South East Queensland (CoMSEQ) delegation to Singapore, Manchester and Paris.
[1908] 
[1909] The Mayor's participation in this delegation cannot be confirmed until authorisation to undertake the overseas travel has been provided by Council.
[1910] 
[1911] 
[1912] ## EXECUTIVE SUMMARY
[1913] 
[1914] At its Board Meeting on 10 May 2024, COMSEQ endorsed its 2024-25 Strategic Action and Advocacy Plan. The proposed international delegation to Singapore, Manchester and Paris forms part of that Plan and aims to increase global awareness of the South-East Queensland region and to promote trade and investment opportunities. Missions such as this also provide excellent opportunities for the Mayors of South-East Queensland to discuss regional priorities and develop relationships.
[1915] 
[1916] While the delegation has been approved by the CoMSEQ Board, at the time of writing this report, which local governments are participating is not yet confirmed. All 11 Mayors that comprise the CoMSEQ Board have been invited and have indicated a willingness to participate.
[1917] 
[1918] COMSEQ confirmed that it will meet the costs of the Mayor's travel, including economy class flights, accommodation, transfers and meals.
[1919] 
[1920] The focus areas of the proposed delegation program are aligned to Council's regional strategies, Corporate Plan and Operational Plan. The learnings, relationships and strategic connections that could be established from participating in this delegation are potentially considerable, particularly given Council's role as a key delivery partner for the Brisbane 2032 Olympic and Paralympic Games and the opportunities and potential challenges this may afford.
[1921] 
[1922] The purpose of the mission is to:
[1923] 
[1924] 1\. Investigate how city regions leverage major events to advance their global identity; and
[1925] 
[1926] 2\. Explore inter-governmental initiatives that promote greater investment while balancing the competing interests from other levels of government, including:
[1927] 
[1928] a. Managing population growth - housing affordability, land-use planning and regional growth management
[1929] 
[1930] b. Funding a growing region - maintaining fiscal sustainability through a partnership model
[1931] 
[1932] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1933] 
[1934] <!-- PageFooter="OM Agenda Page 45 of 88" -->
[1935] 
[1936] 
[1937] <PageNum=46 />
[1938] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1939] 
[1940] <!-- PageHeader="24 JULY 2024" -->
[1941] 
[1942] C. Regional connectivity - mobility and smart infrastructure
[1943] 
[1944] d. Sustainability and resilience - waste management, circular economy, and water security.
[1945] 
[1946] Given the low cost to Council and to rate-payers, this is a high-level local government delegation, covering a broad range of subject areas of interest to Sunshine Coast Council, it is proposed that Mayor Natoli be authorised to undertake overseas travel in an official capacity to participate in the delegation. The nomination of Mayor Natoli to participate in the proposed delegation is also consistent with the Mayor's responsibilities under the adopted Councillor Portfolio System (adopted on 30 May 2024) and the Mayor's position as the Council representative on, and Director of, CoMSEQ.
[1947] 
[1948] In line with the requirements of Council's Corporate Travel Policy, Councillor Expenses and Resources Policy and International Relations Policy, authority from Council for the Mayor to undertake overseas travel in an official capacity is required prior to confirming the Mayor's participation in the delegation.
[1949] 
[1950] OFFICER RECOMMENDATION
[1951] 
[1952] That Council:
[1953] 
[1954] (a) receive and note the report titled "Council of Mayors South East Queensland - Proposed International Delegation"
[1955] 
[1956] (b) approve for Mayor Natoli to undertake overseas travel in an official capacity between 4 February 2025 and 14 February 2025 to participate in the Council of Mayors South East Queensland proposed delegation to Singapore, Manchester and Paris
[1957] 
[1958] (c) authorise the expenditure of up to $1,500 by Council to cover the balance of incidental costs associated with Mayor Natoli's participation in the Council of Mayors South East Queensland delegation to Singapore, Manchester and Paris, with this expenditure to be met from the existing budget for the Civic Governance Group; and
[1959] 
[1960] (d) note that costs associated with Mayor Natoli's travel to Singapore, Manchester and Paris - namely economy class airfares, accommodation, most meals, and on- ground transport - will be met by the Council of Mayors South East Queensland.
[1961] 
[1962] 
[1963] ## FINANCE AND RESOURCING
[1964] 
[1965] COMSEQ has advised that it will meet the following costs associated with the Mayor's participation in the proposed delegation to Singapore, Manchester and Paris:
[1966] 
[1967] • Economy class return airfares
[1968] 
[1969] • Accommodation in the locations to be visited by the delegation (with accommodation selected and determined by CoMSEQ)
[1970] 
[1971] • Most meals
[1972] 
[1973] • All on-ground, in-country, transportation
[1974] 
[1975] <!-- PageFooter="Sunshine Coast Regional Council" -->
[1976] 
[1977] <!-- PageFooter="OM Agenda Page 46 of 88" -->
[1978] 
[1979] 
[1980] <PageNum=47 />
[1981] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[1982] 
[1983] <!-- PageHeader="24 JULY 2024" -->
[1984] 
[1985] It is estimated that up to $1,500 will be required to meet any other incidental costs related to travelling as an official representative of Council. Funding can be sourced from Council's current operational budget to fund expenditure on incidentals.
[1986] 
[1987] Council may nominate an officer to accompany the Mayor as part of the delegation, but all travel and accommodation costs for any accompanying Council officer will need to be met by Council. It is a matter for the Chief Executive Officer, in consultation with the Mayor, to determine whether a Council officer participates in the delegation.
[1988] 
[1989] 
[1990] ## CORPORATE PLAN
[1991] 
[1992] Corporate Plan Goal:
[1993] 
[1994] Our outstanding organisation
[1995] 
[1996] Outcome:
[1997] 
[1998] We serve our community by providing this great service
[1999] 
[2000] Operational Activity:
[2001] 
[2002] S30 - Governance - providing internal leadership, legal opinion,
[2003] 
[2004] governance and audit functions ensuring legislative accountability,
[2005] 
[2006] transparency and ethical obligations are supported.
[2007] 
[2008] 
[2009] ## CONSULTATION
[2010] 
[2011] Councillor Consultation
[2012] 
[2013] Mayor Natoli has been consulted in the preparation of this report.
[2014] 
[2015] The Deputy Mayor Councillor Suarez has also been consulted in relation to her availability to act as Mayor for the period Mayor Natoli is travelling overseas.
[2016] 
[2017] Internal Consultation
[2018] 
[2019] The Acting Chief Executive Officer and Council's Executive Leadership Team have been consulted on the purpose and themes of the delegation and the proposal that the Mayor participate in the delegation.
[2020] 
[2021] External Consultation
[2022] 
[2023] Consultation has been undertaken with CoMSEQ in relation to the proposed travel arrangements, itinerary and briefings required for the proposed delegation.
[2024] 
[2025] Community Engagement
[2026] 
[2027] There has been no Community Engagement undertaken in relation to this item.
[2028] 
[2029] 
[2030] ## PROPOSAL
[2031] 
[2032] At its Board Meeting on 10 May 2024, CoMSEQ committed to examining the opportunity for the Mayors of South East Queensland to undertake an international delegation to Singapore, Manchester and Paris.
[2033] 
[2034] The focus areas for the proposed delegation are to:
[2035] 
[2036] 1\. Investigate how city regions leverage major events to advance their global identity; and
[2037] 
[2038] 2\. Explore inter-governmental initiatives that promote greater investment while balancing the competing interests from other levels of government, including:
[2039] 
[2040] a. Managing population growth - housing affordability, land-use planning and regional growth management
[2041] 
[2042] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2043] 
[2044] <!-- PageFooter="OM Agenda Page 47 of 88" -->
[2045] 
[2046] 
[2047] <PageNum=48 />
[2048] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2049] 
[2050] 24 JULY 2024
[2051] 
[2052] b. Funding a growing region - maintaining fiscal sustainability through a partnership model
[2053] 
[2054] C. Regional connectivity - mobility and smart infrastructure
[2055] 
[2056] d Sustainability and resilience - waste management, circular economy, and water security.
[2057] 
[2058] The key focus areas in each city are:
[2059] 
[2060] 
[2061] ## Singapore
[2062] 
[2063] • Mass transit network
[2064] 
[2065] • Green development / buildings and innovations
[2066] 
[2067] • Resource recovery
[2068] 
[2069] • Promoting food security with SEQ produce
[2070] 
[2071] Greater Manchester
[2072] 
[2073] • Growth funding models and evolution of funding agreements
[2074] 
[2075] • Strategic spatial planning and locally controlled integrated travel network (Transport for Manchester) as part of a broader suite of devolved powers transferred from central government
[2076] 
[2077] • Keeping more business rates collected by Council from local businesses to pay for local services; enterprise zones and local economic partnerships.
[2078] 
[2079] 
[2080] ## Greater Paris Metropolis
[2081] 
[2082] Post-2024 Olympic observatory, including:
[2083] 
[2084] • Performance of 'eco-friendly games'
[2085] 
[2086] · Transport and mobility
[2087] 
[2088] • Housing
[2089] 
[2090] · City operations and planning
[2091] 
[2092] • Legacy and community engagement
[2093] 
[2094] The proposed delegation will undertake technical studies and site visits, executive and civic roundtables as well as undertake networking with government and industry leaders in each of the destination cities.
[2095] 
[2096] At this time, all eleven councils that comprise CoMSEQ have indicated a willingness to participate in the proposed delegation, with the majority of delegates likely to be the Mayor of SEQ councils. The duration of the proposed delegation involves seven days of meetings and associated site visits, two full days of travel and one rest day. The proposed itinerary for the delegation is below:
[2097] 
[2098] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2099] 
[2100] <!-- PageNumber="OM Agenda Page 48 of 88" -->
[2101] 
[2102] 
[2103] <PageNum=49 />
[2104] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2105] 
[2106] 24 JULY 2024
[2107] 
[2108] | Date/Date 2025 | Activity | City / Region |
[2109] | - | - | - |
[2110] | Tues, 4 Feb | Transit | Brisbane to Singapore |
[2111] | Wed, 5 Feb | Full program day | Singapore |
[2112] | Thurs, 6 Feb | Full program day | Singapore |
[2113] | Fri, 7 Feb | Half program day | Singapore / Manchester |
[2114] | Sat, 8 Feb | Full program day | Manchester / Paris |
[2115] | Sun, 9 Feb | Rest day | Paris |
[2116] | Mon, 10 Feb | Full program day | Paris |
[2117] | Tues, 11 Feb | Full program day | Paris |
[2118] | Wed, 12 Feb | Full program day | Paris |
[2119] | Thurs, 13 Feb | Transit | Paris to Brisbane |
[2120] | Fri, 14 Feb | Transit | Arrive Brisbane (AM) |
[2121] 
[2122] Given this is a high-level local government delegation, covering a broad range of subject areas of interest to Sunshine Coast Council, it is proposed that Mayor Natoli be authorised to undertake overseas travel in an official capacity to participate in the delegation. The nomination of Mayor Natoli to participate in the proposed delegation is also consistent with the Mayor's responsibilities under the adopted Councillor Portfolio System (adopted on 30 May 2024) and the Mayor's position as the Council representative on, and Director of, CoMSEQ.
[2123] 
[2124] In line with the requirements of Council's Corporate Travel Policy, Councillor Expenses and Resources Policy and International Relations Policy, authority from Council for the Mayor to undertake overseas travel in an official capacity is required prior to confirming the Mayor's participation in the delegation.
[2125] 
[2126] Each of the focus areas of the proposed delegation are key areas of interest to Council as represented in its regional strategies, 2024-2028 Corporate Plan and 2024-2025 Operational Plan. For example, housing affordability and mass transit as well as contemporary approaches to waste management are all matters that Council is currently examining as part of the development of its new Planning Scheme, the advancement of its Integrated Transport Strategy and the continuing implementation of its Environment and Liveability Strategy 2017. Likewise, Council is currently working towards the development of an appropriate and fit for purpose legacy plan that will best advance the interests of the region. That work could potentially benefit from insights to some of the learnings and experiences of Paris which will, by that time, have recently hosted the Olympic and Paralympic Games in 2024.
[2127] 
[2128] The learnings, relationships and strategic connections that could be established from participating in this proposed delegation are considered to be significant, particularly given the low cost to Council as well as the forecast growth and the impacts of Council's role as a key delivery partner for the Brisbane 2032 Olympic and Paralympic Games.
[2129] 
[2130] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2131] 
[2132] <!-- PageFooter="OM Agenda Page 49 of 88" -->
[2133] 
[2134] 
[2135] <PageNum=50 />
[2136] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2137] 
[2138] <!-- PageHeader="24 JULY 2024" -->
[2139] 
[2140] Should Council authorise the Mayor to undertake the overseas travel proposed in this report, the Mayor will need to update her Register of Interests to record the receipt of travel and accommodation funded by CoMSEQ.
[2141] 
[2142] Legal
[2143] 
[2144] There are no known legal issues associated with Mayor Natoli participating in the proposed delegation to Singapore, Manchester and Paris with COMSEQ. All immigration requirements of the governments of Singapore, the United Kingdom and France will be complied with as part of the Mayor's participation in the proposed delegation.
[2145] 
[2146] Policy
[2147] 
[2148] There are no identified potential conflicts with Council policies associated with the recommendations in this report.
[2149] 
[2150] Risk
[2151] 
[2152] Whilst there is always a risk of negative public reaction to expenditure associated with undertaking international travel, it needs to be borne in mind that:
[2153] 
[2154] • the cost of Mayor Natoli's participation in the proposed delegation is being substantially met by CoMSEQ; and
[2155] 
[2156] • there is a clear advantage for the Sunshine Coast and for Council in Mayor Natoli participating in the delegation and in establishing valuable contacts relevant to planning for and progressing a range of priority activities for Council.
[2157] 
[2158] Council will continue to monitor the travel advisory service provided by the Department of Foreign Affairs and Trade in order to respond appropriately to any heightened level of travel risk that might eventuate prior to February 2025.
[2159] 
[2160] Previous Council Resolution
[2161] 
[2162] Ordinary Meeting 25 August 2022 (OM22/69)
[2163] 
[2164] That Council:
[2165] 
[2166] (a) receive and note the report titled "Council of Mayors South East Queensland - Proposed Delegation to North America"
[2167] 
[2168] (b) approve for Mayor Mark Jamieson to undertake overseas travel in an official capacity on 2 - 12 February 2023 to participate in the Council of Mayors South East Queensland proposed delegation to North America
[2169] 
[2170] (c) authorise the expenditure of up to $12,000 by Council to cover the balance of air travel and other incidental costs associated with Mayor Jamieson's participation in the Council of Mayors South East Queensland delegation to North America, with this expenditure to be met from the existing budget for the Office of the Mayor and
[2171] 
[2172] (d) note that costs associated with Mayor Jamieson's travel to North America - namely accommodation, meals, on-ground transport and a contribution to airfares - will be met by the Council of Mayors South East Queensland.
[2173] 
[2174] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2175] 
[2176] <!-- PageFooter="OM Agenda Page 50 of 88" -->
[2177] 
[2178] 
[2179] <PageNum=51 />
[2180] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2181] 
[2182] 24 JULY 2024
[2183] 
[2184] Related Documentation
[2185] 
[2186] • Constitution of COMSEQ Pty Ltd.
[2187] 
[2188] • COMSEQ Strategic Action Plan 2024-2025
[2189] 
[2190] • Sunshine Coast Council Corporate Plan 2024-2028
[2191] 
[2192] • Sunshine Coast Council Operational Plan 2024-2025
[2193] 
[2194] • Sunshine Coast Council Councillor Portfolio System and Protocols
[2195] 
[2196] 
[2197] ## Critical Dates
[2198] 
[2199] Council needs to confirm by July 2024 whether Mayor Natoli will participate in the proposed CoMSEQ delegation to secure the proposed group bookings.
[2200] 
[2201] In line with the requirements of Council's Corporate Travel Policy, Councillor Expenses and Resources Policy and International Relations Policy, authority from Council for the Mayor to undertake overseas travel in an official capacity is required in order to confirm the Mayor's participation in the delegation.
[2202] 
[2203] 
[2204] ## Implementation
[2205] 
[2206] Should the recommendations in this report be accepted by Council, CoMSEQ will be advised to confirm Mayor Natoli as Council's representative in the delegation. Travel arrangements will be confirmed subsequent to that advice.
[2207] 
[2208] A further report will be provided to Councillors on the outcomes of the delegation in the first quarter of 2025.
[2209] 
[2210] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2211] 
[2212] <!-- PageNumber="OM Agenda Page 51 of 88" -->
[2213] 
[2214] 
[2215] <PageNum=52 />
[2216] 
[2217] 
[2218] <PageNum=53 />
[2219] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2220] 
[2221] <!-- PageHeader="24 JULY 2024" -->
[2222] 
[2223] 8.4 2024 SPORTS FIELD MAINTENANCE FUNDING PROGRAM
[2224] 
[2225] |||
[2226] | - | - |
[2227] | File No: | Council Meetings |
[2228] | Author: | Community Connections & Partnerships Lead |
[2229] || Economic & Community Development Group |
[2230] | Appendices: | App A - 2024 Sports Field Maintenance Funding Program |
[2231] || Recommendations 59 L |
[2232] 
[2233] 
[2234] ## PURPOSE
[2235] 
[2236] This report seeks Council consideration and endorsement of recommendations for the 2024 Sports Field Maintenance Funding Program (Appendix A).
[2237] 
[2238] 
[2239] ## EXECUTIVE SUMMARY
[2240] 
[2241] The inaugural Sports Field Maintenance Funding Program and supporting guidelines were endorsed by Council at its Ordinary Meeting of 2 February 2011. The funding program commenced in the 2011/12 financial year providing funding for up to three years to successful applicants.
[2242] 
[2243] On 20 June 2013, Council adopted the Community Grants Policy under which the Sports Field Maintenance Funding Program guidelines are administered.
[2244] 
[2245] The aim of this program is to contribute to maintenance costs incurred by, and provide expert advice to, organisations maintaining turf surfaces to competition standard.
[2246] 
[2247] The program is jointly managed by the Economic and Community Development and Built Infrastructure Groups and is premised on the following four elements:
[2248] 
[2249] • The sports field maintenance undertaken ensures fields are at competition standard.
[2250] 
[2251] • The costs are attributed per field, per annum and based on the type of sport played.
[2252] 
[2253] • Council's contribution to the maintenance costs is a percentage of the full contractor rate.
[2254] 
[2255] •
[2256] 
[2257] The program is not applicable to specialised, exclusive and/or restricted access sports.
[2258] 
[2259] Pivotal to the program is the availability of a dedicated Council staff member, Parks Community Sports Field Officer (Parks and Gardens), to provide support to sporting organisations by providing expert advice on maintaining their sports field to competition standard.
[2260] 
[2261] There is one Sports Field Maintenance Funding Program round per year. The program has existing commitments to 48 sporting organisations that were awarded multiple years of funding in the 2023 round.
[2262] 
[2263] Applications for the 2024 Sports Field Maintenance Funding Program round closed on 27 May 2024 with one organisation applying to enter the program and one of the existing partner organisations requesting a variation to their funding agreement to include maintenance of an additional field.
[2264] 
[2265] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2266] 
[2267] <!-- PageNumber="OM Agenda Page 53 of 88" -->
[2268] 
[2269] 
[2270] <PageNum=54 />
[2271] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2272] 
[2273] <!-- PageHeader="24 JULY 2024" -->
[2274] 
[2275] This report recommends a total of $15,444 in funding for 2024/25 and $18,620 plus annual CPI in 2025/26 to applicant organisations as follows:
[2276] 
[2277] • $3,176 in 2024/25 and $6,352 plus annual CPI in 2025/26 for Nirimba Hurricanes Junior Rugby League Club to join the program, taking responsibility for new fields in Nirimba from March 2025.
[2278] 
[2279] • an increase of $12,268 in 2024/25 and $12,268 plus annual CPI in 2025/26 to the existing allocation to the Coolum Cricket Club Inc towards costs of maintaining an additional field at Keith Royal Oval, Marcoola.
[2280] 
[2281] Details of this recommendation are provided in Appendix A.
[2282] 
[2283] 
[2284] ## OFFICER RECOMMENDATION
[2285] 
[2286] That Council:
[2287] 
[2288] (a) receive and note the report titled "2024 Sports Field Maintenance Funding Program" and
[2289] 
[2290] (b) endorse the 2024 Sports Field Maintenance Funding Program Recommendations including two years of funding with annual CPI increases to be applied in 2025/26 (Appendix A).
[2291] 
[2292] 
[2293] ## FINANCE AND RESOURCING
[2294] 
[2295] Funding of sporting organisations through the Sports Field Maintenance Funding Program is based on a formula which considers field type, number of fields and period of responsibility and is allocated (subject to budget) for one-, two- or three-year funding periods.
[2296] 
[2297] The 2024 round is the second of the current three-year cycle. There are existing commitments of $943,991 to 48 sporting organisations that were awarded multiple years of funding in the 2023 round.
[2298] 
[2299] It was determined that a maximum of two years of funding will be allocated in the 2024 round to bring the new allocations into line with existing Sports Field Maintenance Funding Program agreements which end on 30 June 2026.
[2300] 
[2301] The 2024/25 Sports Field Maintenance Funding Program budget is sufficient to support:
[2302] 
[2303] • $3,176 for Nirimba Hurricanes Junior Rugby League Club to join the program.
[2304] 
[2305] • An increase of $12,268 to the existing agreement with Coolum Cricket Club Inc to include responsibility for the maintenance of an additional cricket field - the Keith Royal Oval, Marcoola.
[2306] 
[2307] • $943,991 that will be distributed to organisations already committed under multi-year agreements from the 2023 round.
[2308] 
[2309] All applicants are recommended for two years of funding with an annual CPI increase in year two.
[2310] 
[2311] As funding is allocated for the two (2) year funding period, the recommendations in this report have implications for future budgets (see Table 1 below).
[2312] 
[2313] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2314] 
[2315] <!-- PageFooter="OM Agenda Page 54 of 88" -->
[2316] 
[2317] 
[2318] <PageNum=55 />
[2319] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2320] 
[2321] 24 JULY 2024
[2322] 
[2323] | Sports Field Maintenance Funding Program Budget | 2024/2025 | 2025/2026 |
[2324] | - | - | - |
[2325] | 2024 round - 1st year - Recommendations in this report | $15,444 | |
[2326] | 2024 round - 2nd year - Recommendations in this report | | $18,620 + CPI |
[2327] | 2023 round - 2nd year - Existing commitments | $943,991 | |
[2328] | 2023 round - 3rd year - Existing commitments | | $943,991 + CPI |
[2329] | Total Expenditure | $959,435 | $962,611 + CPI |
[2330] 
[2331] Table 1. Budget Implications
[2332] 
[2333] 
[2334] ## CORPORATE PLAN
[2335] 
[2336] Corporate Plan Goal:
[2337] 
[2338] Our strong community
[2339] 
[2340] Outcome:
[2341] 
[2342] 1.1 Healthy and active communities
[2343] 
[2344] Operational Activity:
[2345] 
[2346] 1.4.1 - Provide support to the community and other not-for-profit
[2347] 
[2348] organisations in the delivery of resources, facilities and services
[2349] 
[2350] that meet the needs of communities through the grants program.
[2351] 
[2352] 
[2353] ## CONSULTATION
[2354] 
[2355] Portfolio Councillor Consultation
[2356] 
[2357] • Community Portfolio - Councillor T Bunnag and Councillor D Law
[2358] 
[2359] Internal Consultation
[2360] 
[2361] Consultation has been undertaken with relevant internal stakeholders across various groups and branches of Council including:
[2362] 
[2363] • Parks Community Sports Field Officer, Parks and Gardens
[2364] 
[2365] • Sports Planning and Development team, Sport and Community Venues
[2366] 
[2367] • Senior Project Officer, Community Connections, Community Planning and Development
[2368] 
[2369] • Senior Property Officer, Land Management, Property Management
[2370] 
[2371] 
[2372] ### External Consultation
[2373] 
[2374] Consultation with representatives from partner sporting organisations is undertaken on a regular basis by both the Parks Community Sports Field Officer and relevant Sport and Recreation Officers to ensure compliance with and support for their funding agreements.
[2375] 
[2376] Community Engagement
[2377] 
[2378] No external community engagement was undertaken in relation to this report.
[2379] 
[2380] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2381] 
[2382] <!-- PageNumber="OM Agenda Page 55 of 88" -->
[2383] :unselected: :unselected:
[2384] 
[2385] <PageNum=56 />
[2386] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2387] 
[2388] <!-- PageHeader="24 JULY 2024" -->
[2389] 
[2390] 
[2391] ## PROPOSAL
[2392] 
[2393] The 2024 round is the second of the current three-year cycle. The program has existing commitments to 48 sporting organisations that were awarded multiple years of funding in the 2023 round.
[2394] 
[2395] Opening on 22 April, the 2024 Sports Field Maintenance Funding Program round was promoted through direct contact with organisations identified as potentially eligible for funding as well as through Council's Grants eNews and public website.
[2396] 
[2397] Applications for the 2024 Sports Field Maintenance Funding Program round closed on 27 May with one (1) organisation applying to enter the program, and one (1) current partner organisation seeking an additional funding allocation to take on responsibility for an additional sports field. Both applicants requested two years of funding.
[2398] 
[2399] The assessment of applications for this funding program required clarification of tenure, type of sport played on the field/s, number of fields and period of maintenance responsibility. An assessment panel made up of Council officers with relevant expertise met to review the applications and determine an outcome. Assessment panel membership is detailed in the Internal Consultation section of this report.
[2400] 
[2401] This report recommends a total of $15,444 in funding for 2024/2025 and $18,620 plus annual CPI in 2025/2026 to the two applicant organisations as follows:
[2402] 
[2403] • $3,176 in 2024/25 and $6,352 plus annual CPI in 2025/26 for Nirimba Hurricanes Junior Rugby League Club to join the program, taking responsibility for two new rugby fields in Nirimba for part of the year from March 2025. This club will maintain the fields for three (3) months during 2024/25 and six (6) months during 2025/26. It is anticipated that a summer sports club will take over responsibility for the remaining six (6) months of 2025/26.
[2404] 
[2405] • an increase of $12,268 in 2024/25 and $12,268 plus annual CPI in 2025/26 to the existing allocation to the Coolum Cricket Club Inc towards costs of maintaining an additional field at Keith Royal Oval, Marcoola. This club will have responsibility for maintaining the additional cricket field for a full 12 months each year.
[2406] 
[2407] Details of this recommendation are provided in Appendix A.
[2408] 
[2409] 
[2410] ## Legal
[2411] 
[2412] The act of providing funding does not, of itself, raise issues of legal liability for Council. Rather it is the conduct of the funding recipient in carrying out what may be considered a local government type activity on local government owned or controlled land that will have some legal liability exposure issues for Council.
[2413] 
[2414] This will be mitigated, as best as is possible, by having the successful funding applicants enter into Agreements that clearly articulate the relationship between the parties and obligate them to be incorporated bodies with adequate risk management plans, levels of insurance, training, reporting and financial controls.
[2415] 
[2416] 
[2417] ## Policy
[2418] 
[2419] Delivery of the Sports Field Maintenance Funding Program is governed by the Community Grants Policy and supporting guidelines.
[2420] 
[2421] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2422] 
[2423] <!-- PageFooter="OM Agenda Page 56 of 88" -->
[2424] 
[2425] 
[2426] <PageNum=57 />
[2427] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2428] 
[2429] <!-- PageHeader="24 JULY 2024" -->
[2430] 
[2431] Risk
[2432] 
[2433] There is minimal financial risk associated with the program in supporting identified eligible organisations to carry out field maintenance, due to the ongoing working relationships between the organisations and dedicated Council officers. This relationship is supported by individually tailored funding agreements.
[2434] 
[2435] 
[2436] ## Previous Council Resolution
[2437] 
[2438] Ordinary Meeting 27 July 2023 (OM23/69)
[2439] 
[2440] That Council:
[2441] 
[2442] (a) receive and note the report titled "2023 Sports Field Maintenance Funding Program" and
[2443] 
[2444] (b) endorse the 2023 Sports Field Maintenance Funding Program Recommendations including three years of funding with annual CPI increases to be applied in 2024/25 and 2025/26 (Appendix A).
[2445] 
[2446] 
[2447] ## Ordinary Meeting 20 June 2013 (OM13/109)
[2448] 
[2449] That Council:
[2450] 
[2451] (a) receive and note the report titled "Community Grants Policy"
[2452] 
[2453] (b) adopt the Community Grants Policy (Appendix A)
[2454] 
[2455] (c) note the Community Grants Guidelines (Appendix B) as amended by (f) below to implement the Community Grants Policy
[2456] 
[2457] (d) adopt the Mayoral and Councillor Discretionary Funding Policy as amended (Appendix C)
[2458] 
[2459] (e) note the Mayoral and Councillor Discretionary Funding Program Guidelines (Appendix D) and
[2460] 
[2461] (f) amend the grants guidelines to include a clause that stipulates that each program is subject to annual budget allocations.
[2462] 
[2463] 
[2464] ### Ordinary Meeting 2 February 2011 (OM11/12)
[2465] 
[2466] That Council:
[2467] 
[2468] (a) receive and note the report titled "Sports Field Maintenance Funding Program"
[2469] 
[2470] (b) adopt a 70% Council funding contribution of the full contractor rate to maintain fields noting potential funding sources and refer funding to 2011/12 budget process
[2471] 
[2472] (c) adopt the Sports Field Maintenance Funding Program Policy (Appendix A) and note Guidelines (Appendix B) and
[2473] 
[2474] (d) request the Chief Executive Officer to review the Sports Field Maintenance Funding Program one year after adoption and provide a progress report back to Council.
[2475] 
[2476] 
[2477] #### Related Documentation
[2478] 
[2479] Funding arrangements with individual organisations include individual agreements which must demonstrate field maintenance plans. These plans are formulated in consultation with the Parks Community Sports Field Officer.
[2480] 
[2481] There are related policies and governing legislation that guide and inform the management and delivery of the Sports Field Maintenance Funding Program. These include:
[2482] 
[2483] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2484] 
[2485] <!-- PageNumber="OM Agenda Page 57 of 88" -->
[2486] 
[2487] 
[2488] <PageNum=58 />
[2489] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2490] 
[2491] 24 JULY 2024
[2492] 
[2493] • Local Government Act (2009)
[2494] 
[2495] • Local Government Regulation (2012)
[2496] 
[2497] • Sunshine Coast Community Strategy 2019-2041
[2498] 
[2499] · Community Grants Policy (2013)
[2500] 
[2501] • Environment and Liveability Strategy 2023
[2502] 
[2503] • Corporate Plan 2024-2028
[2504] 
[2505] • Sport and Active Recreation Plan 2011-2026
[2506] 
[2507] • Sports Field Maintenance Funding Program Guidelines
[2508] 
[2509] 
[2510] ### Critical Dates
[2511] 
[2512] The due date for acquittals of 2024/2025 Sports Field Maintenance Funding Program allocations is 31 August 2025.
[2513] 
[2514] 
[2515] #### Implementation
[2516] 
[2517] Upon Council endorsement of the recommendations in this report, the applicants will be notified of the recommended funding amount, funding conditions including reporting and acquittal requirements and any conditions to be met prior to funding being processed.
[2518] 
[2519] The recipients will also develop a maintenance agreement in consultation with Council's Parks Community Sports Field Officer, clarifying expectations in relation to the maintenance of the sports fields.
[2520] 
[2521] Funding will be distributed once the transfer of responsibility is confirmed and the online agreement is submitted to Council. All recipients will be required to display Council supplied signage acknowledging Council's support for field maintenance.
[2522] 
[2523] Details of the successful partner organisations will be posted on Council's website.
[2524] 
[2525] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2526] 
[2527] <!-- PageNumber="OM Agenda Page 58 of 88" -->
[2528] 
[2529] 
[2530] <PageNum=59 />
[2531] <!-- PageHeader="24 JULY 2024" -->
[2532] 
[2533] Ordinary Meeting Agenda Item 8.4 2024 Sports Field Maintenance Funding Program Appendix A
[2534] 
[2535] 2024 Sports Field Maintenance Funding Program Recommendations
[2536] 
[2537] | Application ID | Organisation Name | Property Owner | Primary sport played on fields | No. of effective fields | Period of maintenance (months) | Amount Recommended 2024/25 | Amount Recommended 2025/26 | Period of funding | Division |
[2538] | - | - | - | - | - | - | - | - | - | - |
[2539] | SFMP24001 | Nirimba Rugby League Club Inc. | Council | Rugby League | 2 | 3 months - 2024/25 6 months - 2025/26 | $3,176 | $6,352 + CPI | 2 years | Div.1 |
[2540] | Variation to SFMP23047 | Coolum Cricket Club Inc | Council | Cricket | 1 | 12 months | $12,268 | $12,268 + CPI | 2 years | Div. 8 |
[2541] 
[2542] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2543] 
[2544] <!-- PageNumber="OM Agenda Page 59 of 88" -->
[2545] 
[2546] 2024 Sports Field Maintenance Funding Program Recommendations
[2547] 
[2548] 
[2549] <PageNum=60 />
[2550] 
[2551] 
[2552] <PageNum=61 />
[2553] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2554] 
[2555] <!-- PageHeader="24 JULY 2024" -->
[2556] 
[2557] 8.5 EXCEPTION UNDER LOCAL GOVERNMENT REGULATION 2012 FOR THE DISPOSAL OF AN INTEREST IN LAND, BELLS CREEK
[2558] 
[2559] |||
[2560] | - | - |
[2561] | File No: | Council Meetings |
[2562] | Author: | Senior Property Officer |
[2563] | | Business Performance Group |
[2564] 
[2565] 
[2566] ## PURPOSE
[2567] 
[2568] In accordance with Local Government Regulation 2012, this report seeks Council's endorsement to provide an exception to the tender or auction process for disposing of an interest in land. The purpose of which is to facilitate a lease with Police Citizens Youth Clubs Queensland (PCYC) for the development of a community facility in Bells Creek.
[2569] 
[2570] 
[2571] ## EXECUTIVE SUMMARY
[2572] 
[2573] Police Citizens Youth Club (PCYC) Queensland is a charity providing youth and community programs, services and facilities across Queensland. The organisation partners with Queensland Police Service to focus on personal and leadership development of young people, through activities and programs delivered over 56 facilities across the State.
[2574] 
[2575] During the latter half of 2023, Police Citizens Youth Club began discussions with Council and Stockland regarding opportunities for establishing a community facility in Caloundra South. Police Citizens Youth Club expressed specific location requirements to fit its business model. These site requirements included a location that is well serviced by public transport, has close proximity to activity centres and importantly, close proximity to a police station.
[2576] 
[2577] Police Citizens Youth Club have identified a 6500m2 area of land to the east of the Emergency Services Precinct in the future Aura Town Centre that is desirable for their purpose. The site is proximate to a future police station, fire station, ambulance station and the future Aura Train Station.
[2578] 
[2579] However, the site does not form part of the planned community facilities network to be created and contributed to Council as part of the Aura development. The site is an opportunistic prospect identified by Police Citizens Youth Club.
[2580] 
[2581] Stockland are agreeable to the subject land being used for Police Citizens Youth Club's proposal but this will necessitate a redistribution of planned community facility sites within the broader Aura development. Council's Open Space & Social Policy team have reviewed the proposal to identify any impacts to the provision of future community facilities in the network, confirming that two planned neighbourhood meeting room sites and one planned local community centre site could instead be delivered as a single facility on the 6,500m2 site within the future town centre. The proposal by Police Citizens Youth Club does not adversely impact the overall community facilities network as there will be no net loss of community facilities land. To ensure that the endorsed community facility network outcomes are achieved, it would be anticipated that the Police Citizens Youth Club development integrates flexible and multipurpose bookable spaces in accordance with Council's minimum Desired Standards of Service requirement for local community venues.
[2582] 
[2583] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2584] 
[2585] <!-- PageFooter="OM Agenda Page 61 of 88" -->
[2586] 
[2587] 
[2588] <PageNum=62 />
[2589] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2590] 
[2591] <!-- PageHeader="24 JULY 2024" -->
[2592] 
[2593] The proposal to site the Police Citizens Youth Club facility in the town centre, including a redistribution of sites within the community facilities network will require an amendment to the Caloundra South PDA Infrastructure Agreement (Local Government Infrastructure) 2015 (the "IA"). This amendment can be progressed as a Deed of Variation to the IA.
[2594] 
[2595] The Deed of Variation will ultimately be executed by Stockland, Council and Economic Development Queensland (EDQ) as parties to the IA. It is noted that Economic Development Queensland have given 'in principle' support for the Deed of Variation, given its minor nature.
[2596] 
[2597] Once the lot is created, it is intended to be transferred to Council under a nomination of trust, which will enable Council to enter into a long-term community lease to facilitate third party capital investment. The final leasing terms between Council and Police Citizens Youth Club are yet to be negotiated, finalised and executed.
[2598] 
[2599] The Local Government Regulations 2012 prescribe that Council must dispose of an interest in land (including a lease) in a particular way, being via tender or auction. However, there are exceptions to this requirement should Council agree by resolution that the exception applies. Section 236(1)(b)(ii) of the Local Government Regulations 2012 states that a relevant exception to dispose of an interest in land, other than by tender or auction, may apply where the disposal is to a community organisation.
[2600] 
[2601] Important considerations for this proposal are as follows:
[2602] 
[2603] • Police Citizens Youth Club is a community organisation,
[2604] 
[2605] • The proposal has been initiated by Police Citizens Youth Club for land identified by them as suitable, and confirmed by relevant Council teams as feasible within the planned community facilities network,
[2606] 
[2607] • The site has unique attributes that meets Police Citizens Youth Club's operating model, including proximity to a police station,
[2608] 
[2609] • Police Citizens Youth Club have demonstrated a funding mechanism for delivery of the community facility.
[2610] 
[2611] 
[2612] ## OFFICER RECOMMENDATION
[2613] 
[2614] That Council:
[2615] 
[2616] (a) receive and note the report titled "Exception Under Local Government Regulation 2012 for the Disposal of an Interest in Land, Bells Creek "
[2617] 
[2618] (b) resolve, pursuant to section 236(2) of the Local Government Regulation 2012, that an exception to dispose of an interest in the identified land, other than by tender or auction applies, as the disposal is pursuant to section 236(1)(b)(ii) to a community organisation and
[2619] 
[2620] (c) note that Police Citizens Youth Club Queensland is a community organisation.
[2621] 
[2622] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2623] 
[2624] <!-- PageNumber="OM Agenda Page 62 of 88" -->
[2625] 
[2626] 
[2627] <PageNum=63 />
[2628] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2629] 
[2630] <!-- PageHeader="24 JULY 2024" -->
[2631] 
[2632] 
[2633] ## FINANCE AND RESOURCING
[2634] 
[2635] This proposal seeks to deliver a contemporary community facility on community purpose land at no cost to Council. Council is making a land contribution only (by way of a peppercorn lease) and will have no ongoing operating expenses attributed to the facility. The Queensland Government has publicly announced $15 million in funding for a new Police Citizens Youth Club facility in Caloundra.
[2636] 
[2637] 
[2638] ## CORPORATE PLAN
[2639] 
[2640] Corporate Plan Goal:
[2641] 
[2642] Our strong community
[2643] 
[2644] Outcome:
[2645] 
[2646] We serve our community by providing this great service
[2647] 
[2648] Operational Activity:
[2649] 
[2650] S2 - Community venues - providing, managing, programming and
[2651] 
[2652] administering community and cultural venues including the Events
[2653] 
[2654] Centre, Venue 114 and other community spaces.
[2655] 
[2656] 
[2657] ## CONSULTATION
[2658] 
[2659] Councillor Consultation
[2660] 
[2661] • Councillor J Broderick - Division 1 Councillor
[2662] 
[2663] • Councillor T Bunnag - Portfolio Councillor Strong Community
[2664] 
[2665] • Councillor D Law - Portfolio Councillor Strong Community
[2666] 
[2667] Internal Consultation
[2668] 
[2669] • Manager Community Development, Economic & Community Development
[2670] 
[2671] • Coordinator Open Space & Social Policy, Liveability & Natural Assets
[2672] 
[2673] • Project Manager Urban Growth Projects, Customer & Planning Services
[2674] 
[2675] • Manager Leasing and Land Management, Business Performance
[2676] 
[2677] • Senior Solicitor Property & Commercial, Civic Governance
[2678] 
[2679] External Consultation
[2680] 
[2681] • Stockland
[2682] 
[2683] • Economic Development Queensland
[2684] 
[2685] 
[2686] ### Community Engagement
[2687] 
[2688] There has been no specific community engagement undertaken for this particular report. However, during June and July 2022, Council undertook a community engagement program ("Our People, Our Place, Our Aura") to understand the local community's values and vision, opportunities and challenges, and ideas to build a vibrant and connected community. It also investigated what community facilities, programs and activities people would like available locally. While community venues were not highlighted as a current priority for participants, a desire was expressed for places and spaces for the community to come together.
[2689] 
[2690] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2691] 
[2692] <!-- PageNumber="OM Agenda Page 63 of 88" -->
[2693] 
[2694] 
[2695] <PageNum=64 />
[2696] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2697] 
[2698] <!-- PageHeader="24 JULY 2024" -->
[2699] 
[2700] 
[2701] ## PROPOSAL
[2702] 
[2703] Police Citizens Youth Club (PCYC) Queensland is a charity providing youth and community programs, services and facilities across Queensland. The organisation partners with Queensland Police Service to focus on personal and leadership development of young people, through activities and programs delivered across 56 facilities across the State.
[2704] 
[2705] A Police Citizens Youth Club facility typically consists of spaces designed to provide youth and community programs often delivered through sport, fitness, dance, wellbeing initiatives and after school care. The proposed Club will support community safety and encourage an active and healthy lifestyle providing a number of benefits to the local and surrounding growing community:
[2706] 
[2707] These include a range of programs that are available at no charge to young people and their families including basketball and or boxing after dark, youth leadership team, youth drop in, and brake the cycle driver mentoring program. The facility will also consist of a multi-purpose space for community sport and recreation hire.
[2708] 
[2709] During the latter half of 2023, Police Citizens Youth Club began discussions with Council and Stockland regarding opportunities for establishing a community facility in Caloundra South. Police Citizens Youth Club expressed specific location requirements to fit its business model. These site requirements included a location that is well serviced by public transport, has close proximity to activity centres and importantly, close proximity to a police station.
[2710] 
[2711] Recently, Council has prepared an Expression of Interest package for five parcels of land designated for community facilities throughout the Aura development. The Expression of Interest sought activation of the five sites, located in Baringa and Nirimba, through partnership opportunities with community providers.
[2712] 
[2713] After some consideration, Police Citizens Youth Club advised Council that none of the five sites included in Council's Expression of Interest package were suitable as they did not meet their location requirements. However, Police Citizens Youth Club have identified a 6500m2 area of land to the east of the Emergency Services Precinct in the future Aura Town Centre that is desirable for their purpose. The site is proximate to a future police station, fire station, ambulance station and the future Aura Train Station (planned as part of the Queensland Government's Direct Sunshine Coast Rail Line). A map of the nominated site, described as proposed Lot 8048, is shown in Figure 1.
[2714] 
[2715] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2716] 
[2717] <!-- PageNumber="OM Agenda Page 64 of 88" -->
[2718] 
[2719] 
[2720] <PageNum=65 />
[2721] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2722] 
[2723] 24 JULY 2024
[2724] 
[2725] <figure>
[2726] 
[2727] ![](figures/18)<!-- FigureContent="The image is a map that shows the location of a proposed site in Bells Creek, labelled as "Proposed Lot 8048". 
[2728] 
[2729] The map provides a broad overview of the Bells Creek area, including surrounding roads, landmarks, and natural features such as rivers or creeks. There is a red arrow pointing from the location on the main map to an enlarged inset map for more detail. The enlarged inset map highlights the specific area of "Proposed Lot 8048" within a defined boundary that is marked in red. On the main map, notable locations such as Corbould Park, Caloundra West, Nirimba, and various creeks like Bells Creek and Mallee Creek are identified. 
[2730] 
[2731] The inset map details the exact placement of "Proposed Lot 8048" along a road and adjacent properties or features that are part of the proposed site. The maps are primarily in muted colors with important markings highlighted in red to draw attention." --></figure>
[2732] 
[2733] 
[2734] The subject site (shown in further detail in Figure 2) is currently part of an englobo lot owned by Stockland and is yet to be surveyed and created. The site does not form part of the planned community facilities network to be created and contributed to Council as part of the Aura development. The site is an opportunistic prospect identified by Police Citizens Youth Club.
[2735] 
[2736] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2737] 
[2738] <!-- PageFooter="OM Agenda Page 65 of 88" -->
[2739] 
[2740] 
[2741] <PageNum=66 />
[2742] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2743] 
[2744] <figure>
[2745] 
[2746] ![](figures/19)<!-- FigureContent="The image is a site plan illustrating the location and dimensions of various proposed lots in a development area. The plan is marked with specific lot numbers, sizes, and allocated purposes for each lot. Key features include:
[2747] 
[2748] - The proposed Lot 8048 is depicted in blue, situated near the right center side of the plan, and labeled with an area of 6,500 square meters.
[2749] - Several other lots are shown with various designations and areas:
[2750]   - Lot 80045 is labeled for the "POLICE" and has an area of 5,400 square meters.
[2751]   - Lot 80044 is labeled for the "FIRE" department and has an area of 6,003 square meters.
[2752]   - Lot 80043 is designated for the "AMBULANCE" and has an area of 5,308 square meters.
[2753]   - Lot 80025 is labeled "HEALTH" and covers an area of 2.05 hectares.
[2754]   - Additional lots are labeled with their numbers and areas without specific designations (e.g., Lot 80042 with 6,333 square meters, Lot 80023 with 29,364 square meters, and Lot 80024 with 31,981 square meters).
[2755] - The site includes several roadways with specified widths, providing access to the lots.
[2756] - The locations of various "Stage" boundaries are also noted (e.g., Stage 60b, Stage 61b, Stage 61c, Stage 82, and Stage 83).
[2757] 
[2758] Overall, the map provides a detailed overview of the layout and distribution of the proposed lots within the development area." --></figure>
[2759] 
[2760] 
[2761] The Queensland Government has publicly announced $15 million in funding for a new Police Citizens Youth Club facility in Caloundra. This announcement has brought into focus the need for Police Citizens Youth Club to finalise a site location.
[2762] 
[2763] Stockland are agreeable to the subject land being used for Police Citizens Youth Club's proposal, but this will necessitate a redistribution of planned community facility sites within the broader Aura development. Council's Open Space & Social Policy team have reviewed the proposal to identify any impacts to the provision of future community facilities in the network, confirming that two planned neighbourhood meeting room sites and one planned local community centre site could instead be delivered as a single facility on the 6,500m2 site within the future town centre. The proposal by Police Citizens Youth Club does not adversely impact the overall community facilities network as there is no net loss of community facilities land. To ensure that the endorsed community facility network outcomes are achieved, it
[2764] 
[2765] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2766] 
[2767] <!-- PageFooter="OM Agenda Page 66 of 88" -->
[2768] 
[2769] 24 JULY 2024
[2770] 
[2771] 
[2772] <PageNum=67 />
[2773] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2774] 
[2775] <!-- PageHeader="24 JULY 2024" -->
[2776] 
[2777] would be anticipated that the Police Citizens Youth Club development integrates flexible and multipurpose bookable spaces in accordance with Council's minimum Desired Standards of Service requirement for local community venues.
[2778] 
[2779] The proposal to site the Police Citizens Youth Club facility in the town centre, including a redistribution of sites within the community facilities network will require an amendment to the Caloundra South PDA Infrastructure Agreement (Local Government Infrastructure) 2015 (the "IA"). This amendment can be progressed as a Deed of Variation to the IA. The effect of the change will primarily be to update Map 5 - Community Facilities Infrastructure Network (refer Figures 3 and 4) and the Infrastructure Contribution Schedule - Community Facilities Infrastructure to identify the redistribution of LC5 (5,000sqm), NM5 (750sqm) and NM6 (750sqm) to a single allotment located within Precinct 8 and identified as LC5.
[2780] 
[2781] <figure>
[2782] 
[2783] ![](figures/20)<!-- FigureContent="This image is a detailed map titled "Map 5: Current Map - Community Facilities Infrastructure". It delineates various community facilities within a specific area, divided into several numbered precincts ranging from 1 to 19.
[2784] 
[2785] Here's a breakdown of the important components of the map:
[2786] 
[2787] - **Facilities**:
[2788]   - **Neighbourhood Meeting House/Rooms (NM)**: NM1 to NM14 scattered throughout the map in different precincts.
[2789]   - **Local Community Centre (LC)**: LC1 to LC7 strategically located across the map.
[2790]   - **District Community Centre (DC)**: DC1 to DC3 placed in different areas.
[2791]   - **Major Civic Centre (MCC)**: One labelled MCC in the map.
[2792]   - **Major Sports Facility (MSF)**: One labelled MSF in the map.
[2793]   - **Eco Community Facility (ECF)**: One labelled ECF appearing near the top centre of the map.
[2794] 
[2795] - **Boundaries and Labels**:
[2796]   - **PDA Boundary**: Represented by a bold, dashed black line encircling the map area.
[2797]   - **Precincts**: Shown by thinner, dashed black lines separating the various numbered precincts.
[2798]   
[2799] - **Additional Notes**: 
[2800]   - There's an explanatory note in the bottom right corner of the map which explains that land use boundaries are general and subject to change through detailed investigations and confirmed through subsequent approvals.
[2801]   
[2802] - **Natural Features**:
[2803]   - A blue line runs across the map, indicating a water body (likely a river or stream) crossing through multiple precincts.
[2804] 
[2805] The community facilities are well distributed across the area to ensure accessibility. Each precinct contains a different mix of facilities, showing a planned infrastructure network aimed at serving the community's needs." --></figure>
[2806] 
[2807] 
[2808] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2809] 
[2810] <!-- PageFooter="OM Agenda Page 67 of 88" -->
[2811] :unselected:
[2812] 
[2813] <PageNum=68 />
[2814] <!-- PageHeader="24 JULY 2024" -->
[2815] 
[2816] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2817] 
[2818] <figure>
[2819] 
[2820] ![](figures/21)<!-- FigureContent="The image is a detailed map titled "Proposed Amended Map 5 - Community Facilities Infrastructure Network," which forms part of the Caloundra South Infrastructure Agreement. It depicts a geographic area divided into multiple precincts, each labeled with a unique number from 1 to 19. 
[2821] 
[2822] Key features of the map include:
[2823] 
[2824] - **Legend:** The legend is located in the bottom right-hand corner and explains the symbols used on the map, including:
[2825]   - PDA Boundary
[2826]   - Precincts
[2827]   - Neighborhood Meeting House / Rooms (NM1-NM14)
[2828]   - Local Community Centre (LC1-7)
[2829]   - District Community Centre (DC1-3)
[2830]   - Major Civic Centre (MCC)
[2831]   - Major Sports Facility (MSF)
[2832]   - Eco Community Facility (ECF)
[2833]   - Police-Citizens Youth Club (PCYC)
[2834] 
[2835] - **Major Roads and Landmarks:**
[2836]   - The map includes major roads such as Bruce Highway, Caloundra Road, Bells Creek Road, and East-West Link Road.
[2837]   - The Caloundra Airport is marked in the upper right section.
[2838]   - A proposed CAMCOS Rail Corridor is detailed on the western side.
[2839]   - Major interchanges like Bells Creek Arterial / Roy’s Road Interchange are also noted.
[2840] 
[2841] - **Community Facilities:**
[2842]   - Various types of community facilities are marked in different precincts using codes from the legend, such as NM (Neighborhood Meeting Rooms), LC (Local Community Centre), DC (District Community Centre), MCC (Major Civic Centre), MSF (Major Sports Facility), and ECF (Eco Community Facility). 
[2843] 
[2844] - **Natural and Geographical Features:**
[2845]   - The map shows natural features like Bells Creek and other unnamed watercourses and green areas.
[2846] 
[2847] - **Other Details:**
[2848]   - The scale of the map is provided (1:35,000 @ A3).
[2849]   - Project details, including the Project Number, Date, and version history, are at the bottom right.
[2850]   - A north direction indicator is at the bottom right corner.
[2851] 
[2852] Overall, the map is a well-structured and detailed representation of the proposed layout for community facilities infrastructure within the Caloundra South area, showing existing and proposed elements for public and social infrastructure." --></figure>
[2853] 
[2854] 
[2855] Stockland will prepare a draft Deed of Variation that captures the details of the change to the Map and Schedule, and this draft will then be reviewed by Council's legal team. The Deed of Variation will ultimately be executed by Stockland, Council and Economic Development (EDQ) Queensland as parties to the IA. It is noted that Economic Development Queensland have given 'in principle' support for the Deed of Variation, given its minor nature.
[2856] 
[2857] In addition, it is anticipated that the Caloundra South Community Facilities Network Plan will be updated to reflect the changes to the network as a result of the Police Citizens Youth Club proposal.
[2858] 
[2859] The subject site is intended to be created via a Reconfiguration of a Lot approval and minor change to the Caloundra South Precinct 7-10 Plan of Development to reflect any required administrative changes to permit the Police Citizens Youth Club to establish a facility in this location.
[2860] 
[2861] Once the lot is created, it is intended to be transferred to Council under a nomination of trust, which will enable Council to enter into a long term lease to facilitate third party capital investment. The final leasing terms between Council and Police Citizens Youth Club are yet to be negotiated.
[2862] 
[2863] Preliminary design for the proposed facility includes spaces dedicated to gymnasium, gymnastics, multipurpose spaces, youth programs, meetings rooms, outdoor basketball courts and running track. To remain in line with Council's strategic network directions and partnership approach for delivery of community facilities, the future proposed Police Citizens Youth Club facility will be required to incorporate a community venue space bookable by the community.
[2864] 
[2865] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2866] 
[2867] <!-- PageFooter="OM Agenda Page 68 of 88" -->
[2868] 
[2869] 
[2870] <PageNum=69 />
[2871] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2872] 
[2873] <!-- PageHeader="24 JULY 2024" -->
[2874] 
[2875] A preliminary floorplan and concept design of the proposed facility is shown in Figure 5.
[2876] 
[2877] <figure>
[2878] 
[2879] ![](figures/22)<!-- FigureContent="The image consists of two main sections: a detailed site plan and a set of perspective renderings of a community or recreational facility.
[2880] 
[2881] **Site Plan (Left Side):**
[2882] 1. **Basketball Courts**: Located in the top left corner, there are two outlined basketball courts marked in purple. The shape is irregular, incorporating pathways around them.
[2883] 2. **Building Footprint**: The building is color-coded with multiple labeled sections; these include areas marked as gym (500 sqm), gymnasium (800 sqm), youth area, multipurpose rooms (MP1, MP2), offices, hall, pantry, and various other facilities.
[2884] 3. **Parking Area**: The right side of the plan is dominated by a parking lot, organized with rows of parking spaces interspersed with trees.
[2885] 4. **Landscaping**: The entire site includes green areas with trees and pathways around the building and along the boundaries.
[2886] 
[2887] **Perspective Renderings (Right Side):**
[2888] 1. **Top Image**: Shows an aerial perspective of the entire building complex, highlighting the roofing, surrounding parking area, and detailed landscaping.
[2889] 2. **Middle Image**: Presents a ground-level view, showcasing the main entrance of the building, the greenery, and the parking lot. There are people walking and cars parked around.
[2890] 3. **Bottom Image**: Provides a closer view of the entrance, highlighting architectural features of the building's façade and additional landscaping elements.
[2891] 
[2892] The overall design suggests a focus on recreational activities and community gathering, with ample space for sports, multipurpose usage, and parking." --></figure>
[2893] 
[2894] 
[2895] Figure 5: Preliminary Floorplan and Concept Design of Proposed PCYC Facility
[2896] 
[2897] The Local Government Regulations 2012 prescribe that Council must dispose of an interest in land (including a lease) in a particular way, being via tender or auction. However, there are exceptions to this requirement should Council agree by resolution that the exception applies. Section 236(1)(b)(ii) of the Local Government Regulations 2012 states that a relevant exception to dispose of an interest in land, other than by tender or auction, may apply where the disposal is to a community organisation.
[2898] 
[2899] Important considerations for this proposal are as follows:
[2900] 
[2901] • Police Citizens Youth Club is a community organisation,
[2902] 
[2903] • The proposal has been initiated by Police Citizens Youth Club for land identified by them as suitable, and confirmed by relevant Council teams as feasible within the planned community facilities network,
[2904] 
[2905] • The site has unique attributes that meets Police Citizens Youth Club's operating model, including proximity to a police station,
[2906] 
[2907] • Police Citizens Youth Club have demonstrated a funding mechanism for delivery of the community facility.
[2908] 
[2909] Legal
[2910] 
[2911] The subject land is intended to be contributed to Council as freehold land under a nomination of trust for community purposes. The development of the site is subject to the Trusts Act 1973. The development is required to be in accordance with the trust purpose specific to the site. The development is further required to be in accordance with the planning requirements of the relevant planning instruments, in this case the Caloundra South Precinct 7-10 Plan of Development. The planning requirements may be amended subject to due processes and approvals.
[2912] 
[2913] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2914] 
[2915] <!-- PageFooter="OM Agenda Page 69 of 88" -->
[2916] :selected: :selected:
[2917] 
[2918] <PageNum=70 />
[2919] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2920] 
[2921] <!-- PageHeader="24 JULY 2024" -->
[2922] 
[2923] The Local Government Regulation 2012 provides the processes by which Council may dispose of interests in land.
[2924] 
[2925] Specific terms for the proposed future lease of land between Council and Police Citizens Youth Club are yet to be negotiated.
[2926] 
[2927] 
[2928] ### Policy Community Strategy
[2929] 
[2930] The Community Strategy provides a longer-term framework which reflects a shared responsibility for how Council and our community will work together to advance our shared goal for a strong community through to 2041. One purpose of the Community Strategy is to focus on ensuring community places and spaces are vibrant, inclusive, accessible, adaptable, and meet the needs of people of all ages, abilities and backgrounds.
[2931] 
[2932] Identified actions within the Community Strategy Action Plan include:
[2933] 
[2934] • Collaborate and partner to provide increased access to community infrastructure connecting people to information, services and programs.
[2935] 
[2936] • Collaborate and partner with community and neighbourhood centres, community organisations and local communities to identify opportunities and challenges to inform community infrastructure provision and planning responses.
[2937] 
[2938] 
[2939] #### Environment & Liveability Strategy
[2940] 
[2941] The Sunshine Coast Environment & Liveability Strategy 2017 provides the overarching framework for the planning and delivery of the social infrastructure network. The proposal is in accordance with the strategic direction to prioritise the development of facilities at the District and Council-wide level whilst enabling access to facilities at the local level through partnerships and advocacy.
[2942] 
[2943] 
[2944] ##### Caloundra South Community Facilities Plan
[2945] 
[2946] The Caloundra South Community Facilities Plan 2018 was prepared to respond to a requirement of the Caloundra South Local Government Infrastructure Agreement and provides a further direction for the implementation of the social infrastructure network within the Caloundra South Priority Development Area, in accordance with the directions of the Environment and Liveability Strategy.
[2947] 
[2948] Community Groups Occupying Council Owned and Council Controlled Land and/or Infrastructure
[2949] 
[2950] The policy seeks to establish a structured, consistent and transparent approach to the provision and management of community occupancy arrangements over Council owned and Council controlled land and/or infrastructure.
[2951] 
[2952] This approach maximises use of community land and infrastructure and clarifies roles and responsibilities to expedite decision making and ultimately build resilient, strengthened communities.
[2953] 
[2954] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2955] 
[2956] <!-- PageNumber="OM Agenda Page 70 of 88" -->
[2957] 
[2958] 
[2959] <PageNum=71 />
[2960] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[2961] 
[2962] <!-- PageHeader="24 JULY 2024" -->
[2963] 
[2964] Risk
[2965] 
[2966] The proposal is applicant-led and over land not previously contemplated for the proposed use. Council has assessed the viability of the proposal and whether it can be accommodated without any detrimental impact to the proposed community facilities network.
[2967] 
[2968] The transactional terms are yet to be negotiated and agreed but it is likely that Council will insist on an agreement for lease to cover the period of planning, securing funding, approvals and constructions, and a further lease to cover the period of facility operations. This approach enables Council to build in milestone dates for the development of the site to ensure that a venue is delivered by a particular date.
[2969] 
[2970] Previous Council Resolution
[2971] 
[2972] There is no previous Council resolution relevant to this report.
[2973] 
[2974] Related Documentation
[2975] 
[2976] The Our People, Our Places, our Aura Community Engagement Program Findings Summary Report provides an overview of the community engagement results for people who live, work or play in Aura, and the recommended next steps.
[2977] 
[2978] https://haveyoursay.sunshinecoast.qld.gov.au/aura-ce
[2979] 
[2980] 
[2981] ###### Critical Dates
[2982] 
[2983] There are no critical dates relevant to this report. However, The Queensland Government has publicly announced $15 million in funding for a new Police Citizens Youth Club facility in Caloundra. This announcement has brought into focus the need for Police Citizens Youth Club to finalise a site location.
[2984] 
[2985] 
[2986] ## Implementation
[2987] 
[2988] Should the recommendation be accepted by Council, it is noted that the Chief Executive Officer will delegate to Council officers the negotiation and execution of lease(s) necessary to facilitate the proposal.
[2989] 
[2990] <!-- PageFooter="Sunshine Coast Regional Council" -->
[2991] 
[2992] <!-- PageNumber="OM Agenda Page 71 of 88" -->
[2993] 
[2994] 
[2995] <PageNum=72 />
[2996] 
[2997] 
[2998] <PageNum=73 />
[2999] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3000] 
[3001] <!-- PageHeader="24 JULY 2024" -->
[3002] 
[3003] 8.6 SUNSHINE COAST LIGHTNING SPONSORSHIP AGREEMENT
[3004] 
[3005] |||
[3006] | - | - |
[3007] | File No: | Council Meetings |
[3008] | Author: | Head of Economic Development Economic & Community Development Group |
[3009] | Attachments: | Att 1 - Sunshine Coast Lightning Sponsorship Proposal 2025- 2027 - Confidential |
[3010] 
[3011] 
[3012] ## PURPOSE
[3013] 
[3014] This report seeks Council's endorsement to continue sponsorship of the Sunshine Coast Lightning netball team for a further three seasons from 2025-2027.
[3015] 
[3016] Attachment 1 is confidential as it contains commercial in confidence information and confidential information relating to third parties. It also recognises that until Council makes a decision and a contract is executed, the content and timeframes are not final.
[3017] 
[3018] For these reasons, it is recommended that should Councillors wish to discuss the information in the confidential attachment, Council resolve to close the meeting under section 254J(3)(g) of the Local Government Regulation 2012 as the attachment contains information on negotiations relating to commercial matters involving the local government for which a public discussion would be likely to prejudice the interest of the local government.
[3019] 
[3020] Specifically, the matters that are considered confidential in the attachments and which would be the subject of discussion when the meeting is closed relate to the contract terms and conditions that are still the subject of negotiation with Sunshine Coast Lightning.
[3021] 
[3022] 
[3023] ## EXECUTIVE SUMMARY
[3024] 
[3025] Council has been a key strategic partner in the development and resultant success of the Sunshine Coast Lightning netball team for the past eight years. Council approved a three- year sponsorship agreement with Sunshine Coast Lightning in 2016, supplying both financial, practical and strategic support for the development of Australia's newest professional sports team.
[3026] 
[3027] In 2019, as the terms of the initial contract were reaching a conclusion, Council approved a two-year extension to the sponsorship agreement for the same investment.
[3028] 
[3029] In 2021, as the terms of the 2021-2022 agreement were reaching a conclusion, Council approved a three-year extension to the sponsorship for the 2023-2024 seasons.
[3030] 
[3031] This current agreement will expire at the conclusion of the 2024 season, which commenced in April 2024 and concludes with finals rounds held in August 2024.
[3032] 
[3033] Council has received a proposal and request from Sunshine Coast Lightning for the continuation of the sponsorship agreement for a further three seasons (2025-2027) to continue the partnership to achieve further on and off field successes and secure the club's future.
[3034] 
[3035] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3036] 
[3037] <!-- PageFooter="OM Agenda Page 73 of 88" -->
[3038] 
[3039] 
[3040] <PageNum=74 />
[3041] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3042] 
[3043] <!-- PageHeader="24 JULY 2024" -->
[3044] 
[3045] Sunshine Coast Lightning commenced playing in the Australian Super Netball Championship's first season in 2017, becoming the most successful start-up sporting franchise in Australia's history by winning the premiership in their first two seasons (2017 and 2018), and continuing to be successful on court, competing in the Super Netball Finals in 2019 and 2020.
[3046] 
[3047] In addition to their on-court success, Sunshine Coast Lightning have delivered excellent community and social benefits for the region over the past eight years through their partnership with Council.
[3048] 
[3049] In the development of Sunshine Coast Lightning's sponsorship proposal to Council for the 2025-2027 seasons, Council's Tourism and Major Events team have worked directly with Sunshine Coast Lightning to further evolve the benefits provided to Council through the sponsorship as outlined in the proposal section below.
[3050] 
[3051] The proposal provided to Council strongly demonstrates Sunshine Coast Lightning's understanding of Council's key goals and desired outcomes, and their willingness to work closely with Council to build and promote the region as a high-performance sport destination and the benefits that having a professional sports team in the region provides to the local community.
[3052] 
[3053] 
[3054] ## OFFICER RECOMMENDATION
[3055] 
[3056] That Council:
[3057] 
[3058] (a) receive and note the report titled "Sunshine Coast Lightning Sponsorship Agreement" and
[3059] 
[3060] (b) authorise the Chief Executive Officer to negotiate and execute a contract between Council and Sunshine Coast Lightning for each of the 2025-2027 seasons, which maximises the economic and community return achieved through the continuation of this partnership.
[3061] 
[3062] 
[3063] ## FINANCE AND RESOURCING
[3064] 
[3065] The financial contribution to continue sponsorship of Sunshine Coast Lightning for seasons 2025-2027 would be funded from general rates.
[3066] 
[3067] CORPORATE PLAN
[3068] 
[3069] Corporate Plan Goal:
[3070] 
[3071] Our strong community
[3072] 
[3073] Outcome:
[3074] 
[3075] We serve our community by providing this great service
[3076] 
[3077] Operational Activity:
[3078] 
[3079] S8 - Sporting facilities - providing regional, district and community
[3080] 
[3081] sport and recreation facilities including aquatic centres,
[3082] 
[3083] showgrounds and multi-sports fields.
[3084] 
[3085] 
[3086] ## CONSULTATION
[3087] 
[3088] Councillor Consultation
[3089] 
[3090] All Councillors have been consulted in relation to this report.
[3091] 
[3092] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3093] 
[3094] <!-- PageNumber="OM Agenda Page 74 of 88" -->
[3095] 
[3096] 
[3097] <PageNum=75 />
[3098] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3099] 
[3100] <!-- PageHeader="24 JULY 2024" -->
[3101] 
[3102] Internal Consultation
[3103] 
[3104] • Group Executive - Economic & Community Development
[3105] 
[3106] • Head of Economic Development
[3107] 
[3108] • Manager, Sport and Community Venues
[3109] 
[3110] 
[3111] ## External Consultation
[3112] 
[3113] There has been no external consultation undertaken in relation to this report.
[3114] 
[3115] Community Engagement
[3116] 
[3117] Due to the confidential nature of the sponsorship negotiations with Sunshine Coast Lightning, no community consultation has been undertaken.
[3118] 
[3119] 
[3120] ## PROPOSAL
[3121] 
[3122] Council's initial support for the formation of Sunshine Coast Lightning in 2016 was based on the following assumptions and proposed benefits:
[3123] 
[3124] 1\. The growing awareness and recognition of Sunshine Coast as one of Queensland's and Australia's premier regional major events and sporting destination;
[3125] 
[3126] 2\. The enhancement of female sports on the Sunshine Coast and the increased community linkages by hosting our first national sporting team;
[3127] 
[3128] 3\. The ability to access State, Federal and Netball Australia funding for 'grass-roots' sports programs and infrastructure;
[3129] 
[3130] 4\. The ability to use the national sporting team presence to identify infrastructure gaps and secure funding for future sporting infrastructure needs on the Sunshine Coast;
[3131] 
[3132] 5\. The opportunity for tourism/sport leveraging in key target markets around Australia and New Zealand and increased linkages/performance between sports and tourism;
[3133] 
[3134] 6\. The ability to promote/market the Sunshine Coast to a national audience on television and print media that does not currently exist;
[3135] 
[3136] 7\. The potential for the Sunshine Coast to host the national/international "Netfest" Netball Australia competition at the Maroochydore Multi-Sports Complex; and
[3137] 
[3138] 8\. Securing our first national sporting team sets the stage for the Sunshine Coast over the next decade to build a business case to secure additional national sporting teams to be based in the region.
[3139] 
[3140] In addition, the economic benefits of securing a national netball league on the Sunshine Coast were modelled by AEC Group who determined that a professional netball team based in the region would generate:
[3141] 
[3142] • an estimated $4.9 million economic impact on the local economy, and
[3143] 
[3144] • $249,000 in visitor spend benefits from visiting teams and spectators annually.
[3145] 
[3146] In 2019, as the terms of the initial contract were reaching a conclusion, Council approved a two-year extension to the sponsorship with the same legacy benefit targets - as robust and significant gains had been achieved in all eight strategic categories.
[3147] 
[3148] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3149] 
[3150] <!-- PageFooter="OM Agenda Page 75 of 88" -->
[3151] 
[3152] 
[3153] <PageNum=76 />
[3154] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3155] 
[3156] <!-- PageHeader="24 JULY 2024" -->
[3157] 
[3158] In 2021, as the terms of the two-year extension were reaching a conclusion, Council approved a three-year extension (2022-2024). This contract was at an increased investment which saw Council receive significant increases to its contractual benefits received from the Lightning. These benefits focused on greater community engagement and facilitated Councils ability to amplifying community programs via Lightning ambassadors. These programs included, but weren't limited to, domestic & family violence prevention awareness, beach safety, and responsible pet ownership.
[3159] 
[3160] The current contract is set to expire at the conclusion of the 2024 season. As the Partnership matures and extends into its ninth year, Council's key objectives have evolved beyond the initial partnership legacy benefits. Key considerations when assessing the partnership renewal is the regions status as a delivery partner of the Brisbane 2032 Olympic and Paralympic Games, the legacy infrastructure that is associated with the Games and the opportunity for the Lightning to become an anchor tenant of the new indoor arena plus the region becoming a UNESCO Biosphere. When assessing the extension of the partnership, the desired 2025- 2027 strategic outcomes now include:
[3161] 
[3162] 1\. Optimise the national exposure of the Sunshine Coast region as a tourism destination through the audience and media reach of the high performing Sunshine Coast Lightning team;
[3163] 
[3164] 2\. Optimise the national awareness and exposure of the Sunshine Coast region as a UNESCO Biosphere and the uniqueness of the Sunshine Coast Lightning being the only professional sporting team to operate within a Biosphere. Leverage the partnership and promote Council's support in assisting the Sunshine Coast Lightning and Storm Group to become more sustainable and position the region as leaders and industry experts.
[3165] 
[3166] 3\. Assist Council to ensure as many opportunities as possible for local businesses to gain benefit through Sunshine Coast Lightning - as suppliers, sponsors or organisations that can promote their own growth and activity through the Sunshine Coast Lightning partnership;
[3167] 
[3168] 4\. Continue to leverage Sunshine Coast Lightning athletes to foster community engagement and pride;
[3169] 
[3170] 5\. Continue to foster youth involvement and developments via community engagement programs. The presence of successful athletes in the community has seen an increase in youth sports participation resulting in a range of social benefits;
[3171] 
[3172] 6\. The opportunity for tourism/sport leveraging in key target markets around Australia and New Zealand and increased linkages/performance between sports and tourism;
[3173] 
[3174] 7\. Continue to promote/market the Sunshine Coast to a national audience on television and print media;
[3175] 
[3176] 8\. Demonstration of Council's commitments to supporting professional sporting franchises positions the region in good stead to continue conversations with additional national and international leagues who have expressed interest in league expansion within the region; and
[3177] 
[3178] 9\. Communication and support for promotion of key Council messaging of all types to local fans and match audiences.
[3179] 
[3180] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3181] 
[3182] <!-- PageFooter="OM Agenda Page 76 of 88" -->
[3183] 
[3184] 
[3185] <PageNum=77 />
[3186] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3187] 
[3188] <!-- PageHeader="24 JULY 2024" -->
[3189] 
[3190] Partnership History:
[3191] 
[3192] Sunshine Coast Lightning commenced playing in the Australian Super Netball Championship's first season in 2017, becoming the most successful start-up sporting franchise in Australia's history by winning the premiership in their first two years (2017, 2018) and continuing to be successful on court competing in the Super Netball Finals in 2019 and 2020.
[3193] 
[3194] In addition to their on-court success, Sunshine Coast Lightning has achieved excellent results for the region throughout the current partnership with Council such as:
[3195] 
[3196] • Every home game at UniSC Arena selling out in 2017-2020 seasons (demonstrating very high levels of local and visitor support for the team), including more than 2,000 paid annual members;
[3197] 
[3198] • Generating an estimated economic impact (both direct and indirect) for the region of over $100 million over the first five years ($20 million per season generated by visitation, local business development and national marketing and broadcast reach for the region) (Source: Lucid Economics);
[3199] 
[3200] • Continually exceeding expectations of community engagement by the team - with Lightning players appearing for over 700 hours at community events during the 2023 season alone;
[3201] 
[3202] • An average of 48,000 viewers per match for Sunshine Coast Lightning matches and a total season attendance at games of 16,083 during the 2023 season;
[3203] 
[3204] • Active social media audience of 87,000 nationally (two thirds of which are from New South Wales and Victoria), receiving direct promotion of the Sunshine Coast region and a wide range of Council backed activities (such as tourism, events, investment or community); and
[3205] 
[3206] • Sunshine Coast Lightning has major partnerships with local businesses based on the Sunshine Coast, who are achieving strong results in promotion of their products and services to a national audience.
[3207] 
[3208] 
[3209] ## Legal
[3210] 
[3211] There are no legal implications relevant to this Council report. Once a draft sponsorship agreement is prepared it will be referred to Legal Services for review.
[3212] 
[3213] There are no barriers to entry or event obligations that would restrict access to a person or violate one of the 26 Human Rights.
[3214] 
[3215] 
[3216] ## Policy
[3217] 
[3218] The proposed sponsorship with the Sunshine Coast Lightning is in line with Council's Sponsorship Policy.
[3219] 
[3220] Risk
[3221] 
[3222] If Council were not to continue sponsorship of Sunshine Coast Lightning, there would likely be a significant impact on their continued operations.
[3223] 
[3224] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3225] 
[3226] <!-- PageFooter="OM Agenda Page 77 of 88" -->
[3227] 
[3228] 
[3229] <PageNum=78 />
[3230] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3231] 
[3232] <!-- PageHeader="24 JULY 2024" -->
[3233] 
[3234] Previous Council Resolution Ordinary Meeting 29 April 2021 (OM21/30)
[3235] 
[3236] That Council:
[3237] 
[3238] (a) receive and note the report titled "Sunshine Coast Lightning Sponsorship Proposal" and
[3239] 
[3240] (b) authorise the Chief Executive Officer to negotiate and execute a contract between Council and Sunshine Coast Lightning for each of the 2022-2024 seasons, which maximises the economic and community return achieved through the continuation of this partnership.
[3241] 
[3242] Ordinary Meeting 17 May 2018 (OM18/78)
[3243] 
[3244] That Council approve the sponsorship in accordance with the recommendations of the Sunshine Coast Events Board as discussed in confidential session in relation to Proposed Sporting Initiative.
[3245] 
[3246] Ordinary Meeting 21 April 2016 (OM16/64)
[3247] 
[3248] That Council approve the sponsorship in accordance with the recommendations of the Sunshine Coast Events Board as discussed in confidential session in relation to Proposed Sporting Initiative.
[3249] 
[3250] 
[3251] ## Related Documentation
[3252] 
[3253] Regional Economic Development Strategy 2013-2033 (2023 Refresh) Sunshine Coast Major Events Strategy 2018-2027 (2023 Refresh)
[3254] 
[3255] Critical Dates
[3256] 
[3257] Council's current sponsorship arrangement ends in August 2024. It is important that any future agreement with Sunshine Coast Lightning be finalised to allow them to adequately plan for the following season.
[3258] 
[3259] 
[3260] ## Implementation
[3261] 
[3262] Should the recommendation be accepted by Council, it is noted that the Chief Executive Officer will proceed to negotiate and finalise a sponsorship agreement with Sunshine Coast Lightning.
[3263] 
[3264] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3265] 
[3266] <!-- PageNumber="OM Agenda Page 78 of 88" -->
[3267] 
[3268] 
[3269] <PageNum=79 />
[3270] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3271] 
[3272] <!-- PageHeader="24 JULY 2024" -->
[3273] 
[3274] 8.7 VISIT SUNSHINE COAST FUNDING AND PERFORMANCE DEED
[3275] 
[3276] |||
[3277] | - | - |
[3278] | File No: | Council Meetings |
[3279] | Author: | Head of Economic Development Economic & Community Development Group |
[3280] | Attachments: | Att 1 - Draft Funding and Performance Deed between Sunshine Coast Council and Visit Sunshine Coast Limited - Confidential |
[3281] 
[3282] 
[3283] ## PURPOSE
[3284] 
[3285] The purpose of this report is to seek Council endorsement to allow the Chief Executive Officer to finalise and execute a Funding and Performance Deed (the Deed) negotiated with Visit Sunshine Coast Limited, commencing from execution.
[3286] 
[3287] Attachment 1 is confidential as it contains commercial in confidence information and confidential information relating to third parties. It also recognises that until Council makes a decision and a contract is executed, the content and timeframes are not final.
[3288] 
[3289] For these reasons, it is recommended that should Councillors wish to discuss the information in the confidential attachment, Council resolve to close the meeting under section 254J(3)(g) of the Local Government Regulation 2012 as the attachment contains information on negotiations relating to commercial matters involving the local government for which a public discussion would be likely to prejudice the interest of the local government.
[3290] 
[3291] Specifically, the matters that are considered confidential in the attachments and which would be the subject of discussion when the meeting is closed relate to the contract terms and conditions that are still the subject of negotiation with Visit Sunshine Coast Limited.
[3292] 
[3293] 
[3294] ## EXECUTIVE SUMMARY
[3295] 
[3296] Tourism is an integral part of our visitor economy, one of six economies identified in the Regional Economic Development Strategy (REDS). On the Sunshine Coast, the tourism sector provides:
[3297] 
[3298] • 9.3% of the Sunshine Coast Local Government Area's employment
[3299] 
[3300] · An estimated 15,957 Full Time Equivalent jobs, 9,034 directly
[3301] 
[3302] • Over $2.5 billion in visitor expenditure across the 2023 calendar year.
[3303] 
[3304] Council's investment in the tourism industry occurs predominantly through two mechanisms: through the execution of the Sunshine Coast Major Events Strategy 2018-2028 and sponsorship of major events with the objective of driving visitation during low tourism periods; and Council allocates funds to the Regional Tourism Organisation (RTO), Visit Sunshine Coast who deliver destination marketing strategies, industry development programs and support to local business to help achieve agreed visitor expenditure and visitation targets. Visit Sunshine Coast is one of 13 Regional Tourism Organisations within Queensland and is currently ranked in the top three in terms of performance by Tourism and Events Queensland.
[3305] 
[3306] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3307] 
[3308] <!-- PageNumber="OM Agenda Page 79 of 88" -->
[3309] 
[3310] 
[3311] <PageNum=80 />
[3312] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3313] 
[3314] <!-- PageHeader="24 JULY 2024" -->
[3315] 
[3316] A Funding and Performance Deed with Visit Sunshine Coast was first entered into by Council in July 2012, with an updated deed executed in 2018 and a Heads of Agreement executed in 2023.
[3317] 
[3318] Throughout the year, Visit Sunshine Coast is required to report back on a range of key performance indicators, culminating in an annual report presented to Council on the outcomes of Visit Sunshine Coast's Annual Tourism Plan each year.
[3319] 
[3320] 
[3321] ## OFFICER RECOMMENDATION
[3322] 
[3323] That Council:
[3324] 
[3325] (a) receive and note the report titled "Visit Sunshine Coast Funding and Performance Deed" and
[3326] 
[3327] (b) delegate authority to the Chief Executive Officer to finalise and execute the Funding and Performance Deed with Visit Sunshine Coast Limited.
[3328] 
[3329] 
[3330] ## FINANCE AND RESOURCING
[3331] 
[3332] Subject to execution, the new Funding and Performance Deed permits Visit Sunshine Coast to submit a budget request containing a minimum base contribution plus an increase for employee cost increases.
[3333] 
[3334] Financial commitment from Council to Visit Sunshine Coast in future years will be considered as part of annual budget deliberations and presentation of an annual tourism plan.
[3335] 
[3336] 
[3337] ## CORPORATE PLAN
[3338] 
[3339] Corporate Plan Goal:
[3340] 
[3341] Our resilient economy
[3342] 
[3343] Outcome:
[3344] 
[3345] 3.3 - Business retention and expansion
[3346] 
[3347] Operational Activity:
[3348] 
[3349] 3.3.3 - Continue to position Visit Sunshine Coast as a key delivery
[3350] 
[3351] partner for Council to deliver significant tourism expenditure
[3352] 
[3353] outcomes across the local government area including reviewing
[3354] 
[3355] the funding and performance deed.
[3356] 
[3357] 
[3358] ## CONSULTATION
[3359] 
[3360] Councillor Consultation
[3361] 
[3362] • Councillor R Natoli, Mayor
[3363] 
[3364] • Councillor J Broderick
[3365] 
[3366] • Councillor T Landsberg
[3367] 
[3368] • Councillor T Burns
[3369] 
[3370] • Councillor J Natoli
[3371] 
[3372] • Councillor W Johnston OAM
[3373] 
[3374] • Councillor T Bunnag
[3375] 
[3376] • Councillor M Suarez
[3377] 
[3378] • Councillor D Law
[3379] 
[3380] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3381] 
[3382] <!-- PageNumber="OM Agenda Page 80 of 88" -->
[3383] 
[3384] 
[3385] <PageNum=81 />
[3386] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3387] 
[3388] <!-- PageHeader="24 JULY 2024" -->
[3389] 
[3390] Internal Consultation
[3391] 
[3392] • Group Executive, Economic and Community Development
[3393] 
[3394] • Head of Economic Development
[3395] 
[3396] . Head of Tourism and Major Events, Economic Development
[3397] 
[3398] • Tourism Lead, Economic Development
[3399] 
[3400] 
[3401] ### External Consultation
[3402] 
[3403] . David Ryan, Chair, Visit Sunshine Coast Board
[3404] 
[3405] · Matt Stoeckel, Chief Executive Officer, Visit Sunshine Coast
[3406] 
[3407] Community Engagement
[3408] 
[3409] There was no community engagement conducted as part of this report. However, membership survey reporting found a satisfaction rating of 85% with service provision by Visit Sunshine Coast.
[3410] 
[3411] 
[3412] ## PROPOSAL
[3413] 
[3414] Since 2012, as part of Council's agreement with Visit Sunshine Coast, a Funding and Performance Deed has been in place to ensure that Council receives strategic plans, budgets and reports to ensure the funding is used effectively and that Council is receiving value for ratepayer money. An updated Funding and Performance Deed was executed in 2018 with Key Performance Indicators running until the end of 2023 which are due for renewal. On 4 April 2023, Council signed a Heads of Agreement with Visit Sunshine Coast with guiding principles on how the new funding deed would be structured. This was also based on the Investment in Tourism Review that Council commissioned in 2021.
[3415] 
[3416] The funding support of Visit Sunshine Coast is the main function that Council can perform to support an industry that drives economic growth and allows our visitor economy to continue to increase. The Regional Tourism Organisation model of destination marketing provides a region-wide focus, encouraging visitation from out of the region to all parts of the Sunshine Coast. On average, a visitor will spend $944 on their trip, staying on average 3.6 nights in region.
[3417] 
[3418] Visit Sunshine Coast's vision is to lead the tourism industry in promoting the Sunshine Coast's offering as a destination utilising the regions natural advantages, and to build a sustainable sector that will maximise the long-term benefits of the visitor economy to the region. Visit Sunshine Coast align closely with Council's strategic priorities and work closely with Council to promote growth in a key sector identified in the Regional Economic Development Strategy (REDS).
[3419] 
[3420] Key updates incorporated within the draft deed:
[3421] 
[3422] • Changes to reporting templates to provide greater insights for Sunshine Coast Council, whilst supporting visitor economy growth.
[3423] 
[3424] • Commitment to working through the recommendations from the Investment in Tourism Review.
[3425] 
[3426] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3427] 
[3428] <!-- PageNumber="OM Agenda Page 81 of 88" -->
[3429] 
[3430] 
[3431] <PageNum=82 />
[3432] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3433] 
[3434] <!-- PageHeader="24 JULY 2024" -->
[3435] 
[3436] • Alignment to the REDS and Major Events Strategy, supporting the Tourism Industry as a pivotal part of the local economy.
[3437] 
[3438] The draft Funding and Performance Deed has been developed by Council staff alongside Visit Sunshine Coast representatives through a series of meetings.
[3439] 
[3440] Legal
[3441] 
[3442] Council's Legal Services team has been involved throughout the negotiation process and provided input into the development of the proposed deed.
[3443] 
[3444] Policy
[3445] 
[3446] Until 2018/19, Council charged a Tourism and Major Events Levy that was paid by properties categorised as transitory accommodation, commercial and industrial and iconic tourism that benefited from tourism and major events.
[3447] 
[3448] In 2018/19 the Levy was incorporated into additional categories in the general rates. This amount has been increased in subsequent years in line with general rates until the 2023/24 year. The funding of Visit Sunshine Coast aligns with the Regional Economic Development Strategy, supporting the visitor economy.
[3449] 
[3450] Risk
[3451] 
[3452] Financial
[3453] 
[3454] By executing the draft Funding and Performance Deed, Council is committing to the ongoing funding of Visit Sunshine Coast. The financial exposure for Council has been mitigated through the annual budget process and termination clauses included within the deed.
[3455] 
[3456] Economic
[3457] 
[3458] A decrease or the cessation of funding would have significant ramifications on the visitor economy, impacting local businesses and nearly 10% of residents employed within the tourism sector. The Tourism industry provides $2.5 billion in spend within the Sunshine Coast Local Government Area and plays a significant role in delivering the Regional Economic Development Strategy. There would also be an impact on the feasibility of several major projects including infrastructure and hotel and tourism developments, which would affect the region's ability to successfully deliver elements of the Brisbane 2032 Olympic and Paralympic Games.
[3459] 
[3460] 
[3461] ## Previous Council Resolution
[3462] 
[3463] Ordinary Meeting 9 November 2017 (OM17/225)
[3464] 
[3465] That Council:
[3466] 
[3467] (a) receive and note the report titled "Funding and Performance Deed - Sunshine Coast Destination Limited (trading as Visit Sunshine Coast)"
[3468] 
[3469] (b) delegate authority to the Chief Executive Officer to finalise and execute the Funding and Performance Deed with Sunshine Coast Destination Limited and
[3470] 
[3471] (c) refer the funding agreement to the financial year 2018/19 budget review process.
[3472] 
[3473] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3474] 
[3475] <!-- PageFooter="OM Agenda Page 82 of 88" -->
[3476] 
[3477] 
[3478] <PageNum=83 />
[3479] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3480] 
[3481] <!-- PageHeader="24 JULY 2024" -->
[3482] 
[3483] Ordinary Meeting 26 February 2015 (OM15/28)
[3484] 
[3485] That Council delegate authority to the Chief Executive Officer to finalise and execute the Three- Year Funding and Performance Deed with Sunshine Coast Destination Limited and further undertake the actions as discussed in confidential session.
[3486] 
[3487] 
[3488] ## Related Documentation
[3489] 
[3490] Sunshine Coast Regional Economic Development Strategy 2013-2033 (2023 refresh) Sunshine Coast Major Events Strategy 2018-2028 (2023 refresh)
[3491] 
[3492] Critical Dates
[3493] 
[3494] There are no critical dates relevant to this report.
[3495] 
[3496] Implementation
[3497] 
[3498] Should the recommendation be accepted by Council, it is noted that the Chief Executive Officer will:
[3499] 
[3500] • Finalise and execute the agreement with Visit Sunshine Coast, and
[3501] 
[3502] • Refer to the funding deed during Council's annual budget process.
[3503] 
[3504] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3505] 
[3506] <!-- PageNumber="OM Agenda Page 83 of 88" -->
[3507] 
[3508] 
[3509] <PageNum=84 />
[3510] 
[3511] 
[3512] <PageNum=85 />
[3513] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3514] 
[3515] 24 JULY 2024
[3516] 
[3517] 9 NOTIFIED MOTIONS
[3518] 
[3519] 
[3520] ## 10 TABLING OF PETITIONS
[3521] 
[3522] Petitions only eligible for submission if:
[3523] 
[3524] \* Legible
[3525] 
[3526] \* Have purpose of the petition on top of each page
[3527] 
[3528] \* Contain at least 10 signatures
[3529] 
[3530] \* Motion limited to:
[3531] 
[3532] · Petition received and referred to a future meeting
[3533] 
[3534] · Petition received and referred to the Chief Executive Officer for report and consideration of recommendation
[3535] 
[3536] · Petition not be received
[3537] 
[3538] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3539] 
[3540] <!-- PageFooter="OM Agenda Page 85 of 88" -->
[3541] 
[3542] 
[3543] <PageNum=86 />
[3544] 
[3545] 
[3546] <PageNum=87 />
[3547] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3548] 
[3549] <!-- PageHeader="24 JULY 2024" -->
[3550] 
[3551] 
[3552] ## 11 CONFIDENTIAL SESSION
[3553] 
[3554] 11.1 CONFIDENTIAL - NOT FOR PUBLIC RELEASE - STRATEGIC LAND ACQUISITION - MOUNT MELLUM
[3555] 
[3556] |||
[3557] | - | - |
[3558] | File No: | Council Meetings |
[3559] | Authors: | Coordinator Biodiversity & Waterways Liveability & Natural Assets Group |
[3560] || Head of Strategic Property Business Performance Group |
[3561] 
[3562] In preparing this report, the Chief Executive Officer recommends it be considered confidential in accordance with Section 254J (3) (g) of the Local Government Regulation 2012 as it contains information relating to negotiations relating to a commercial matter involving the local government for which a public discussion would be likely to prejudice the interests of the local government.
[3563] 
[3564] The report is confidential in respect to the content and timeframes of negotiations with the landowners and recognising that, until Council makes a decision and the sale contracts are executed, the acquisition has no certainty.
[3565] 
[3566] Public disclosure at this time would potentially impact adversely on the finalisation of a current negotiation process and Council's ability to secure the land parcels at a price that represents the best value for the ratepayers of the region.
[3567] 
[3568] The report contains a recommendation to release details relating to the site location and price of the acquisitions once negotiations have been finalised and the transfer of the property title has been registered with the Titles Registry.
[3569] 
[3570] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3571] 
[3572] <!-- PageNumber="OM Agenda Page 87 of 88" -->
[3573] 
[3574] 
[3575] <PageNum=88 />
[3576] <!-- PageHeader="ORDINARY MEETING AGENDA" -->
[3577] 
[3578] 24 JULY 2024
[3579] 
[3580] 
[3581] ### 12 NEXT MEETING
[3582] 
[3583] The next Ordinary Meeting will be held on 29 August 2024.
[3584] 
[3585] 
[3586] ## 13 MEETING CLOSURE
[3587] 
[3588] <!-- PageFooter="Sunshine Coast Regional Council" -->
[3589] 
[3590] <!-- PageFooter="OM Agenda Page 88 of 88" -->
[3591] 
[3592] 
[3593] 
"""

In [2]:
markdown="""<PageNum=1 />
Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba Attachment 1 Detailed Officer Report


# PROPOSAL:

The application seeks approval for a Development Permit for Material Change of Use of Premises to Establish an Extension to Tourist Attraction (Car Museum).

The proposal seeks a 4 storey, 32,019m2 gross floor area vintage car museum building providing 879 car display bays (including turntables), 363 motorcycle display bays, a lobby area, a special display zone, lifts and a partial upper-level viewing terrace. The proposed car museum represents an extension to, and relocation of, the existing approved tourist attraction (MCU12/0190).

The vintage car museum, including the roof top terrace area, would be 18m high to the upper level of the building perimeter and 21.6m to the top of the roof terrace lobby and lift overruns from finished fill level.

An approved Multi-Level Car Park on the adjoining site (Resort sub-Precinct 1B) has been included in the application to demonstrate the availability of onsite car parking for patrons visiting the Palmer Resort, golf course and/or car museum. The approval was issued on 9 January 2024 and provides 498 car bays plus 34 motorcycle parking spaces at the location of the existing at grade resort car park within Sub-Precinct 1B (resort car parking facility) at the Palmer Resort.

Vehicle access to both the multi-level car park and vintage car museum is proposed from the main resort entrance from Warran Road and another existing entrance from Warran Road (currently used for back-of-house facilities). Pedestrian access to the car museum would be provided via one prominent entrance at the southwest ground floor.

Existing mature vegetation around the periphery of the resort would be retained to buffer and screen the proposal from external viewpoints and road. Additional landscaping is proposed along the front of the development, between the built form and the main internal private entry road to the resort. Additional landscaping is proposed along the eastern edge of the site to soften the built form and screen the structure.












<PageNum=2 />
20 JUNE 2024

Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba Attachment 1 Detailed Officer Report

<figure>

![](figures/0)<!-- FigureContent="The image is a detailed map layout showing a specific area with labeled structures. The map comprises two primary sections:

1. **Multi-Level Car Park (south)**: Positioned in the southern part of the map, this area is designated for parking. It appears structured and segmented to accommodate a large number of vehicles, with clear demarcations for different levels.

2. **Palmer Car Museum (north)**: Situated north of the Multi-Level Car Park, this part of the map represents the car museum. The museum building and its surroundings are depicted, indicating accessibility and pathways leading to the museum.

The layout also includes landscape elements such as trees, a water body labeled as a wetland, and various pathways and roads connecting the main structures. The design suggests a well-organized area optimized for both parking and visitors' access to the museum." --></figure>


<figure>

![](figures/1)<!-- FigureContent="The image illustrates a proposed design for a vintage car museum, as indicated in the caption "Figure 2. Proposed vintage car museum - perspective 1". The building is modern and sleek with a multi-story glass facade. There are large windows at the ground floor, providing a clear view of the interior. The entry to the museum is marked by the word "ENTRY" displayed prominently above the doorway.

The name "Palmer Auto Museum" is visible both near the entry and on a sign situated among lush, landscaped greenery in front of the building. Palm trees and other greenery are prominent in the landscape around the museum, suggesting a tropical or warm climate. 

A crosswalk is visible in the foreground with a person walking across it, and a car is driving on a road that leads directly to the museum's entrance. The sky is bright with scattered clouds, indicating a sunny day. Overall, the scene is inviting with well-maintained landscaping and a modern architectural design." --></figure>


Detailed Officer Recommendation Report








<PageNum=3 />
20 JUNE 2024

Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba Attachment 1 Detailed Officer Report

<figure>

![](figures/2)<!-- FigureContent="The image depicts a modern architectural setting with a roadway leading into what appears to be a large, contemporary building complex. Palm trees and lush green landscaping line both sides of the road, adding a touch of nature to the modern, urban environment. The building has a sleek, metal and glass facade with a visually appealing pattern of vertical lines. The sky above is bright with scattered clouds, suggesting a clear day. There is a single white car driving along the curved road toward the building. The overall ambiance of the image is clean, modern, and well-maintained." --></figure>


Figure 3. Proposed vintage car museum - perspective 2

<figure>

![](figures/3)<!-- FigureContent="The image appears to depict the exterior of a modern building with a prominent entrance labeled "ENTRY." There are several palm trees and lush green shrubbery surrounding the pathway leading to the entrance, contributing to a tropical and inviting atmosphere. The building has a contemporary architectural style with clean lines and geometric shapes. There are people walking towards the entrance and along the pathway, adding a sense of activity and scale. The sky above is partly cloudy, and the overall environment looks well-maintained and aesthetically pleasing." --></figure>


Figure 4. Proposed vintage car museum - perspective 3

Figure 5. Multi-level car park (perspective)

Detailed Officer Recommendation Report







<figure>

![](figures/4)<!-- FigureContent="The image depicts a modern building, likely a library given the visible signage. The building's design incorporates clean lines and a mix of materials, including large glass windows and dark paneling. 

In the foreground, there is a driveway with two lanes marked by white arrows directing traffic flow. Adjacent to the driveway is a sidewalk or walkway bordered by landscaped greenery, including various plants and palm trees, creating a lush and inviting environment. 

The sky is clear with a few scattered clouds, suggesting a pleasant weather day. A person is walking towards the entrance of the building, and a white car is parked nearby. The overall atmosphere of the image is calm, modern, and welcoming." --></figure>



<PageNum=4 />


Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba Attachment 1 Detailed Officer Report

<figure>

![](figures/5)<!-- FigureContent="The image depicts a detailed architectural ground floor plan for the Palmer Car Museum. The layout is clearly labeled with various functional areas, pathways, and different structures within the museum complex.

### Key Elements of the Ground Floor Plan:

**Building Structure:**
- The main building is a rectangular structure.
- Inside the museum, different areas are demarcated: general display areas, amenities, lobby, and entry/exit points.

**Parking and Access:**
- There's an entry spot labeled "TRUCK ENTRY" presumably for deliveries.
- The south-west side has a main entry labeled "car park" or "entry."

**Displays and Spaces:**
- The central part of the building has "TT" indications which could represent specific display tables or turntables.
- Multiple display areas with ample spacing for exhibits.

**Landscaping and Surroundings:**
- There are bioretention basins around the building for environmental management.
- Trees and greenery surround the museum, adding aesthetic value and likely providing environmental benefits.
- The right side of the plan features an existing lake, which harmonizes with the museum structure.

**Other Facilities:**
- The design includes amenities areas and potential restrooms or locker spaces on the right end of the building.
- The plan suggests paths for pedestrian movement around the site.

The plan includes various notes on infrastructure like water management, including stormwater pipes and retaining walls for optimal environmental integration." --></figure>


<figure>

![](figures/6)<!-- FigureContent="The image shows a detailed architectural floor plan for a building. The plan appears to include different structural elements, dimensions, and annotations indicating various features of the layout. Here are some key elements identified in the plan:

1. **Grid Layout**: The plan is gridded with lines and labeled columns, helping to indicate precise measurements and layout sections.
  
2. **Specific Measurements**: There are multiple instances of dimensions specified, like "8,400" repeated across the top, likely indicating distances in millimeters.

3. **Areas and Spaces**:
   - There is an annotated space labeled "amenite" with a specified area of "47 m2".
   - Adjacent spaces are marked with symbols like "TT" which might indicate specific features or room types.

4. **Annotations and Symbols**:
   - Various symbols are used throughout, such as circular and rectangular icons, perhaps indicating structural elements like columns, staircases, and amenities.
   - Terms such as "solid wall facing golf hole," "impact-resistant transparent polycarb light & ventilation 'bands' to each floor," and "solid boundary" indicate architectural and structural features.

5. **Stairs and Exits**:
   - There are labels indicating "feature stairs" and "fire stairs" along with arrows suggesting directions or additional notes.
   
6. **Boundaries**:
   - The plan marks the "site boundary," indicating the outer limits of the area covered by the plan.
   - Angled lines representing "line of awning above" suggest an overhead feature or extension.

7. **Text Annotations**:
   - Various text annotations provide details such as the Relative Level (RL) at "RL +13.300 (nom.)" suggesting elevation or floor height.
   - References to materials and external features like "impact-resistant transparent polycarb light & ventilation."

The layout seems to be an organized and annotated architectural plan that aids in the construction or understanding of the specified floor, containing details necessary for structural, design, and planning purposes." --></figure>


1 car museum - first floor plan

<figure>

<figcaption>

Figure 7. Palmer Car Museum (Level 1)

</figcaption>

![](figures/7)<!-- FigureContent="The image is an architectural floor plan marked "Figure 7. Palmer Car Museum (Level 1)." The plan shows the layout and structural elements of the first level of the Palmer Car Museum.

Key features of the floor plan:
1. **Overall Layout**: The plan is drawn in what appears to be a grid layout, which helps in the precise placement and dimensions of each element in the design.
2. **Boundaries and Structure**:
   - The outer boundaries of the structure are marked clearly. The site boundaries are also indicated.
   - There is a "solid wall facing golf hole" on one side, suggesting the museum is adjacent to a golf course.
3. **Entrances and Exits**:
   - There are designated entrance and exit points which are emphasized, particularly fire staircases and feature stairs.
4. **Rooms and Areas**:
   - The plan includes various rooms and areas such as an "amenities" room (47 sq. meters), "lifts" area (15 sq. meters), and other possibly labeled as 'TT'.
5. **Facilities**:
   - Elevators/lifts, stairs, and passageways are labeled.
   - Feature stairs and fire staircases are marked at different points in the layout.
6. **Additional Notes**:
   - There are annotations indicating some specific design elements like "impact-resistant polycarb light & ventilation 'bands' to each floor."
   - The grid dimensions and specific measurements are labeled around the perimeter, such as "8.400" which likely denotes specific spacing or dimensions.

Overall, the document provides a detailed and measured layout of the first floor of the Palmer Car Museum, useful for construction, design planning, and reference purposes." --></figure>


car museum - second floor plan

Figure 8. Palmer Car Museum (Level 2)

Detailed Officer Recommendation Report






:selected: :selected: :selected: :selected: :selected: :selected: :selected: :selected: :selected: :selected: :selected: :selected: :selected: :selected:

<PageNum=5 />


Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba Attachment 1 Detailed Officer Report

<figure>

![](figures/8)<!-- FigureContent="The image depicts an architectural or engineering floor plan for a multi-level structure. Here are the notable components and annotations found in the plan:

1. **Grids and Measurements:**
   - The plan is overlaid with a grid system, including numerous lines and points for precise measurement.
   - The measurements are listed in meters (e.g., 8.430, 7.920).

2. **Design Elements:**
   - The plan shows multiple levels with various features and sections labeled.
   - Features labeled include "solid wall facing golf hole," "impact-resistant transparent polycarb light & ventilation bands to each floor," "popcarb bands," "future stairs," and "future escalators."

3. **Areas and Facilities:**
   - Specific areas are marked such as "7.91m²" (square meter area).
   - A section labeled "Amenities 47.8m²" suggests the presence of a facilities or amenities area.
   - There are various rooms or designated spaces, indicated by the outline and text labels provided.

4. **Symbols and Notations:**
   - The plan has a variety of symbols and markings, including circles with letters inside (e.g., "T"), which typically represent specific fixtures or components.
   - Hatched areas indicate different materials or designations such as "solid wall," "white mesh screening."

5. **Structural Elements:**
   - Staircases are indicated, with labels for "first staircase" and "future stairs."
   - Lines and cross hatching indicate structural features or boundaries.

6. **Boundaries:**
   - The plan includes dashed lines indicating site boundaries and lines of awning above.

Overall, the image is a detailed blueprint, illustrating the layout, measurement, and various architectural and structural elements of a planned building or facility." --></figure>


car museum - third floor plan

<figure>

![](figures/9)<!-- FigureContent="The image contains architectural drawings of the Palmer Car Museum which includes the roof plan and multiple elevations of the building.

1. **Roof Plan**:
   - The roof is irregularly shaped, resembling an elongated trapezoid with a narrower section on the right side.
   - The plan shows various functional components, including:
     - Photovoltaic panels, indicated on both sides of the roof.
     - Metal roof sheeting, polycarbonate roof lights, and roof lap joint system.
     - A high-level MRS awning over the loading dock in the center.
     - Screeds and a perimeter of terraces along the edges.
     - Service enclosures and other labeled mechanical elements.
     - Dimensions and annotations marking the placement of different elements on the roof.

2. **Elevations**:
   - **East Elevation**: Demonstrates a front view of the building, detailing the roof terrace lobby, mural walls, plantings, and various levels annotated with measurements.
   - **West Elevation**: Features similar descriptions to the east elevation, with added details like entry awnings, auto doors, metal sheathing, planter boxes, and ventilation panels.
   - **South Elevation**: A side view highlighting the polycarbonate roof lights and terrace areas.
   - **North Elevation**: Contains descriptions similar to the other elevations, mentioning the roof terrace lobby, metal cladding, and parking drive-in ramp.

Each elevation is carefully annotated with heights, levels, and specific features of the building. The diagram provides a comprehensive visual description of the roof structure and facade detailing of the Palmer Car Museum." --></figure>


Detailed Officer Recommendation Report






:selected: :selected: :selected: :selected: :selected: :selected: :selected: :selected: :selected: :unselected: :unselected: :unselected: :unselected: :unselected: :selected: :selected: :unselected:

<PageNum=6 />
20 JUNE 2024

Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba Attachment 1 Detailed Officer Report

<figure>

![](figures/10)<!-- FigureContent="This image appears to be an architectural cross-section diagram of a multi-level building. The diagram includes various annotations and dimensions to aid in the understanding of the building's structure and design. Here are some key features:

1. **Levels/Floors**: The building has multiple levels, including a ground floor and what appears to be several upper floors, indicated by the annotations (GFL, FFL, etc.).

2. **Roof**: The roof is landscaped, featuring shade trees and planters. It has various annotations regarding drainage and cladding details.

3. **Facade**: The building features full-height glazing (glass windows) on its facade. There are also horizontal bands, likely detailing materials or architectural features.

4. **Vegetation**: There's landscaping vegetation indicated behind retaining walls at the lower level.

5. **Materials**: 
   - The building includes cages labeled “CAGE (FSC)” and panels like “SCR” (perforated mesh screening) and “EP2” finish.
   - There appears to be the use of aluminum cladding, as indicated by the protruding aluminum clad awning and other aluminum framed glazing.

6. **Annotations and Dimensions**: Various architectural annotations and dimensions are included to specify height levels, the extent of materials, and other construction requirements.

7. **Ground Level Features**: The ground level shows access points and possibly pedestrian pathways or entrances.

This image is a technical blueprint used for architectural planning and construction detailing." --></figure>



DO.2003

\----

Figure 12. Elevation Details - North

<figure>

![](figures/11)<!-- FigureContent="This image is an architectural elevation drawing of the south side of a building labeled "PALMER AUTO MUSEUM." The drawing shows detailed external features and dimensions, and provides specific material and color notations.

- The upper section of the facade has a white paneling with a black aluminum plant logo.
- Below the white paneling, there is a horizontal band that reads "PALMER AUTO MUSEUM" in bold letters.
- This section is further supported by vertical green panels.
- To the left side, there is an entrance with detailed annotations specifying the materials and dimensions.
- The roof has metal sheeting with gutters and eaves.
- There are notes indicating various heights at specific points (e.g., 32,380 FFL, 29,350 FFL).
- At the very bottom, there is a grassy area labeled as "landscape vegetation in front of retaining wall".

The drawing was designed by "BILLY DAWSON ARCHITECTS PTY LTD" and is labeled as "detail elevation - south." The document is dated July 2023 and includes a scale and drawing details." --></figure>


Detailed Officer Recommendation Report






:selected:

<PageNum=7 />
20 JUNE 2024

Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba Attachment 1 Detailed Officer Report

<figure>

![](figures/12)<!-- FigureContent="The image depicts a large building, which appears to be a modern multi-story structure partially covered with vegetation and trees on the rooftop. It is situated in a lush, green area with dense trees surrounding it. In the background, there is a prominent hill or small mountain covered in greenery. The overall scene appears to be tranquil and is set in a natural environment with clear skies." --></figure>


Figure 14. 3D Render 1

<figure>

![](figures/13)<!-- FigureContent="The image depicts an aerial view of a modern building complex nestled within a lush, forested area. The structure appears to be a large, multi-story building with green roofing and rooftop vegetation, harmoniously blending with the surrounding landscape. The building features several architectural elements, such as solar panels on the roof and a unique design with curved supports. In the foreground, there is an open, paved area with evenly spaced trees planted in it. Beyond the complex, there is a substantial expanse of greenery, possibly a park or golf course, leading to a distant view of a coastline and ocean under a clear sky." --></figure>


Figure 15. 3D Render 2

Detailed Officer Recommendation Report








<PageNum=8 />
20 JUNE 2024

Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba Attachment 1 Detailed Officer Report

<figure>

![](figures/14)<!-- FigureContent="The image showcases an aerial view of a modern, multi-storey building surrounded by lush greenery. The structure appears to be large and contemporary in design, with a flat roof and expansive windows. The roof also features numerous solar panels.

The building is set within a verdant environment; it's adjacent to what seems to be a large open field or golf course, bordered by forests or wooded areas. In the distance, there are suburban developments with numerous houses dotting the landscape, and hills or low mountains can be seen on the horizon under a partly cloudy sky. The overall ambiance is serene and picturesque, blending natural and built environments harmoniously." --></figure>


Figure 16. 3D Render 3

The following table describes the key development parameters for the proposal:

<table>
| MATERIAL CHANGE OF USE | DEVELOPMENT PARAMETERS ||
|| Required | Proposed |
| - | - | - |
| Gross Floor Area | 200m2 | 32,019m2 (Proposed Car Museum Component) |
| Building Height/Storeys | 8.5m | 21.6m |
| Setbacks | Not specified | 6.3m to the northern boundary. From 6.3m to 15.8m to the eastern boundary and 7.3m to the southern boundary. On the western side, building setbacks range from 3.3m to 18.47m to the boundary. |
| Site Cover | Not specified | Approx. 68% (Proposed Car Museum on Lot 28) |
| Landscaping | The visual appearance of buildings is softened through the retention of the existing mature landscaping. | Existing mature vegetation between the boundaries of the Master Plan area retained. Landscaping proposed along the front of the development and along the eastern edge of the site. |
</table>

Detailed Officer Recommendation Report








<PageNum=9 />


Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba Attachment 1 Detailed Officer Report

SITE DETAILS: Site Features and Location

<table>
|||
| - | - |
| SITE AND LOCALITY DESCRIPTION ||
| Land Area: | Lot 28 - 1.163 ha (proposed car museum site) Lot 10 - 14.49 ha (approx. 1.27 ha carpark site) |
| Existing Use of Land: | Vacant site- temporary marquees |
| Road Frontage: | N/A |
| Significant Site Features: | Site has been cleared, levelled and filled. The proposed car museum site currently accommodates three temporary marque structures. |
| Topography: | Flat |
| Surrounding Land Uses: | Within the Master Plan Area, the development site is surrounded on all sides by golf course. Back-of-house facilities are located to the north and central resort facilities and accommodation buildings located to the south. The Palmer Coolum Resort is bordered by low density residential land to the north, west and south in the form of traditional residential lots, accommodating mainly detached dwellings. There is a medium density residential precinct comprising detached houses, dual occupancies and low-rise multiple dwelling units south of Tanah Street West at the southern extremity of the golf course. The Mount Coolum Local Centre is located at the intersection of the David Low Way and Suncoast Beach Drive to the south of the site. Mount Coolum National Park is located south of the resort precinct and is the dominant landform in the locality. The parabolic dune and Precinct 3 (Beachside) are located on the eastern side of the David Low Way. |
</table>

The location of the subject site in relation to its surrounds is shown below:

Detailed Officer Recommendation Report








<PageNum=10 />
20 JUNE 2024

Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba Attachment 1 Detailed Officer Report

<figure>

![](figures/15)<!-- FigureContent="This image is an aerial view of a parking lot and nearby structures, likely industrial or large storage buildings. It appears to be overlaid with boundary lines in blue and brown. The area within the boundaries includes:

- Two large, long rectangular structures with white roofs.
- A sizeable parking lot containing numerous vehicles, located towards the left part of the image.
- Surrounding greenery, suggesting a forested or landscaped area, possibly parkland or a golf course given the presence of sand traps and putting greens in the upper and lower right corners of the image.
- The boundary lines appear to demarcate specific sections of the area, with the blue line encompassing both the parking lot and the buildings." --></figure>


Figure 17. Aerial view

Detailed Officer Recommendation Report








<PageNum=11 />


Ordinary Meeting Agenda Item 8.1 Development Application - Material Change of Use to Establish an Extension to Tourist Attraction (Car Museum) - Warran Road, Yaroomba

Attachment 1 Detailed Officer Report

<figure>

![](figures/16)<!-- FigureContent="The image is a locality plan, marked as "Figure 18. Locality plan." It shows a detailed map of an area, highlighting various streets, buildings, and natural features. The map includes several residential zones with a dense network of roads. 

Key features:
- A network of roads and buildings that signify urban development.
- A highlighted area in the center of the map, bordered in blue with a star at its center, possibly indicating a significant or central point of interest.
- The Palmer Coolum Resort Golf Club is marked southwest of the highlighted area with adjacent water bodies and green spaces.
- The eastern edge of the map features a coastline, suggesting the presence of a beach or ocean.
- Major roads such as Warran Road and David Low Way are labeled and visible, facilitating access to different parts of the locality. 

This plan helps in visualizing the spatial relationship between various elements within the area and clearly demarcates key locations and landmarks." --></figure>



## Development History of Site

Existing Use rights

The original Hyatt Health Resort site operates under the cumulative effect of existing use rights pursuant to several historic planning permits including, but not limited to:

. 14 May 1987 initial rezoning approval to rezone from Rural at the Hyatt site B to Special Facility Zone (Health Management Complex & Associated Accommodation and Ancillary Facilities) as identified on Sheet 39 Accompanied by Rezoning Deed dated 27/08/87 (14/05/1987).

. 29 October 1987 - A second rezoning (150 ha) from part Special Facilities, Part Rural B to "Special Facilities - Health Management Complex, Four (4) Storey Hotel Resort of 300 Rooms, Associated Accommodation, Golf Course and Ancillary Facilities" zone was gazetted over an expanded site in accordance with Drawings No 1-4 dated 03/02/1986 Job 85018 as amended by Master Plan 85018 dated 12/12/1986 and by the inclusion of Drawing No 7/85018 "Beach Hotel" dated 23/11/1986 (original A1 on file 20/5/072); Drawing No 8 / 85018 "villas" dated December 1986, which were gazetted and formed part of the scheme maps. The approval was accompanied by a Rezoning Headworks Agreement.

. 14 August 1998 - A third rezoning (151.39 ha) from Residential A, Rural B & Special Facilities to "Special Facilities - Recreation and Tourism Resort, Conference Centre, Special Events & Ancillary Facilities" zone based on a new Plan of Development (Hyatt Regency Coolum Plan of Development and Plan of Development Schedule) dated January 1998, which was gazetted and formed part of the scheme maps. """

In [5]:

class Section(BaseModel):
    title: str = Field(description="main topic of this section of the document (very descriptive)")
    start_index: int = Field(description="line number where the section begins (inclusive)")
    end_index: int = Field(description="line number where the section ends (inclusive)")


class StructuredDocument(BaseModel):
    """obtains meaningful sections, each centered around a single concept/topic"""
    sections: List[Section] = Field(description="a list of sections of the document")

In [31]:
# prompt_1="""Read the document below and extract a StructuredDocument object from it where each section of the document is centered around a single concept/topic. Whenever possible, your sections (and section titles) should match up with the natural sections of the document (i.e. Introduction, Conclusion, References, etc.). Sections can vary in length, but should generally be anywhere from a few paragraphs to a few pages long.
# Each line of the document is marked with its line number in square brackets (e.g. [1], [2], [3], etc). Use the line numbers to indicate section start and end.
# The start and end line numbers will be treated as inclusive. For example, if the first line of a section is line 5 and the last line is line 10, the start_index should be 5 and the end_index should be 10.
# The sections MUST be non-overlapping and cover the entire document. In other words, they must form a partition of the document.
# Section titles should be descriptive enough such that a person who is just skimming over the section titles and not actually reading the document can get a clear idea of what each section is about.
# Note: the document provided to you may just be an excerpt from a larger document, rather than a complete document. Therefore, you can't always assume, for example, that the first line of the document is the beginning of the Introduction section and the last line is the end of the Conclusion section (if those section are even present).
#  """
prompt_1="""Read the document below and extract a StructuredDocument object from it where each section of the document is centered around a single concept/topic. Whenever possible, your sections (and section titles) should match up with the natural sections of the document (i.e. Introduction, Conclusion, References, etc.). Sections can vary in length, but should generally be anywhere from a few paragraphs to a few pages long.

Each line of the document is marked with its line number in square brackets (e.g. [1], [2], [3], etc). Use the line numbers to indicate section start and end. The start and end line numbers will be treated as inclusive. For example, if the first line of a section is line 5 and the last line is line 10, the start_index should be 5 and the end_index should be 10.

The sections MUST be non-overlapping and cover the entire document. In other words, they must form a partition of the document. Section titles should be descriptive enough such that a person who is just skimming over the section titles and not actually reading the document can get a clear idea of what each section is about.

Note: the document provided to you may just be an excerpt from a larger document, rather than a complete document. Therefore, you can't always assume, for example, that the first line of the document is the beginning of the Introduction section and the last line is the end of the Conclusion section (if those section are even present).

Present the sections in the order they appear in the document.
"""

In [32]:
prompt_2="""You are tasked with performing a document analysis for the purpose of breaking it down into its logical units. You assume that each document consists of multiple sections with each section in turn consisting of sub-sections, which are further broken down into logical units. Logical units represent the smallest hierarchical unit in a document and consists of multiple sentences that are logically interlinked and convey an information or idea. Each section must be analyzed for its structure individually and may consist of one or multiple sub-sections and logical units. Your output consists of a JSON of the document's outline that follows the logic of the JSON schema provided. You must strictly consider all of the document's content in your analysis. JSON schema:
{
    "hierarchy_level": 0,
    "title": "Document title (verbatim)",
    "children": [
        {
            "hierarchy_level": 1,
            "title": "Section title (verbatim, include preceding letter, number or roman numeral if applicable)",
            "start_line":"number of the line where the section starts based on the unique line identifier in the document",
            "children": [
                {
                    "hierarchy_level": 2, 
                    "title": "Sub-section title (verbatim, include preceding letter, number or roman numeral if applicable, N/A if no title)",
                    "start_line":"number of the line where the sub-section starts based on the unique line identifier in the document",
                    "children": [
                    {
                        "hierarchy_level": 3, 
                        "title": "Sub-section title (verbatim, include preceding letter, number or roman numeral if applicable, N/A if no title)",
                        "start_line":"number of the line where the sub-section starts based on the unique line identifier in the document",
                        "children": []
                    }
                    ]
                }
            ]
        }
    ]
}

"""

In [33]:
generation_config={

    "response_mime_type": "application/json",
    "response_schema": StructuredDocument
}
lm1 = ChatGoogleGenerativeAI(model="gemini-1.5-pro-exp-0827",google_api_key=key,generation_config=generation_config)

In [8]:
generation_config={

    "response_mime_type": "application/json"
}
lm2 = ChatGoogleGenerativeAI(model="gemini-1.5-pro-exp-0827",google_api_key=key,generation_config=generation_config)

In [37]:
prompt1=prompt_1+"\n"+numbered_markdown
prompt2=prompt_2+"\n"+numbered_markdown




In [38]:
answer=lm1.invoke(prompt1)

In [39]:
print(answer.content)

```json
[
  {
    "section_title": "Meeting Agenda",
    "start_index": 8,
    "end_index": 12
  },
  {
    "section_title": "Table of Contents",
    "start_index": 25,
    "end_index": 47
  },
  {
    "section_title": "Meeting Items - Tabling of Petitions, Confidential Session, Next Meeting, Meeting Closure",
    "start_index": 57,
    "end_index": 83
  },
  {
    "section_title": "Ordinary Meeting Notice",
    "start_index": 103,
    "end_index": 122
  },
  {
    "section_title": "Declaration of Opening",
    "start_index": 133,
    "end_index": 136
  },
  {
    "section_title": "Welcome and Opening",
    "start_index": 137,
    "end_index": 138
  },
  {
    "section_title": "Record of Attendance and Leave of Absence",
    "start_index": 139,
    "end_index": 140
  },
  {
    "section_title": "Receipt and Confirmation of Minutes",
    "start_index": 141,
    "end_index": 144
  },
  {
    "section_title": "Mayoral Minute",
    "start_index": 145,
    "end_index": 146
  },
  {
    "sec

In [36]:
print(answer.content)

```json
{
    "hierarchy_level": 0,
    "title": "Ordinary Meeting Wednesday, 24 July 2024",
    "children": [
        {
            "hierarchy_level": 1,
            "title": "TABLE OF CONTENTS",
            "start_line": 25,
            "children": []
        },
        {
            "hierarchy_level": 1,
            "title": "1 DECLARATION OF OPENING",
            "start_line": 133,
            "children": []
        },
        {
            "hierarchy_level": 1,
            "title": "2 WELCOME AND OPENING",
            "start_line": 137,
            "children": []
        },
        {
            "hierarchy_level": 1,
            "title": "3 RECORD OF ATTENDANCE AND LEAVE OF ABSENCE",
            "start_line": 139,
            "children": []
        },
        {
            "hierarchy_level": 1,
            "title": "4 RECEIPT AND CONFIRMATION OF MINUTES",
            "start_line": 141,
            "children": []
        },
        {
            "hierarchy_level": 1,
            "t

In [ ]:
json_string = a.content[7:-3]
print(json_string)
# convert json_string to json object
import json
json_object = json.loads(json_string)
print(json_object)

In [ ]:

def get_line_with_number(text: str) -> List[str]:
    lines = text.split('\n')
    return [f"[{i+1}] {line}" for i, line in enumerate(lines)]


In [ ]:
import re
def extract_sections(json_data, numbered_markdown):
    sections = []
    lines = numbered_markdown.split('\n')
    line_numbers = {}
    current_line_number = 0

    # First, map actual line numbers to their indices
    for i, line in enumerate(lines):
        match = re.match(r'\[(\d+)\]', line)
        if match:
            line_numbers[int(match.group(1))] = i
            current_line_number = int(match.group(1))
        else:
            line_numbers[current_line_number] = i

    for item in json_data:
        section_title = item['section_title']
        start_index = item['start_index']
        end_index = item['end_index']
        
        # Find the actual start and end indices in the lines list
        start_line = min(line_numbers.keys(), key=lambda x: abs(x - start_index))
        end_line = min(line_numbers.keys(), key=lambda x: abs(x - end_index))
        
        start_idx = line_numbers[start_line]
        end_idx = line_numbers[end_line]
        
        # Extract all lines between start_idx and end_idx, inclusive
        section_content = lines[start_idx:end_idx+1]
        
        # Create a section dictionary
        section = {
            'title': section_title,
            'content': '\n'.join(section_content)
        }
        
        sections.append(section)
    
    return sections

In [ ]:
sections=extract_sections(json_object,sample_text)